In [1]:
import pandas as pd
import numpy as np
from pandasql import  sqldf


def make_select(query: str) -> pd.DataFrame | None:
   return sqldf(query)

In [11]:
data = [
    [1, 70000, 'Sales'], 
    [2, 80000, 'Sales'], 
    [3, 80000, 'Sales'], 
    [4, 90000, 'Sales'], 
    [5, 55000, 'IT'], 
    [6, 65000, 'IT'], 
    [7, 65000, 'IT'], 
    [8, 50000, 'Marketing'], 
    [9, 55000, 'Marketing'], 
    [10, 55000, 'HR']
]

employees = pd.DataFrame(data, columns=['emp_id', 'salary', 'dept'])

query = """  

SELECT 
emp_id, dept
FROM  
(SELECT 
*,
DENSE_RANK() OVER(PARTITION BY dept ORDER BY salary DESC) AS ranking
FROM employees) AS sub_query
WHERE ranking = 2
ORDER BY emp_id
"""

make_select(query)

,emp_id,dept
0,2,Sales
1,3,Sales
2,5,IT
3,8,Marketing


In [15]:
data = [[1, 8], [2, 8], [3, 8], [4, 7], [5, 9], [6, 9]]
employee = pd.DataFrame(data, columns=['employee_id', 'team_id']).astype({'employee_id':'Int64', 'team_id':'Int64'})

query = """  

SELECT 
employee_id,
COUNT(employee_id) OVER(PARTITION BY team_id) AS team_size
FROM employee

"""

make_select(query)

,employee_id,team_size
0,4,1
1,1,3
2,2,3
3,3,3
4,5,2
5,6,2


In [24]:
data = [[10, 1, 500], [11, 2, 1000], [12, 3, 800], [13, 4, 1000]]
orders = pd.DataFrame(data, columns=['order_id', 'item_count', 'order_occurrences']).astype({'order_id':'Int64', 'item_count':'Int64', 'order_occurrences':'Int64'})


query = """   

SELECT  ROUND(SUM(item_count*order_occurrences) *1.0/SUM(order_occurrences), 2) AS average_items_per_order  FROM orders

"""

make_select(query)

,average_items_per_order
0,2.7


In [28]:
data = [['1', '2018', '50'], ['1', '2021', '30'], ['1', '2020', '70'], ['2', '2021', '-50'], ['3', '2018', '10'], ['3', '2016', '50'], ['4', '2021', '20']]
customers = pd.DataFrame(data, columns=['customer_id', 'year', 'revenue']).astype({'customer_id':'Int64', 'year':'Int64', 'revenue':'Int64'})


query = """  

SELECT 
customer_id,
SUM(revenue)
FROM customers
WHERE year = 2021
GROUP BY customer_id
HAVING SUM(revenue) >= 0
"""

make_select(query)

,customer_id,SUM(revenue)
0,1,30
1,4,20


In [ ]:
data = [['LCHouse1', 1, 1], ['LCHouse1', 2, 10], ['LCHouse1', 3, 5], ['LCHouse2', 1, 2], ['LCHouse2', 2, 2], ['LCHouse3', 4, 1]]
warehouse = pd.DataFrame(data, columns=['name', 'product_id', 'units']).astype({'name':'object', 'product_id':'Int64', 'units':'Int64'})
data = [[1, 'LC-TV', 5, 50, 40], [2, 'LC-KeyChain', 5, 5, 5], [3, 'LC-Phone', 2, 10, 10], [4, 'LC-T-Shirt', 4, 10, 20]]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'Width', 'Length', 'Height']).astype({'product_id':'Int64', 'product_name':'object', 'Width':'Int64', 'Length':'Int64', 'Height':'Int64'})


query = """ 

SELECT name AS warehouse_name, SUM(volume) AS volume FROM (SELECT 
*,w.units * (p.Width*p.Length*p.Height) AS volume
FROM warehouse w INNER JOIN products p ON w.product_id = p.product_id) AS sub_query
GROUP BY name

"""

make_select(query)

,name,volume
0,LCHouse1,12250
1,LCHouse2,20250
2,LCHouse3,800


In [52]:
data = [['Aron', 'F', '2020-01-01', 17], ['Alice', 'F', '2020-01-07', 23], ['Bajrang', 'M', '2020-01-07', 7], ['Khali', 'M', '2019-12-25', 11], ['Slaman', 'M', '2019-12-30', 13], ['Joe', 'M', '2019-12-31', 3], ['Jose', 'M', '2019-12-18', 2], ['Priya', 'F', '2019-12-31', 23], ['Priyanka', 'F', '2019-12-30', 17]]
scores = pd.DataFrame(data, columns=['player_name', 'gender', 'day', 'score_points']).astype({'player_name':'object', 'gender':'object', 'day':'datetime64[ns]', 'score_points':'Int64'})


query = """ 

SELECT 
gender, day,
SUM(score_points) OVER(PARTITION BY gender ORDER BY day) AS total
FROM scores
ORDER BY gender


"""

make_select(query)

,gender,day,total
0,F,2019-12-30 00:00:00.000000,17
1,F,2019-12-31 00:00:00.000000,40
2,F,2020-01-01 00:00:00.000000,57
3,F,2020-01-07 00:00:00.000000,80
4,M,2019-12-18 00:00:00.000000,2
5,M,2019-12-25 00:00:00.000000,13
6,M,2019-12-30 00:00:00.000000,26
7,M,2019-12-31 00:00:00.000000,29
8,M,2020-01-07 00:00:00.000000,36


In [68]:
data = [[6, 1290, 425], [11, 2677, 8659], [1, 4446, 2760], [7, 8569, 6086], [13, 2050, 4164], [10, 9002, 7446]]
problems = pd.DataFrame(data, columns=['problem_id', 'likes', 'dislikes']).astype({'problem_id':'Int64', 'likes':'Int64', 'dislikes':'Int64'})


query = """   

SELECT problem_id FROM (
SELECT problem_id,
ROUND(likes  * 1.0 / (likes+dislikes)* 100, 2) AS total
FROM problems GROUP BY problem_id
ORDER BY problem_id) AS sub_query
WHERE total < 60
"""

make_select(query)

,problem_id
0,7
1,10
2,11
3,13


In [89]:
data = [[1, 'Alice'], [2, 'Bob'], [3, 'Jerry']]
salesperson = pd.DataFrame(data, columns=['salesperson_id', 'name']).astype({'salesperson_id':'Int64', 'name':'object'})
data = [[1, 1], [2, 1], [3, 2]]
customer = pd.DataFrame(data, columns=['customer_id', 'salesperson_id']).astype({'customer_id':'Int64', 'salesperson_id':'Int64'})
data = [[1, 2, 892], [2, 1, 354], [3, 3, 988], [4, 3, 856]]
sales = pd.DataFrame(data, columns=['sale_id', 'customer_id', 'price']).astype({'sale_id':'Int64', 'customer_id':'Int64', 'price':'Int64'})

query = """  

WITH cte AS (SELECT sl.sale_id, sl.customer_id, sl.price, s.salesperson_id, s.name FROM sales sl
FULL OUTER JOIN customer c ON sl.customer_id = c.customer_id
FULL OUTER JOIN salesperson s ON s.salesperson_id = c.salesperson_id)


SELECT salesperson_id, name , COALESCE(SUM(price), 0) AS total
FROM cte
GROUP BY salesperson_id, name

"""

make_select(query)

,salesperson_id,name,total
0,1,Alice,1246
1,2,Bob,1844
2,3,Jerry,0


In [95]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [1, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})



query = """  

SELECT 
player_id, event_date,
SUM(games_played) OVER(PARTITION BY player_id ORDER BY event_date) AS games_played_so_far
FROM activity

"""

make_select(query)

,player_id,event_date,games_played_so_far
0,1,2016-03-01 00:00:00.000000,5
1,1,2016-05-02 00:00:00.000000,11
2,1,2017-06-25 00:00:00.000000,12
3,3,2016-03-02 00:00:00.000000,0
4,3,2018-07-03 00:00:00.000000,5


In [100]:
data = [[1, 100, 2008, 10, 5000], [2, 100, 2009, 12, 5000], [7, 200, 2011, 15, 9000]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'year', 'quantity', 'price']).astype({'sale_id':'Int64', 'product_id':'Int64', 'year':'Int64', 'quantity':'Int64', 'price':'Int64'})
data = [[100, 'Nokia'], [200, 'Apple'], [300, 'Samsung']]
product = pd.DataFrame(data, columns=['product_id', 'product_name']).astype({'product_id':'Int64', 'product_name':'object'})


query = """ 

SELECT 
p.product_id, SUM(s.quantity ) AS total_quantity
FROM sales s INNER JOIN product p ON s.product_id = p.product_id
GROUP BY p.product_id
"""

make_select(query)

,product_id,total_quantity
0,100,22
1,200,15


In [104]:
data = [[113, 'LosAngeles', 7560386], [136, 'SanFrancisco', 2380268], [92, 'Chicago', 9833209], [60, 'Chicago', 5147582], [8, 'Chicago', 5274441], [79, 'SanFrancisco', 8372065], [37, 'Chicago', 7939595], [53, 'LosAngeles', 4965123], [178, 'SanFrancisco', 999207], [51, 'NewYork', 5951718], [121, 'NewYork', 2893760]]
listings = pd.DataFrame(data, columns=['listing_id', 'city', 'price']).astype({'listing_id':'Int64', 'city':'object', 'price':'Int64'})


query = """   

SELECT 
city
FROM listings
GROUP BY city
HAVING AVG(price) > (SELECT AVG(price) FROM listings)
"""

make_select(query)

,city
0,Chicago
1,LosAngeles


In [108]:
data = [[1, 'W00576', '2012-03-25 11:30:00', '2012-03-25 12:40:00'], [2, 'W00300', '2012-03-25 10:30:00', '2012-03-25 10:50:00'], [3, 'W00455', '2012-03-26 14:30:00', '2012-03-26 17:40:00'], [4, 'W00455', '2012-03-25 12:30:00', '2012-03-25 13:40:00'], [5, 'W00576', '2012-03-25 08:10:00', '2012-03-25 09:10:00'], [6, 'W00576', '2012-03-28 02:30:00', '2012-03-28 02:50:00']]
bikes = pd.DataFrame(data, columns=['ride_id', 'bike_number', 'start_time', 'end_time']).astype({'ride_id':'Int64', 'bike_number':'object', 'start_time':'datetime64[ns]', 'end_time':'datetime64[ns]'})



query = """   

SELECT 
bike_number, 
MAX(end_time) AS end_time 
FROM bikes
GROUP BY bike_number
ORDER BY end_time DESC
"""

make_select(query)

,bike_number,end_time
0,W00576,2012-03-28 02:50:00.000000
1,W00455,2012-03-26 17:40:00.000000
2,W00300,2012-03-25 10:50:00.000000


In [115]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 5, '2019-08-02', '2019-08-02'], [3, 1, '2019-08-11', '2019-08-11'], [4, 3, '2019-08-24', '2019-08-26'], [5, 4, '2019-08-21', '2019-08-22'], [6, 2, '2019-08-11', '2019-08-13']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})


query = """  

SELECT 
ROUND(COUNT(CASE WHEN order_type = 'immediate' THEN 1 END) * 1.0 /COUNT(*) * 100, 2) AS immediate_percentage
FROM  (SELECT  
*,
CASE WHEN order_date = customer_pref_delivery_date THEN 'immediate' ELSE 'scheduled' END AS order_type
FROM delivery
) AS sub_query
"""

make_select(query)

,immediate_percentage
0,33.33


In [121]:
data = [[-1], [0], [2]]
point = pd.DataFrame(data, columns=['x']).astype({'x':'Int64'})


query = """  

SELECT MIN(distance) AS shortest FROM (SELECT 
*,
x- LAG(x) OVER(ORDER BY x) AS distance
FROM point)

"""

make_select(query)

,shortest
0,1


In [139]:
data = [[1, 1, '2020-06-01', 1, 10], [2, 1, '2020-06-08', 2, 10], [3, 2, '2020-06-02', 1, 5], [4, 3, '2020-06-03', 3, 5], [5, 4, '2020-06-04', 4, 1], [6, 4, '2020-06-05', 5, 5], [7, 5, '2020-06-05', 1, 10], [8, 5, '2020-06-14', 4, 5], [9, 5, '2020-06-21', 3, 5]]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'order_date', 'item_id', 'quantity']).astype({'order_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'object', 'quantity':'Int64'})
data = [[1, 'LC Alg. Book', 'Book'], [2, 'LC DB. Book', 'Book'], [3, 'LC SmarthPhone', 'Phone'], [4, 'LC Phone 2020', 'Phone'], [5, 'LC SmartGlass', 'Glasses'], [6, 'LC T-Shirt XL', 'T-shirt']]
items = pd.DataFrame(data, columns=['item_id', 'item_name', 'item_category']).astype({'item_id':'object', 'item_name':'object', 'item_category':'object'})




query = """   

WITH cte AS (SELECT o.order_id, o.customer_id, o.order_date, o.item_id, o.quantity,
i.item_name, i.item_category,
strftime('%w', o.order_date) AS day_of_week
FROM orders o 
OUTER FULL JOIN items i ON o.item_id = i.item_id)

SELECT 
item_category AS Category  ,
SUM(CASE WHEN day_of_week = 1 THEN quantity ELSE 0 END) AS Monday,
SUM(CASE WHEN day_of_week = 2 THEN quantity ELSE 0 END) AS Tuesday,
SUM(CASE WHEN day_of_week = 3 THEN quantity ELSE 0 END) AS Wednesday,
SUM(CASE WHEN day_of_week = 4 THEN quantity ELSE 0 END) AS Thursday,
SUM(CASE WHEN day_of_week = 5 THEN quantity ELSE 0 END) AS Friday,
SUM(CASE WHEN day_of_week = 6 THEN quantity ELSE 0 END) AS Saturday,
SUM(CASE WHEN day_of_week = 0 THEN quantity ELSE 0 END) AS Sunday
FROM cte
GROUP BY item_category
ORDER BY item_category
"""

make_select(query)

,Category,Monday,Tuesday,Wednesday,Thursday,Friday,Saturday,Sunday
0,Book,0,0,0,0,0,0,0
1,Glasses,0,0,0,0,0,0,0
2,Phone,0,0,0,0,0,0,0
3,T-shirt,0,0,0,0,0,0,0


In [220]:
data = [[103, 'Australia', 84, 'WhisperingPines'], [737, 'Australia', 85, 'GrapesGalore'], [848, 'Australia', 100, 'HarmonyHill'], [222, 'Hungary', 60, 'MoonlitCellars'], [116, 'USA', 47, 'RoyalVines'], [124, 'USA', 45, "Eagle'sNest"], [648, 'India', 69, 'SunsetVines'], [894, 'USA', 39, 'RoyalVines'], [677, 'USA', 9, 'PacificCrest']]
wineries = pd.DataFrame(data, columns=['id', 'country', 'points', 'winery']).astype({'id':'Int64', 'country':'object', 'points':'Int64', 'winery':'object'})


query = """   


WITH cte AS (SELECT 
*,
SUM(points) OVER(PARTITION BY winery) AS total
FROM wineries
),


cte2 AS (SELECT 
*,
DENSE_RANK() OVER(PARTITION BY country ORDER BY total DESC) AS ranking
FROM cte)


SELECT 
    country,
    MAX(CASE WHEN ranking = 1 THEN CONCAT(winery, ' (', total, ')') END) AS top_winery,
     COALESCE(
        MAX(CASE WHEN ranking = 2 THEN CONCAT(winery, ' (', total, ')') END),
        'No second winery'
    ) AS second_winery,
    COALESCE(
        MAX(CASE WHEN ranking = 3 THEN CONCAT(winery, ' (', total, ')') END),
        'No third winery'
    ) AS third_winery 
FROM cte2
WHERE ranking <= 3
GROUP BY country



"""

make_select(query)

,country,top_winery,second_winery,third_winery
0,Australia,HarmonyHill (100),GrapesGalore (85),WhisperingPines (84)
1,Hungary,MoonlitCellars (60),No second winery,No third winery
2,India,SunsetVines (69),No second winery,No third winery
3,USA,RoyalVines (86),Eagle'sNest (45),PacificCrest (9)


In [ ]:
data = [[1341, 123424, 1500.6, '2019-12-31 12:00:00'], [1423, 123424, 1000.2, '2020-12-31 12:00:00'], [1623, 123424, 1246.44, '2021-12-31 12:00:00'], [1322, 123424, 2145.32, '2022-12-31 12:00:00']]
user_transactions = pd.DataFrame(data, columns=['transaction_id', 'product_id', 'spend', 'transaction_date']).astype({'transaction_id': 'int','product_id': 'int','spend': 'float','transaction_date': 'datetime64[ns]'})

query = """   

WITH cte AS (SELECT 
EXTRACT(YEAR FROM transaction_date) AS year,
product_id,
SUM(spend) AS curr_year_spend
FROM user_transactions
GROUP BY year, product_id),

cte2 AS (SELECT
year, product_id, curr_year_spend,
LAG(curr_year_spend,1) OVER(PARTITION BY product_id ORDER BY year) AS prev_year_spend
FROM cte)


SELECT 
*,
ROUND((curr_year_spend - prev_year_spend) / prev_year_spend * 100, 2) AS yoy_rate
FROM cte2
ORDER BY product_id, year
"""

make_select(query)

,year,product_id,curr_year_spend,prev_year_spend,yoy_rate
0,2019,123424,1500.60,NaN,NaN
1,2020,123424,1000.20,1500.60,-33.35
2,2021,123424,1246.44,1000.20,24.62
3,2022,123424,2145.32,1246.44,72.12


In [13]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 2, 3, '2019-06-02', 1, 800], [3, 3, 4, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})


query = """   

WITH cte AS (SELECT *, 
DENSE_RANK() OVER(ORDER BY total_price DESC) AS ranking
FROM  (SELECT s.seller_id, SUM(s.price) AS total_price
FROM product p INNER JOIN sales s ON p.product_id = s.product_id
GROUP BY s.seller_id) AS sub_query)



SELECT seller_id FROM cte WHERE ranking = 1


"""

make_select(query)

,seller_id
0,1
1,3


In [16]:
data = [[1, 'Todd', 'Wilson', 110000, 'D1006'], [1, 'Todd', 'Wilson', 106119, 'D1006'], [2, 'Justin', 'Simon', 128922, 'D1005'], [2, 'Justin', 'Simon', 130000, 'D1005'], [3, 'Kelly', 'Rosario', 42689, 'D1002'], [4, 'Patricia', 'Powell', 162825, 'D1004'], [4, 'Patricia', 'Powell', 170000, 'D1004'], [5, 'Sherry', 'Golden', 44101, 'D1002'], [6, 'Natasha', 'Swanson', 79632, 'D1005'], [6, 'Natasha', 'Swanson', 90000, 'D1005']]
salary = pd.DataFrame(data, columns=['emp_id', 'firstname', 'lastname', 'salary', 'department_id']).astype({'emp_id':'Int64', 'firstname':'object', 'lastname':'object', 'salary':'Int64', 'department_id':'object'})


query = """   

SELECT 
emp_id,	firstname,lastname,MAX(salary) AS salary,department_id
FROM salary
GROUP BY emp_id,	firstname,lastname,department_id
ORDER BY emp_id ASC
"""

make_select(query)

,emp_id,firstname,lastname,salary,department_id
0,1,Todd,Wilson,110000,D1006
1,2,Justin,Simon,130000,D1005
2,3,Kelly,Rosario,42689,D1002
3,4,Patricia,Powell,170000,D1004
4,5,Sherry,Golden,44101,D1002
5,6,Natasha,Swanson,90000,D1005


In [56]:
data = [[1, '2020-09-15', 1, 30], [2, '2020-09-17', 2, 90], [3, '2020-10-06', 3, 20], [4, '2020-10-20', 3, 21], [5, '2020-11-10', 1, 10], [6, '2020-11-21', 2, 15], [7, '2020-12-01', 4, 55], [8, '2020-12-03', 4, 77], [9, '2021-01-07', 3, 31], [10, '2021-01-15', 2, 20]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'invoice']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'invoice':'Int64'})


query = """  

SELECT 
strftime('%Y-%m', order_date) AS month,
COUNT(DISTINCT order_id) AS order_count,
COUNT(DISTINCT customer_id) AS customer_count
FROM orders
WHERE invoice > 20
GROUP BY month

"""

make_select(query)


,month,order_count,customer_count
0,2020-09,2,2
1,2020-10,1,1
2,2020-12,2,1
3,2021-01,1,1


In [60]:
data = [['China', 10, 10, 20], ['South Sudan', 0, 0, 1], ['USA', 10, 10, 20], ['Israel', 2, 2, 3], ['Egypt', 2, 2, 2]]
olympic = pd.DataFrame(data, columns=['country', 'gold_medals', 'silver_medals', 'bronze_medals']).astype({'country':'object', 'gold_medals':'Int64', 'silver_medals':'Int64', 'bronze_medals':'Int64'})


query = """   

SELECT * 
FROM olympic
ORDER BY gold_medals DESC, silver_medals DESC, bronze_medals DESC, country ASC

"""

make_select(query)

,country,gold_medals,silver_medals,bronze_medals
0,China,10,10,20
1,USA,10,10,20
2,Israel,2,2,3
3,Egypt,2,2,2
4,South Sudan,0,0,1


In [70]:
data = [[17, 'Addison'], [14, 'Ethan'], [4, 'Michael'], [2, 'Avery'], [10, 'Eleanor']]
users = pd.DataFrame(data, columns=['user_id', 'name']).astype({'user_id':'Int64', 'name':'object'})
data = [[72, 17, 160], [42, 14, 161], [45, 4, 59], [32, 2, 197], [15, 4, 357], [56, 2, 196], [10, 14, 25]]
rides = pd.DataFrame(data, columns=['ride_id', 'user_id', 'distance']).astype({'ride_id':'Int64', 'user_id':'Int64', 'distance':'Int64'})



query = """  

SELECT 
u.user_id, u.name, COALESCE(SUM(r.distance), 0) AS "traveled distance"
FROM users u 
FULL OUTER JOIN rides r ON u.user_id = r.user_id
GROUP BY u.user_id, u.name
ORDER BY u.user_id

"""

make_select(query)

,user_id,name,traveled distance
0,2,Avery,393
1,4,Michael,416
2,10,Eleanor,0
3,14,Ethan,186
4,17,Addison,160


In [75]:
data = [[303651, "Heart Won't Forget", 'Ed Sheeran'], [1046089, 'Shape of you', 'Sia'], [33445, "I'm the one", 'DJ Khalid'], [811266, 'Young Dumb & Broke', 'DJ Khalid'], [505727, 'Happier', 'Ed Sheeran']]
spotify = pd.DataFrame(data, columns=['id', 'track_name', 'artist']).astype({'id':'Int64', 'track_name':'object', 'artist':'object'})


query = """   

SELECT 
artist, COUNT(artist) AS occurrences
FROM spotify
GROUP BY artist
ORDER BY occurrences DESC, artist ASC
"""

make_select(query)


,artist,occurrences
0,DJ Khalid,2
1,Ed Sheeran,2
2,Sia,1


In [79]:
data = [[1, 1, 101, 10], [2, 2, 101, 1], [3, 3, 102, 3], [4, 3, 102, 2], [5, 2, 103, 3]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'user_id', 'quantity']).astype({'sale_id':'Int64', 'product_id':'Int64', 'user_id':'Int64', 'quantity':'Int64'})
data = [[1, 10], [2, 25], [3, 15]]
product = pd.DataFrame(data, columns=['product_id', 'price']).astype({'product_id':'Int64', 'price':'Int64'})


query = """  

SELECT 
s.user_id,
SUM(s.quantity * p.price) AS spending
FROM sales s INNER JOIN product p ON s.product_id = p.product_id
GROUP BY s.user_id
ORDER BY spending DESC, s.user_id ASC
"""

make_select(query)

,user_id,spending
0,101,125
1,102,75
2,103,75


In [ ]:
data = [['2020-06-10 08:00', 1, 'LC-Channel'], ['2020-05-11 12:00', 2, 'LC-Channel'], ['2020-05-12 12:00', 3, 'LC-Channel'], ['2020-05-13 14:00', 4, 'Disney Ch'], ['2020-06-18 14:00', 4, 'Disney Ch'], ['2020-07-15 16:00', 5, 'Disney Ch']]
tv_program = pd.DataFrame(data, columns=['program_date', 'content_id', 'channel']).astype({'program_date':'datetime64[ns]', 'content_id':'Int64', 'channel':'object'})
data = [[1, 'Leetcode Movie', 'N', 'Movies'], [2, 'Alg. for Kids', 'Y', 'Series'], [3, 'Database Sols', 'N', 'Series'], [4, 'Aladdin', 'Y', 'Movies'], [5, 'Cinderella', 'Y', 'Movies']]
content = pd.DataFrame(data, columns=['content_id', 'title', 'Kids_content', 'content_type']).astype({'content_id':'object', 'title':'object', 'Kids_content':'object', 'content_type':'object'})


query = """  

SELECT * 
FROM tv_program t 
INNER JOIN content c ON t.content_id = c.content_id
WHERE c.content_type = 'Movies' AND c.Kids_content = 'Y' AND TO_CHAR(t.program_date,'YYYY-MM') = '2020-06'
"""

make_select(query)

,program_date,content_id,channel,content_id,title,Kids_content,content_type
0,2020-05-13 14:00:00.000000,4,Disney Ch,4,Aladdin,Y,Movies
1,2020-06-18 14:00:00.000000,4,Disney Ch,4,Aladdin,Y,Movies
2,2020-07-15 16:00:00.000000,5,Disney Ch,5,Cinderella,Y,Movies


In [103]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 1, 3, '2019-06-02', 1, 800], [3, 3, 3, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})


query = """   

WITH cte AS (SELECT 
p.product_id, p.product_name, p.unit_price, s.seller_id, s.buyer_id, s.sale_date, s.quantity, s.price
FROM product p INNER JOIN sales s ON p.product_id = s.product_id),


cte2 AS (SELECT 
buyer_id
FROM cte
WHERE product_name = 'S8'),

cte3 AS (SELECT buyer_id FROM cte WHERE product_name != 'S8')


SELECT a.buyer_id
FROM cte2 a LEFT JOIN cte3 b ON a.buyer_id = b.buyer_id  
WHERE b.buyer_id IS NULL

"""

make_select(query)

,buyer_id
0,1


In [124]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 1], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})



query = """


WITH cte AS (SELECT
p.project_id,
e.employee_id AS emp_id,
COUNT(p.employee_id) OVER(PARTITION BY p.project_id ) AS employess
FROM project p INNER JOIN employee e ON p.employee_id = e.employee_id)

SELECT DISTINCT project_id 
FROM cte
WHERE employess = (SELECT MAX(employess) FROM CTE)

"""

make_select(query)

,project_id
0,1


In [ ]:
data = [[20, 20, 23], [20, 20, 20], [20, 21, 22], [13, 14, 30]]
triangles = pd.DataFrame(data, columns=['A', 'B', 'C']).astype({'A':'Int64', 'B':'Int64', 'C':'Int64'})


query = """ 

SELECT 
    CASE 
        WHEN A + B <= C OR A + C <= B OR B + C <= A THEN 'Not A Triangle'
        WHEN A = B AND A = C THEN 'Equilateral'
        WHEN A = B OR A = C OR B = C THEN 'Isosceles'
        ELSE 'Scalene'
    END AS triangle_type
FROM triangles;

"""

make_select(query)

,triangle_type
0,Isosceles
1,Equilateral
2,Scalene
3,Not A Triangle


In [193]:
data = [[0, 'ham'], [1, 'bacon']]
product = pd.DataFrame(data, columns=['product_id', 'name']).astype({'product_id':'Int64', 'name':'object'})
data = [[23, 0, 2, 0, 5, 0], [12, 0, 0, 4, 0, 3], [1, 1, 1, 1, 0, 1], [2, 1, 1, 0, 1, 1], [3, 1, 0, 1, 1, 1], [4, 1, 1, 1, 1, 0]]
invoice = pd.DataFrame(data, columns=['invoice_id', 'product_id', 'rest', 'paid', 'canceled', 'refunded']).astype({'invoice_id':'Int64', 'product_id':'Int64', 'rest':'Int64', 'paid':'Int64', 'canceled':'Int64', 'refunded':'Int64'})

query = """  

SELECT 
p.name, 
COALESCE(SUM(i.rest),0) AS rest,
COALESCE(SUM(i.paid),0) AS paid,
COALESCE(SUM(i.canceled),0) AS canceled,
COALESCE(SUM(i.refunded),0) AS refunded
FROM product p LEFT JOIN invoice i ON p.product_id = i.product_id 
GROUP BY p.name


"""


make_select(query)

,name,rest,paid,canceled,refunded
0,bacon,3,3,3,3
1,ham,2,4,5,3


In [202]:
data = [[1, 1, '2019-07-20', 'open_session'], [1, 1, '2019-07-20', 'scroll_down'], [1, 1, '2019-07-20', 'end_session'], [2, 4, '2019-07-20', 'open_session'], [2, 4, '2019-07-21', 'send_message'], [2, 4, '2019-07-21', 'end_session'], [3, 2, '2019-07-21', 'open_session'], [3, 2, '2019-07-21', 'send_message'], [3, 2, '2019-07-21', 'end_session'], [3, 5, '2019-07-21', 'open_session'], [3, 5, '2019-07-21', 'scroll_down'], [3, 5, '2019-07-21', 'end_session'], [4, 3, '2019-06-25', 'open_session'], [4, 3, '2019-06-25', 'end_session']]
activity = pd.DataFrame(data, columns=['user_id', 'session_id', 'activity_date', 'activity_type']).astype({'user_id':'Int64', 'session_id':'Int64', 'activity_date':'datetime64[ns]', 'activity_type':'object'})


query = """  

WITH cte AS (SELECT 
user_id, 
COUNT(DISTINCT session_id) AS total
FROM activity
WHERE activity_date BETWEEN '2019-06-28' AND '2019-07-27'
GROUP BY user_id)


SELECT COALESCE(ROUND(SUM(total) * 1.0/COUNT(total), 2), 0) AS average_sessions_per_user FROM cte
"""

make_select(query)

,average_sessions_per_user
0,1.33


In [ ]:
data = [['0', 'store1', '95'], ['0', 'store3', '105'], ['0', 'store2', '100'], ['1', 'store1', '70'], ['1', 'store3', '80']]
products = pd.DataFrame(data, columns=['product_id', 'store', 'price']).astype({'product_id':'Int64', 'store':'category', 'price':'Int64'})


def products_price(products: pd.DataFrame) -> pd.DataFrame:
    pivot = pd.pivot(products, index="product_id", columns="store", values="price").reset_index()

    
    df = pivot.rename_axis(None, axis=1)

    return df


products_price(products)

,product_id,store1,store2,store3
0,0,95,100,105
1,1,70,<NA>,80


In [269]:
data = [[1, 'Alex', 'Singer'], [3, 'Alice', 'Actor'], [2, 'Bob', 'Player'], [4, 'Messi', 'Doctor'], [6, 'Tyson', 'Engineer'], [5, 'Meir', 'Lawyer']]
person = pd.DataFrame(data, columns=['person_id', 'name', 'profession']).astype({'person_id':'Int64', 'name':'object', 'profession':'object'})


def concatenate_info(person: pd.DataFrame) -> pd.DataFrame:
    person["name"] = person["name"] + "(" +person["profession"].str[0]+")"


    result =  person[["person_id", "name"]].sort_values(by="person_id", ascending=False)

    return result


concatenate_info(person)

,person_id,name
4,6,Tyson(E)
5,5,Meir(L)
3,4,Messi(D)
1,3,Alice(A)
2,2,Bob(P)
0,1,Alex(S)


In [280]:
data = [[123, 'Python'], [234, 'R'], [123, 'Tableau'], [123, 'PostgreSQL'], [234, 'PowerBI'], [234, 'SQL Server'], [147, 'Python'], [147, 'Tableau'], [147, 'Java'], [147, 'PostgreSQL'], [256, 'Tableau'], [102, 'DataAnalysis']]
candidates = pd.DataFrame(data, columns=['candidate_id', 'skill']).astype({'candidate_id':'Int64', 'skill':'object'})


query = """   

SELECT candidate_id
FROM candidates
WHERE skill IN ('Python', 'Tableau', 'PostgreSQL')
GROUP BY candidate_id
HAVING COUNT(DISTINCT skill) >= 3
ORDER BY candidate_id 

"""

make_select(query)


,candidate_id
0,123
1,147


In [286]:
data = [['Kathy', 'Engineering', 50000], ['Roy', 'Marketing', 30000], ['Charles', 'Engineering', 45000], ['Jack', 'Engineering', 85000], ['Benjamin', 'Marketing', 34000], ['Anthony', 'Marketing', 42000], ['Edward', 'Engineering', 102000], ['Terry', 'Engineering', 44000], ['Evelyn', 'Marketing', 53000], ['Arthur', 'Engineering', 32000]]
salaries = pd.DataFrame(data, columns=['emp_name', 'department', 'salary']).astype({'emp_name':'object', 'department':'object', 'salary':'Int64'})


query = """   

WITH cte AS (
SELECT MAX(salary) AS e_sal FROM salaries WHERE department = 'Engineering'
),

cte2 AS (SELECT MAX(salary) AS m_sal FROM salaries WHERE department = 'Marketing')

SELECT (SELECT e_sal FROM cte) - (SELECT m_sal FROM cte2) AS salary_difference 

"""

make_select(query)

,salary_difference
0,49000


In [ ]:
data = [[1, 'Winston', 'USA'], [2, 'Jonathan', 'Peru'], [3, 'Moustafa', 'Egypt']]
customers = pd.DataFrame(data, columns=['customer_id', 'name', 'country']).astype({'customer_id':'Int64', 'name':'object', 'country':'object'})
data = [[10, 'LC Phone', 300], [20, 'LC T-Shirt', 10], [30, 'LC Book', 45], [40, 'LC Keychain', 2]]
product = pd.DataFrame(data, columns=['product_id', 'description', 'price']).astype({'product_id':'Int64', 'description':'object', 'price':'Int64'})
data = [[1, 1, 10, '2020-06-10', 1], [2, 1, 20, '2020-07-01', 1], [3, 1, 30, '2020-07-08', 2], [4, 2, 10, '2020-06-15', 2], [5, 2, 40, '2020-07-01', 10], [6, 3, 20, '2020-06-24', 2], [7, 3, 30, '2020-06-25', 2], [9, 3, 30, '2020-05-08', 3]]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'product_id', 'order_date', 'quantity']).astype({'order_id':'Int64', 'customer_id':'Int64', 'product_id':'Int64', 'order_date':'datetime64[ns]', 'quantity':'Int64'})

# WHERE o.order_date BETWEEN '2020-06-01' AND '2020-07-31'

query = """   

WITH cte AS (SELECT 
o.order_id, o.customer_id, o.product_id, o.order_date, o.quantity * p.price AS total_price,c.name, c.country
FROM orders o
INNER JOIN product p ON p.product_id = o.product_id
INNER JOIN customers c ON c.customer_id = o.customer_id),

cte2 AS (SELECT 
customer_id, name, strftime('%Y-%m', order_date) AS date, SUM(total_price) AS total_spent
FROM cte
WHERE date BETWEEN '2020-06' AND '2020-07'
GROUP BY customer_id, name, date)

SELECT 
*
FROM cte2

"""

make_select(query)

,customer_id,name,date,total_spent
0,1,Winston,2020-06,300
1,1,Winston,2020-07,100
2,2,Jonathan,2020-06,600
3,2,Jonathan,2020-07,20
4,3,Moustafa,2020-06,110


In [325]:
data = [[2, 'USA'], [3, 'Australia'], [7, 'Peru'], [5, 'China'], [8, 'Morocco'], [9, 'Spain']]
countries = pd.DataFrame(data, columns=['country_id', 'country_name']).astype({'country_id':'Int64', 'country_name':'object'})
data = [[2, 15, '2019-11-01'], [2, 12, '2019-10-28'], [2, 12, '2019-10-27'], [3, -2, '2019-11-10'], [3, 0, '2019-11-11'], [3, 3, '2019-11-12'], [5, 16, '2019-11-07'], [5, 18, '2019-11-09'], [5, 21, '2019-11-23'], [7, 25, '2019-11-28'], [7, 22, '2019-12-01'], [7, 20, '2019-12-02'], [8, 25, '2019-11-05'], [8, 27, '2019-11-15'], [8, 31, '2019-11-25'], [9, 7, '2019-10-23'], [9, 3, '2019-12-23']]
weather = pd.DataFrame(data, columns=['country_id', 'weather_state', 'day']).astype({'country_id':'Int64', 'weather_state':'Int64', 'day':'datetime64[ns]'})


query = """   

WITH cte AS (SELECT 
c.country_name, AVG(w.weather_state) AS avg_weather
FROM countries c
INNER JOIN weather w ON c.country_id = w.country_id
WHERE strftime('%Y-%m', day) = '2019-11'
GROUP BY  c.country_name)


SELECT 
country_name,
CASE 
    WHEN avg_weather <= 15 THEN 'Cold'
    WHEN avg_weather >= 25 THEN 'Hot'
    ELSE 'Warm'

END AS weather_type 
FROM cte

"""

make_select(query)

,country_name,weather_type
0,Australia,Cold
1,China,Warm
2,Morocco,Hot
3,Peru,Hot
4,USA,Cold


In [ ]:
data = {
    'ad_id': [1, 2, 3, 5, 1, 2, 3, 1, 2, 1],
    'user_id': [1, 2, 3, 5, 7, 7, 5, 4, 11, 2],
    'action': ['Clicked', 'Clicked', 'Viewed', 'Ignored', 'Ignored', 'Viewed', 'Clicked', 'Viewed', 'Viewed', 'Clicked']
}

ads = pd.DataFrame(data)

query = """   

WITH cte AS (SELECT 
ad_id, 
COUNT(CASE WHEN action = 'Clicked' THEN 1 END) AS clicks,
COUNT(CASE WHEN action = 'Viewed' THEN 1 END) AS views
FROM ads
GROUP BY ad_id)


SELECT ad_id, COALESCE(ROUND(clicks * 1.0 / NULLIF(clicks + views, 0) * 100, 2), 0) AS ctr  FROM cte ORDER BY ctr DESC, ad_id ASC

"""

make_select(query)

,ad_id,ctr
0,1,66.67
1,3,50.00
2,2,33.33
3,5,0.00


In [364]:
data = [[1, 1, '2019-07-01', 'view', None], [1, 1, '2019-07-01', 'like', None], [1, 1, '2019-07-01', 'share', None], [2, 4, '2019-07-04', 'view', None], [2, 4, '2019-07-04', 'report', 'spam'], [3, 4, '2019-07-04', 'view', None], [3, 4, '2019-07-04', 'report', 'spam'], [4, 3, '2019-07-02', 'view', None], [4, 3, '2019-07-02', 'report', 'spam'], [5, 2, '2019-07-04', 'view', None], [5, 2, '2019-07-04', 'report', 'racism'], [5, 5, '2019-07-04', 'view', None], [5, 5, '2019-07-04', 'report', 'racism']]
actions = pd.DataFrame(data, columns=['user_id', 'post_id', 'action_date', 'action', 'extra']).astype({'user_id':'Int64', 'post_id':'Int64', 'action_date':'datetime64[ns]', 'action':'object', 'extra':'object'})


query = """   

SELECT 
extra AS report_reason, 
COUNT(DISTINCT CASE WHEN action = 'report' THEN post_id END) AS report_count
FROM actions
WHERE action_date BETWEEN '2019-07-04' AND '2019-07-05' AND report_reason IS NOT NULL
GROUP BY extra

"""

make_select(query)

,report_reason,report_count
0,racism,2
1,spam,1


In [379]:
data = [[8, '2021-4-3 15:57:28', 57], [9, '2021-4-28 08:47:25', 21], [1, '2021-4-29 13:28:30', 58], [5, '2021-4-28 16:39:59', 40], [6, '2021-4-29 23:39:28', 58]]
transactions = pd.DataFrame(data, columns=['transaction_id', 'day', 'amount']).astype({'transaction_id':'Int64', 'day':'datetime64[ns]', 'amount':'Int64'})


query = """   

WITH cte AS (SELECT 
transaction_id, amount,strftime('%d', day) AS day,
MAX(amount) OVER(PARTITION BY strftime('%d', day)) AS max_amount
FROM transactions)

SELECT transaction_id FROM cte WHERE amount = max_amount ORDER BY transaction_id ASC

"""

make_select(query)

,transaction_id
0,1
1,5
2,6
3,8


In [383]:
data = [[1, 'Jack', 'M', 1], [2, 'Jane', 'F', 1], [3, 'Mark', 'M', 2]]
student = pd.DataFrame(data, columns=['student_id', 'student_name', 'gender', 'dept_id']).astype({'student_id':'Int64', 'student_name':'object', 'gender':'object', 'dept_id':'Int64'})
data = [[1, 'Engineering'], [2, 'Science'], [3, 'Law']]
department = pd.DataFrame(data, columns=['dept_id', 'dept_name']).astype({'dept_id':'Int64', 'dept_name':'object'})

query = """   

SELECT
d.dept_name,
COUNT(s.student_id) AS student_number 
FROM department d LEFT JOIN student s ON d.dept_id = s.dept_id
GROUP BY d.dept_name
ORDER BY student_number DESC, d.dept_name ASC

"""

make_select(query)

,dept_name,student_number
0,Engineering,2
1,Science,1
2,Law,0


In [390]:
data = [[1, 'Winston'], [2, 'Jonathan'], [3, 'Annabelle'], [4, 'Marwan'], [5, 'Khaled']]
customers = pd.DataFrame(data, columns=['customer_id', 'name']).astype({'customer_id':'Int64', 'name':'object'})
data = [[1, '2020-07-31', 1, 30], [2, '2020-7-30', 2, 40], [3, '2020-07-31', 3, 70], [4, '2020-07-29', 4, 100], [5, '2020-06-10', 1, 1010], [6, '2020-08-01', 2, 102], [7, '2020-08-01', 3, 111], [8, '2020-08-03', 1, 99], [9, '2020-08-07', 2, 32], [10, '2020-07-15', 1, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'cost']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'cost':'Int64'})


query = """  

WITH cte AS (SELECT 
c.customer_id, c.name, o.order_id, o.order_date, o.cost,
RANK() OVER(PARTITION BY c.customer_id ORDER BY o.order_date DESC) AS recent_orders
FROM customers c INNER JOIN orders o ON c.customer_id = o.customer_id
)


SELECT name, customer_id, order_id, order_date FROM cte WHERE recent_orders <=3 ORDER BY name 

"""

make_select(query)

,name,customer_id,order_id,order_date
0,Annabelle,3,7,2020-08-01 00:00:00.000000
1,Annabelle,3,3,2020-07-31 00:00:00.000000
2,Jonathan,2,9,2020-08-07 00:00:00.000000
3,Jonathan,2,6,2020-08-01 00:00:00.000000
4,Jonathan,2,2,2020-07-30 00:00:00.000000
5,Marwan,4,4,2020-07-29 00:00:00.000000
6,Winston,1,8,2020-08-03 00:00:00.000000
7,Winston,1,1,2020-07-31 00:00:00.000000
8,Winston,1,10,2020-07-15 00:00:00.000000


In [402]:
data = [[1, 'Alice'], [4, 'Bob'], [5, 'Charlie']]
customers = pd.DataFrame(data, columns=['customer_id', 'customer_name']).astype({'customer_id':'Int64', 'customer_name':'object'})

def find_missing_ids(customers: pd.DataFrame) -> pd.DataFrame:
    
    min_ids = customers["customer_id"].min()
    max_ids = customers["customer_id"].max()

    unique_ids = customers["customer_id"].unique().tolist()

    all_ids = list(range(1, max_ids+1))

    res = []

    for id in all_ids:
        if id not in unique_ids:
            res.append(id)

    return pd.DataFrame({"ids":res})


find_missing_ids(customers)

,ids
0,2
1,3


In [ ]:
data = [[125, 7771, '2022-06-14 09:30:00'], [433, 1052, '2022-07-09 08:15:00'], [234, 7005, '2022-08-20 10:00:00']]
emails = pd.DataFrame(data, columns=['email_id', 'user_id', 'signup_date']).astype({
    'email_id': 'Int64',  # Nullable integer type
    'user_id': 'Int64',   # Nullable integer type
    'signup_date': 'datetime64[ns]'  # DateTime type for signup date
})
data = [[1, 125, 'Verified', '2022-06-15 08:30:00'], [2, 433, 'Not Verified', '2022-07-10 10:45:00'], [4, 234, 'Verified', '2022-08-21 09:30:00']]
texts = pd.DataFrame(data, columns=['text_id', 'email_id', 'signup_action', 'action_date']).astype({
    'text_id': 'Int64',             # Nullable integer type for text_id
    'email_id': 'Int64',            # Nullable integer type for email_id
    'signup_action': 'object',      # Using 'object' to store string values for ENUM
    'action_date': 'datetime64[ns]' # DateTime type for action_date
})

query = """  

SELECT 
e.email_id, e.user_id, e.signup_date, t.signup_action, t.action_date
FROM emails e FULL OUTER JOIN texts t ON e.email_id = t.email_id
WHERE t.action_date - e.signup_date = 1
  AND t.signup_action = 'Verified'

"""

make_select(query)

In [16]:
data = [[683, 101, 'Mortgage'], [218, 101, 'AutoLoan'], [802, 101, 'Inschool'], [593, 102, 'Mortgage'], [138, 102, 'Refinance'], [294, 102, 'Inschool'], [308, 103, 'Refinance'], [389, 104, 'Mortgage']]
loans = pd.DataFrame(data, columns=['loan_id', 'user_id', 'loan_type']).astype({'loan_id':'Int64', 'user_id':'Int64', 'loan_type':'object'})


query = """   

WITH cte AS (SELECT 
user_id, 
COUNT(CASE WHEN loan_type = 'Refinance' THEN 1 END) AS Refinance,
COUNT(CASE WHEN loan_type = 'Mortgage' THEN 1 END) AS Mortgage
FROM loans
GROUP BY user_id)


SELECT user_id FROM cte WHERE Refinance >= 1 AND Mortgage >=1

"""

make_select(query)

,user_id
0,102


In [46]:
data = [[1, 'Winston', 'USA'], [2, 'Jonathan', 'Peru'], [3, 'Moustafa', 'Egypt']]
customers = pd.DataFrame(data, columns=['customer_id', 'name', 'country']).astype({'customer_id':'Int64', 'name':'object', 'country':'object'})
data = [[10, 'LC Phone', 300], [20, 'LC T-Shirt', 10], [30, 'LC Book', 45], [40, 'LC Keychain', 2]]
product = pd.DataFrame(data, columns=['product_id', 'description', 'price']).astype({'product_id':'Int64', 'description':'object', 'price':'Int64'})
data = [[1, 1, 10, '2020-06-10', 1], [2, 1, 20, '2020-07-01', 1], [3, 1, 30, '2020-07-08', 2], [4, 2, 10, '2020-06-15', 2], [5, 2, 40, '2020-07-01', 10], [6, 3, 20, '2020-06-24', 2], [7, 3, 30, '2020-06-25', 2], [9, 3, 30, '2020-05-08', 3]]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'product_id', 'order_date', 'quantity']).astype({'order_id':'Int64', 'customer_id':'Int64', 'product_id':'Int64', 'order_date':'datetime64[ns]', 'quantity':'Int64'})


query = """  

WITH cte AS (SELECT 
o.customer_id, o.product_id, strftime('%Y-%m', o.order_date) AS order_date, (o.quantity * p.price) AS spent, c.name
FROM orders o 
INNER JOIN product p ON o.product_id = p.product_id
INNER JOIN customers c ON o.customer_id = c.customer_id
),

cte2 AS (SELECT 
order_date, name, customer_id ,SUM(spent) AS total_spend
FROM cte
WHERE order_date BETWEEN '2020-06' AND '2020-07'
GROUP BY order_date, name),


cte3 AS (SELECT 
*,
CASE 
    WHEN order_date = '2020-06' AND total_spend >= 100 THEN 'Yes' 
    WHEN order_date = '2020-07' AND total_spend >= 100 THEN 'Yes2' 
    ELSE 'No'
END AS type
FROM cte2)


SELECT 
customer_id,name 
FROM cte3 
WHERE type IN ('Yes','Yes2') 
GROUP BY name
HAVING COUNT(DISTINCT type) >= 2

"""

make_select(query)

,customer_id,name
0,1,Winston


In [63]:
data = [[1, 30], [2, 87]]
new_york = pd.DataFrame(data, columns=['student_id', 'score']).astype({'student_id':'Int64', 'score':'Int64'})
data = [[2, 89], [3, 88]]
california = pd.DataFrame(data, columns=['student_id', 'score']).astype({'student_id':'Int64', 'score':'Int64'})



def find_winner(new_york: pd.DataFrame, california: pd.DataFrame) -> pd.DataFrame:
    new_york["winner"] = "New York University"
    california["winner"] = "California University"

    df = pd.concat([new_york, california])

    result = df[df["score"] >= 90]

    if len(result) == 2:
        return pd.DataFrame({"winner":["No Winner"]})

    return result[["winner"]]


find_winner(new_york, california)


,winner


In [69]:
data = [[1, '2021-11-07', 'Deposit', 2000], [1, '2021-11-09', 'Withdraw', 1000], [1, '2021-11-11', 'Deposit', 3000], [2, '2021-12-07', 'Deposit', 7000], [2, '2021-12-12', 'Withdraw', 7000]]
transactions = pd.DataFrame(data, columns=['account_id', 'day', 'type', 'amount']).astype({'account_id':'Int64', 'day':'datetime64[ns]', 'type':'object', 'amount':'Int64'})


query = """  

SELECT  
account_id, day,
SUM(CASE WHEN type = 'Deposit' THEN amount ELSE -amount END) OVER(PARTITION BY account_id ORDER BY day) AS balance
FROM transactions
ORDER BY account_id ASC,day ASC

"""

make_select(query)

,account_id,day,balance
0,1,2021-11-07 00:00:00.000000,2000
1,1,2021-11-09 00:00:00.000000,1000
2,1,2021-11-11 00:00:00.000000,4000
3,2,2021-12-07 00:00:00.000000,7000
4,2,2021-12-12 00:00:00.000000,0


In [84]:
data = [[156, 'Michael', 107, 'Manager'], [112, 'Lucas', 107, 'Consultant'], [8, 'Isabella', 101, 'Manager'], [160, 'Joseph', 100, 'Manager'], [80, 'Aiden', 100, 'Engineer'], [190, 'Skylar', 100, 'Freelancer'], [196, 'Stella', 101, 'Coordinator'], [167, 'Audrey', 100, 'Consultant'], [97, 'Nathan', 101, 'Supervisor'], [128, 'Ian', 101, 'Administrator'], [81, 'Ethan', 107, 'Administrator']]
employees = pd.DataFrame(data, columns=['emp_id', 'emp_name', 'dep_id', 'position']).astype({
    'emp_id': 'Int64', 
    'emp_name': 'object', 
    'dep_id': 'Int64', 
    'position': 'object'
})


query = """   

WITH cte AS (SELECT 
*,
DENSE_RANK() OVER(ORDER BY total_employees DESC) AS ranking
FROM (SELECT 
*,
COUNT(emp_id) OVER(PARTITION BY dep_id) AS total_employees
FROM employees) AS sub_query)


SELECT emp_name AS manager_name,dep_id  FROM cte WHERE ranking =1 AND position = 'Manager' ORDER BY dep_id ASC

"""

make_select(query)

,manager_name,dep_id
0,Joseph,100
1,Isabella,101


In [105]:
data = [[1, 'Alice'], [2, 'Bob'], [3, 'Tom'], [4, 'Jerry'], [5, 'John']]
customers = pd.DataFrame(data, columns=['customer_id', 'name']).astype({'customer_id':'Int64', 'name':'object'})
data = [[1, '2020-07-31', 1, 1], [2, '2020-7-30', 2, 2], [3, '2020-08-29', 3, 3], [4, '2020-07-29', 4, 1], [5, '2020-06-10', 1, 2], [6, '2020-08-01', 2, 1], [7, '2020-08-01', 3, 3], [8, '2020-08-03', 1, 2], [9, '2020-08-07', 2, 3], [10, '2020-07-15', 1, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'product_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'product_id':'Int64'})
data = [[1, 'keyboard', 120], [2, 'mouse', 80], [3, 'screen', 600], [4, 'hard disk', 450]]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'price']).astype({'product_id':'Int64', 'product_name':'object', 'price':'Int64'})


query = """  

WITH cte AS (SELECT
o.order_id, o.order_date, o.customer_id, o.product_id, p.product_name,p.price, c.name
FROM orders o 
INNER JOIN products p ON o.product_id = p.product_id
INNER JOIN customers c ON o.customer_id = c.customer_id),

cte2 AS (SELECT 
customer_id,product_id,product_name, COUNT(order_id) AS total_orders
FROM cte
GROUP BY customer_id, product_id, product_name)


SELECT customer_id,product_id, product_name  FROM (SELECT 
*,
MAX(total_orders) OVER(PARTITION BY customer_id) AS freq_orders
FROM cte2) AS sub_query
WHERE total_orders = freq_orders
"""

make_select(query)

,customer_id,product_id,product_name
0,1,2,mouse
1,2,1,keyboard
2,2,2,mouse
3,2,3,screen
4,3,3,screen
5,4,1,keyboard


In [118]:
data = [[11, '2023-11-07', 1126], [15, '2023-11-30', 7473], [17, '2023-11-14', 2414], [12, '2023-11-24', 9692], [8, '2023-11-03', 5117], [1, '2023-11-16', 5241], [10, '2023-11-12', 8266], [13, '2023-11-24', 12000]]
purchases = pd.DataFrame(data, columns=['user_id', 'purchase_date', 'amount_spend']).astype({'user_id':'Int64', 'purchase_date':'datetime64[ns]', 'amount_spend':'Int64'})



query = """   

WITH cte AS (SELECT 
*,strftime('%w', purchase_date) AS day_week
FROM purchases
WHERE strftime('%Y-%m', purchase_date) = '2023-11')

SELECT
(strftime('%d', purchase_date) - 1) / 7 + 1 AS week_of_month, 
purchase_date, SUM(amount_spend) AS total_amount
FROM cte 
WHERE day_week = '5'
GROUP BY purchase_date
"""

make_select(query)

,week_of_month,purchase_date,total_amount
0,1,2023-11-03 00:00:00.000000,5117
1,4,2023-11-24 00:00:00.000000,21692


In [125]:
data = [[11, 'Atticus', 1, 101], [9, 'Ruben', 6, 104], [6, 'Aliza', 10, 109], [8, 'Alfredo', 0, 107]]
candidates = pd.DataFrame(data, columns=['candidate_id', 'name', 'years_of_exp', 'interview_id']).astype({'candidate_id':'Int64', 'name':'object', 'years_of_exp':'Int64', 'interview_id':'Int64'})
data = [[109, 3, 4], [101, 2, 8], [109, 4, 1], [107, 1, 3], [104, 3, 6], [109, 1, 4], [104, 4, 7], [104, 1, 2], [109, 2, 1], [104, 2, 7], [107, 2, 3], [101, 1, 8]]
rounds = pd.DataFrame(data, columns=['interview_id', 'round_id', 'score']).astype({'interview_id':'Int64', 'round_id':'Int64', 'score':'Int64'})

query = """  

SELECT 
c.candidate_id
FROM candidates c INNER JOIN rounds r ON c.interview_id = r.interview_id
WHERE c.years_of_exp >= 2
GROUP BY c.candidate_id
HAVING SUM(r.score) > 15
"""

make_select(query)

,candidate_id
0,9


In [18]:
products_data = [
    [1, 'Electronics', 1000],
    [2, 'Clothing', 50],
    [3, 'Electronics', 1200],
    [4, 'Home', 500]
]
products = pd.DataFrame(products_data, columns=['product_id', 'category', 'price'])

discounts_data = [
    ['Electronics', 10],
    ['Clothing', 20]
]
discounts = pd.DataFrame(discounts_data, columns=['category', 'discount'])




query = """  

SELECT 
p.product_id, p.price - p.price * COALESCE(d.discount, 0) *1.0/ 100 AS final_price,p.category
FROM products p 
FULL OUTER JOIN discounts d ON p.category = d.category
WHERE p.product_id IS NOT NULL
ORDER BY p.product_id 
"""

make_select(query)


,product_id,final_price,category
0,1,900.0,Electronics
1,2,40.0,Clothing
2,3,1080.0,Electronics
3,4,500.0,Home


In [25]:
data = [[1, 1], [1, 2], [1, 3], [2, 1], [2, 4]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id']).astype({'project_id':'Int64', 'employee_id':'Int64'})
data = [[1, 'Khaled', 3], [2, 'Ali', 2], [3, 'John', 3], [4, 'Doe', 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'name', 'experience_years']).astype({'employee_id':'Int64', 'name':'object', 'experience_years':'Int64'})


query = """  

SELECT project_id, employee_id FROM (SELECT 
p.project_id, p.employee_id, e.name, e.experience_years,
MAX(e.experience_years) OVER(PARTITION BY p.project_id) AS most_experienced
FROM project p 
INNER JOIN employee e ON p.employee_id = e.employee_id) AS sub_query
WHERE experience_years = most_experienced

"""

make_select(query)

,project_id,employee_id
0,1,1
1,1,3
2,2,1


In [49]:
data = [[1, 1, 101, 10], [2, 3, 101, 7], [3, 1, 102, 9], [4, 2, 102, 6], [5, 3, 102, 10], [6, 1, 102, 6]]
sales = pd.DataFrame(data, columns=['sale_id', 'product_id', 'user_id', 'quantity']).astype({'sale_id':'Int64', 'product_id':'Int64', 'user_id':'Int64', 'quantity':'Int64'})
data = [[1, 10], [2, 25], [3, 15]]
product = pd.DataFrame(data, columns=['product_id', 'price']).astype({'product_id':'Int64', 'price':'Int64'})


query = """  

WITH cte AS (SELECT 
s.product_id, s.user_id, s.quantity * p.price AS amount
FROM sales s INNER JOIN product p ON s.product_id = p.product_id),

cte2 AS (SELECT 
*,
MAX(total_spent) OVER(PARTITION BY user_id) AS max_spent
FROM (SELECT 
product_id, user_id, SUM(amount) AS total_spent
FROM cte
GROUP BY product_id, user_id) AS sub_query)


SELECT user_id ,product_id FROM cte2 WHERE total_spent = max_spent

"""

make_select(query)

,user_id,product_id
0,101,3
1,102,1
2,102,2
3,102,3


In [59]:
data = [[1, 1, 'Tony', 2000], [1, 2, 'Pronub', 21300], [1, 3, 'Tyrrox', 10800], [2, 1, 'Pam', 300], [2, 7, 'Bassem', 450], [2, 9, 'Hermione', 700], [3, 7, 'Bocaben', 100], [3, 2, 'Ognjen', 2200], [3, 13, 'Nyancat', 3300], [3, 15, 'Morninngcat', 7777]]
salaries = pd.DataFrame(data, columns=['company_id', 'employee_id', 'employee_name', 'salary']).astype({'company_id':'Int64', 'employee_id':'Int64', 'employee_name':'object', 'salary':'Int64'})


query = """  


WITH cte AS (SELECT 
company_id, employee_id, employee_name,
CASE 
     WHEN max_salary < 1000 THEN salary 
     WHEN max_salary BETWEEN 1000 AND 10000 THEN salary * 0.76
     WHEN max_salary > 10000 THEN salary * 0.51 
END AS salary
FROM (SELECT 
*,
MAX(salary) OVER(PARTITION BY company_id) AS max_salary
FROM salaries) AS sub_query)


SELECT company_id, employee_id, employee_name, ROUND(salary, 0) AS salary FROM cte 

"""

make_select(query)

,company_id,employee_id,employee_name,salary
0,1,1,Tony,1020.0
1,1,2,Pronub,10863.0
2,1,3,Tyrrox,5508.0
3,2,1,Pam,300.0
4,2,7,Bassem,450.0
5,2,9,Hermione,700.0
6,3,7,Bocaben,76.0
7,3,2,Ognjen,1672.0
8,3,13,Nyancat,2508.0
9,3,15,Morninngcat,5911.0


In [73]:
data = [[1, 'Jonathan D.', 'Eating'], [2, 'Jade W.', 'Singing'], [3, 'Victor J.', 'Singing'], [4, 'Elvis Q.', 'Eating'], [5, 'Daniel A.', 'Eating'], [6, 'Bob B.', 'Horse Riding']]
friends = pd.DataFrame(data, columns=['id', 'name', 'activity']).astype({'id':'Int64', 'name':'object', 'activity':'object'})
data = [[1, 'Eating'], [2, 'Singing'], [3, 'Horse Riding']]
activities = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})


query = """  

WITH cte AS (SELECT 
*,
COUNT(name) OVER(PARTITION BY type) AS total
FROM (SELECT 
f.id, f.name, a.name AS type, a.id AS activity_id
FROM friends f 
INNER JOIN activities a ON f.activity = a.name))

SELECT 
DISTINCT type AS activity
FROM cte
WHERE total != (SELECT MIN(total) FROM cte) AND total != (SELECT MAX(total) FROM cte)

"""

make_select(query)

,activity
0,Singing


In [89]:
data = [[1, '2022-01-07', -12], [1, '2022-03-07', 5], [1, '2022-07-07', 24], [2, '2022-08-07', 37], [2, '2022-08-17', 37], [3, '2022-02-07', -7], [3, '2022-12-07', -6]]
weather = pd.DataFrame(data, columns=['city_id', 'day', 'degree']).astype({'city_id':'Int64', 'day':'datetime64[ns]', 'degree':'Int64'})

query = """  

WITH cte AS (SELECT * FROM (SELECT 
*,
MAX(degree) OVER(PARTITION BY city_id ) AS max_degree
FROM weather)
WHERE degree = max_degree)


SELECT city_id,
MIN(day) AS day
, degree
FROM cte
GROUP BY city_id,degree
ORDER BY city_id


"""

make_select(query)

,city_id,day,degree
0,1,2022-07-07 00:00:00.000000,24
1,2,2022-08-07 00:00:00.000000,37
2,3,2022-12-07 00:00:00.000000,-6


In [ ]:
data = [[9, '2020-02-18', '2021-10-30'], [3, '2021-09-21', '2021-11-13'], [11, '2020-02-28', '2020-08-18'], [13, '2021-04-20', '2021-09-22'], [4, '2020-10-26', '2021-05-08'], [5, '2020-09-11', '2021-01-17']]
subscriptions = pd.DataFrame(data, columns=['account_id', 'start_date', 'end_date']).astype({'account_id':'Int64', 'start_date':'datetime64[ns]', 'end_date':'datetime64[ns]'})
data = [[14, 9, '2020-05-16'], [16, 3, '2021-10-27'], [18, 11, '2020-04-29'], [17, 13, '2021-08-08'], [19, 4, '2020-12-31'], [13, 5, '2021-01-05']]
streams = pd.DataFrame(data, columns=['session_id', 'account_id', 'stream_date']).astype({'session_id':'Int64', 'account_id':'Int64', 'stream_date':'datetime64[ns]'})



query = """   

WITH cte AS (SELECT 
sb.account_id, sb.start_date, sb.end_date, s.session_id, EXTRACT(YEAR, s.stream_date) AS year
FROM streams s 
INNER JOIN subscriptions sb ON s.account_id = sb.account_id
WHERE EXTRACT(YEAR FROM sb.start_date) = 2021 OR EXTRACT(YEAR FROM sb.end_date) = 2021)

SELECT COUNT(DISTINCT account_id) AS accounts_count FROM cte WHERE year != 2021

"""

make_select(query)

,accounts_count
0,2


In [127]:
data = [[2, 2, 95], [2, 3, 95], [1, 1, 90], [1, 2, 99], [3, 1, 80], [3, 2, 75], [3, 3, 82]]
enrollments = pd.DataFrame(data, columns=['student_id', 'course_id', 'grade']).astype({'student_id':'Int64', 'course_id':'Int64', 'grade':'Int64'})


query = """  

WITH cte AS (SELECT * FROM (SELECT 
*,
MAX(grade) OVER(PARTITION BY student_id) AS high_grade
FROM enrollments )
WHERE grade = high_grade)

SELECT student_id ,MIN(course_id) AS course_id, grade 
FROM cte
GROUP BY student_id, grade

"""

make_select(query)

,student_id,course_id,grade
0,1,2,99
1,2,2,95
2,3,3,82


In [144]:
data = [['1', '2020-11-28'], ['1', '2020-10-20'], ['1', '2020-12-3'], ['2', '2020-10-5'], ['2', '2020-12-9'], ['3', '2020-11-11']]
user_visits = pd.DataFrame(data, columns=['user_id', 'visit_date']).astype({'user_id':'Int64', 'visit_date':'datetime64[ns]'})


query = """ 

WITH cte AS (SELECT 
*,
LAG(visit_date) OVER(PARTITION BY user_id ORDER BY visit_date) AS prev_visits
FROM user_visits),

cte2 AS (SELECT user_id,
MAX(julianday(visit_date)- julianday(prev_visits)) AS diff
FROM cte
GROUP BY user_id),


cte3 AS (SELECT user_id,
julianday('2021-01-01') - MAX(julianday(visit_date)) AS diff
FROM user_visits
GROUP BY user_id)


SELECT user_id, MAX(diff) AS biggest_window
FROM (SELECT * FROM cte2
UNION ALL
SELECT * FROM cte3) 
GROUP BY user_id

"""

make_select(query)

,user_id,biggest_window
0,1,39.0
1,2,65.0
2,3,51.0


In [ ]:
data = [[1, 1, '2019-08-01', '2019-08-02'], [2, 2, '2019-08-01', '2019-08-01'], [3, 1, '2019-08-01', '2019-08-01'], [4, 3, '2019-08-02', '2019-08-13'], [5, 3, '2019-08-02', '2019-08-02'], [6, 2, '2019-08-02', '2019-08-02'], [7, 4, '2019-08-03', '2019-08-03'], [8, 1, '2019-08-03', '2019-08-03'], [9, 5, '2019-08-04', '2019-08-18'], [10, 2, '2019-08-04', '2019-08-18']]
delivery = pd.DataFrame(data, columns=['delivery_id', 'customer_id', 'order_date', 'customer_pref_delivery_date']).astype({'delivery_id':'Int64', 'customer_id':'Int64', 'order_date':'datetime64[ns]', 'customer_pref_delivery_date':'datetime64[ns]'})

query = """  

WITH cte AS (SELECT 
*,
CASE 
   WHEN order_date = customer_pref_delivery_date THEN 'immediate' 
   ELSE 'scheduled'
END AS order_type,
COUNT(delivery_id) OVER(PARTITION BY order_date ORDER BY order_date) AS number_orders
FROM delivery)


SELECT 
order_date, 
ROUND(COUNT(CASE WHEN order_type = 'immediate' THEN order_type END) * 1.0 / number_orders * 100, 2) AS immediate_percentage
FROM cte
GROUP BY order_date
ORDER BY order_date


"""

make_select(query)

,order_date,immediate_percentage
0,2019-08-01 00:00:00.000000,66.67
1,2019-08-02 00:00:00.000000,66.67
2,2019-08-03 00:00:00.000000,100.00
3,2019-08-04 00:00:00.000000,0.00


In [182]:
events_data = [
    [1, 'reviews', 3],
    [3, 'reviews', 3],
    [1, 'ads', 11],
    [2, 'ads', 7],
    [3, 'ads', 11],
    [1, 'page views', 3],
    [2, 'page views', 12],
    [3, 'page views', 10]
]

events = pd.DataFrame(events_data, columns=['business_id', 'event_type', 'occurrences'])


query = """  

WITH cte AS (SELECT 
SUM(CASE WHEN event_type = 'reviews' THEN occurrences END) * 1.0 
/ COUNT(CASE WHEN event_type = 'reviews' THEN event_type END) AS reviews_avg,

SUM(CASE WHEN event_type = 'ads' THEN occurrences END) * 1.0 
/ COUNT(CASE WHEN event_type = 'ads' THEN event_type END) AS ads_avg,

SUM(CASE WHEN event_type = 'page views' THEN occurrences END) * 1.0 
/ COUNT(CASE WHEN event_type = 'page views' THEN event_type END) AS avg_page_views
FROM events),

cte2 AS (SELECT 
business_id,
COALESCE(SUM(CASE WHEN event_type = 'reviews' THEN occurrences END),0) AS review,
COALESCE(SUM(CASE WHEN event_type = 'ads' THEN occurrences END),0) AS ads,
COALESCE(SUM(CASE WHEN event_type = 'page views' THEN occurrences END),0) AS page_view
FROM events
GROUP BY business_id),

cte3 AS (SELECT 
business_id,
CASE WHEN review > (SELECT reviews_avg FROM cte) THEN 1 ELSE 0 END AS review,
CASE WHEN ads > (SELECT ads_avg FROM cte) THEN 1 ELSE 0 END AS ads,
CASE WHEN page_view > (SELECT avg_page_views FROM cte) THEN 1 ELSE 0 END AS page
FROM cte2
GROUP BY business_id)

SELECT 
business_id
FROM cte3
GROUP BY business_id
HAVING review+ads+page >= 2
"""

make_select(query)

,business_id
0,3


In [188]:
data = [[1, 'Winston'], [2, 'Jonathan'], [3, 'Annabelle'], [4, 'Marwan'], [5, 'Khaled']]
customers = pd.DataFrame(data, columns=['customer_id', 'name']).astype({'customer_id':'Int64', 'name':'object'})
data = [[1, '2020-07-31', 1, 1], [2, '2020-7-30', 2, 2], [3, '2020-08-29', 3, 3], [4, '2020-07-29', 4, 1], [5, '2020-06-10', 1, 2], [6, '2020-08-01', 2, 1], [7, '2020-08-01', 3, 1], [8, '2020-08-03', 1, 2], [9, '2020-08-07', 2, 3], [10, '2020-07-15', 1, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'customer_id', 'product_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'customer_id':'Int64', 'product_id':'Int64'})
data = [[1, 'keyboard', 120], [2, 'mouse', 80], [3, 'screen', 600], [4, 'hard disk', 450]]
products = pd.DataFrame(data, columns=['product_id', 'product_name', 'price']).astype({'product_id':'Int64', 'product_name':'object', 'price':'Int64'})


query = """  

WITH cte AS (SELECT 
o.order_id, o.order_date, o.customer_id, o.product_id,
c.name, p.product_name, p.price,
MAX(order_date) OVER(PARTITION BY product_name ) AS recent_order
FROM orders o 
INNER JOIN customers c ON o.customer_id = c.customer_id 
INNER JOIN products p ON o.product_id = p.product_id)


SELECT product_name, product_id, order_id, order_date
FROM cte
WHERE order_date = recent_order

"""

make_select(query)

,product_name,product_id,order_id,order_date
0,keyboard,1,6,2020-08-01 00:00:00.000000
1,keyboard,1,7,2020-08-01 00:00:00.000000
2,mouse,2,8,2020-08-03 00:00:00.000000
3,screen,3,3,2020-08-29 00:00:00.000000


In [227]:
data = [[1, 'John', 'Doe'], [2, 'Jane', 'Smith'], [3, 'Alice', 'Johnson'], [4, 'Michael', 'Brown'], [5, 'Emily', 'Davis']]
contacts = pd.DataFrame(data, columns=['id', 'first_name', 'last_name']).astype({'id':'Int64', 'first_name':'object', 'last_name':'object'})

data = [[1, 'incoming', 120], [1, 'outgoing', 180], [2, 'incoming', 300], [2, 'outgoing', 240], [3, 'incoming', 150], [3, 'outgoing', 360], [4, 'incoming', 420], [4, 'outgoing', 200], [5, 'incoming', 180], [5, 'outgoing', 280]]
calls = pd.DataFrame(data, columns=['contact_id', 'type', 'duration']).astype({'contact_id': 'Int64', 'type': 'category', 'duration': 'Int64'})


query = """  

WITH cte AS (SELECT 
first_name, type, 
duration * 1.0 / 60 AS total
FROM contacts c INNER JOIN calls cl ON c.id = cl.contact_id
GROUP BY first_name,type),

cte2 AS (SELECT 
*,
MAX(total)  OVER(PARTITION BY first_name) AS max_total
FROM cte)

SELECT
first_name, type, total AS duration
FROM (SELECT 
*,
RANK() OVER(PARTITION BY type ORDER BY total DESC) AS ranking
FROM cte2 )
WHERE ranking <= 3
ORDER BY type, duration, total DESC


"""

make_select(query)

,first_name,type,duration
0,Emily,incoming,3.000000
1,Jane,incoming,5.000000
2,Michael,incoming,7.000000
3,Jane,outgoing,4.000000
4,Emily,outgoing,4.666667
5,Alice,outgoing,6.000000


In [238]:
servers_data = [
    [3, '2023-11-04 16:29:47', 'start'],
    [3, '2023-11-05 01:49:47', 'stop'],
    [3, '2023-11-25 01:37:08', 'start'],
    [3, '2023-11-25 03:50:08', 'stop'],
    [1, '2023-11-13 03:05:31', 'start'],
    [1, '2023-11-13 11:10:31', 'stop'],
    [4, '2023-11-29 15:11:17', 'start'],
    [4, '2023-11-29 15:42:17', 'stop'],
    [4, '2023-11-20 00:31:44', 'start'],
    [4, '2023-11-20 07:03:44', 'stop'],
    [1, '2023-11-20 00:27:11', 'start'],
    [1, '2023-11-20 01:41:11', 'stop'],
    [3, '2023-11-04 23:16:48', 'start'],
    [3, '2023-11-05 01:15:48', 'stop'],
    [4, '2023-11-30 15:09:18', 'start'],
    [4, '2023-11-30 20:48:18', 'stop'],
    [4, '2023-11-25 21:09:06', 'start'],
    [4, '2023-11-26 04:58:06', 'stop'],
    [5, '2023-11-16 19:42:22', 'start'],
    [5, '2023-11-16 21:08:22', 'stop']
]

servers = pd.DataFrame(servers_data, columns=['server_id', 'status_time', 'session_status'])


query = """  

WITH cte AS (SELECT *,
julianday(status_time) - julianday(s) AS total
FROM  (SELECT 
*,
LAG(CASE WHEN session_status = 'start' THEN status_time END) OVER(PARTITION BY server_id ORDER BY status_time) AS s
FROM servers
))


SELECT FLOOR(SUM(total)) AS total_uptime_days FROM cte
"""

make_select(query)

,total_uptime_days
0,1


In [8]:
data = [[1, 'A'], [2, 'B'], [3, 'C'], [4, 'D'], [5, 'E']]
candidate = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[1, 2], [2, 4], [3, 3], [4, 2], [5, 5]]
vote = pd.DataFrame(data, columns=['id', 'candidateId']).astype({'id':'Int64', 'candidateId':'Int64'})


query = """  

WITH cte AS (SELECT 
v.candidateId, c.name, COUNT(v.id) AS total_votes
FROM candidate c INNER JOIN vote v ON c.id = v.candidateId
GROUP BY v.candidateId, c.name
)

SELECT 
name
FROM cte
WHERE total_votes = (SELECT MAX(total_votes) FROM cte)

"""

make_select(query)

,name
0,B


In [ ]:
data = [[8, 4, '2021-08-24 22:46:07', 'Houston'], [4, 8, '2021-08-24 22:57:13', 'Houston'], [5, 1, '2021-08-11 21:28:44', 'Houston'], [8, 3, '2021-08-17 22:04:15', 'Houston'], [11, 3, '2021-08-17 13:07:00', 'New York'], [8, 11, '2021-08-17 14:22:22', 'New York']]
calls = pd.DataFrame(data, columns=['caller_id', 'recipient_id', 'call_time', 'city']).astype({'caller_id':'Int64', 'recipient_id':'Int64', 'call_time':'datetime64[ns]', 'city':'object'})


query = """   

WITH cte AS (SELECT 
city,
DATE_PART('hour', call_time)  AS hour,
COUNT(recipient_id) AS total_calls
FROM calls
GROUP BY city, hour)


SELECT 
city, hour AS peak_calling_hour, total_calls AS number_of_calls
FROM (SELECT 
*,
MAX(total_calls) OVER(PARTITION BY city) AS max_calls
FROM cte)
WHERE total_calls = max_calls
ORDER BY peak_calling_hour DESC, city DESC

"""

make_select(query)

,city,peak_calling_hour,number_of_calls
0,Houston,22,3
1,New York,14,1
2,New York,13,1


In [55]:
data = [[11, '2023-11-03', 1126], [15, '2023-11-10', 7473], [17, '2023-11-17', 2414], [12, '2023-11-24', 9692], [8, '2023-11-24', 5117], [1, '2023-11-24', 5241], [10, '2023-11-22', 8266], [13, '2023-11-21', 12000]]
purchases = pd.DataFrame(data, columns=['user_id', 'purchase_date', 'amount_spend']).astype({'user_id':'Int64', 'purchase_date':'datetime64[ns]', 'amount_spend':'Int64'})

data = [[11, 'Premium'], [15, 'VIP'], [17, 'Standard'], [12, 'VIP'], [8, 'Premium'], [1, 'VIP'], [10, 'Standard'], [13, 'Premium']]
users = pd.DataFrame(data, columns=['user_id', 'membership']).astype({
    'user_id': 'Int64',
    'membership': pd.CategoricalDtype(categories=['Standard', 'Premium', 'VIP'])
})


query = """   

WITH cte AS (SELECT 
p.user_id, p.purchase_date, p.amount_spend, u.membership, 
strftime('%w', p.purchase_date) AS week_day,
(strftime('%d', p.purchase_date) - 1) / 7 + 1 AS week_of_month
FROM purchases p INNER JOIN users u ON p.user_id = u.user_id
WHERE strftime('%Y-%m', p.purchase_date) = '2023-11' AND  membership IN ('VIP','Premium')
),

cte2 AS (SELECT 
week_of_month, membership, SUM(amount_spend) AS total
FROM cte
WHERE week_day = '5'
GROUP BY week_of_month, membership),


weeks AS (SELECT DISTINCT week_of_month FROM cte),

types AS (SELECT DISTINCT membership FROM cte),

combination AS (SELECT w.week_of_month, t.membership
FROM weeks w 
CROSS JOIN types t)

SELECT 
cm.week_of_month, 
cm.membership,
COALESCE(c.total, 0) AS total_amount
FROM combination cm 
LEFT JOIN cte2 c ON cm.week_of_month = c.week_of_month AND cm.membership = c.membership
GROUP BY cm.week_of_month, cm.membership
ORDER BY cm.week_of_month, cm.membership

"""
# SELECT DISTINCT membership FROM cte
# AND week_day = '5' AND  membership IN ('VIP','Premium')

make_select(query)

,week_of_month,membership,total_amount
0,1,Premium,1126
1,1,VIP,0
2,2,Premium,0
3,2,VIP,7473
4,3,Premium,0
5,3,VIP,0
6,4,Premium,5117
7,4,VIP,14933


In [ ]:
data = {
    "lot_id": [1, 1, 2, 2, 3],
    "car_id": [1001, 1001, 1001, 1002, 1002],
    "entry_time": [
        "2023-06-01 08:00:00",
        "2023-06-02 10:00:00",
        "2023-06-03 12:00:00",
        "2023-06-04 09:00:00",
        "2023-06-05 11:00:00",
    ],
    "exit_time": [
        "2023-06-01 09:30:00",
        "2023-06-02 11:00:00",
        "2023-06-03 14:00:00",
        "2023-06-04 10:45:00",
        "2023-06-05 12:30:00",
    ],
    "fee_paid": [5, 5, 8, 6, 4],
}

df = pd.DataFrame(data)

# Convert entry_time and exit_time to datetime
df["entry_time"] = pd.to_datetime(df["entry_time"])
df["exit_time"] = pd.to_datetime(df["exit_time"])




query = """  

WITH cte AS (SELECT 
*,
(julianday(exit_time) - julianday(entry_time) ) * 24 AS time_spent,
MAX((julianday(exit_time) - julianday(entry_time) ) * 24) OVER(PARTITION BY car_id) AS max_lot
FROM df),


cte3 AS (SELECT 
car_id,
SUM(fee_paid) AS total_fee_paid,
ROUND(SUM(fee_paid) * 1.0 / SUM(time_spent), 2) AS avg_hourly_fee
FROM cte 
GROUP BY car_id
),

cte2 AS (SELECT 
*,
MAX(total) OVER(PARTITION BY car_id) AS most
FROM (SELECT 
lot_id, car_id,
SUM(time_spent) AS total
FROM cte
GROUP BY lot_id, car_id)),


final AS (SELECT lot_id, car_id FROM cte2 WHERE total=most
)


SELECT DISTINCT b.car_id, b.total_fee_paid, b.avg_hourly_fee, a.lot_id AS most_time_lot
FROM final a INNER JOIN cte3 b ON a.car_id = b.car_id
ORDER BY b.car_id


"""



make_select(query)

,car_id,total_fee_paid,avg_hourly_fee,most_time_lot
0,1001,18,4.00,1
1,1002,10,3.08,2


In [ ]:
data = [[7274, 123, 'open', 4.5], [2425, 123, 'send', 3.5], [1413, 456, 'send', 5.67], [2536, 456, 'open', 3.0], [8564, 456, 'send', 8.24], [5235, 789, 'send', 6.24], [4251, 123, 'open', 1.25], [1435, 789, 'open', 5.25]]
activities = pd.DataFrame(data, columns=['activity_id', 'user_id', 'activity_type', 'time_spent']).astype({'activity_id':'Int64', 'user_id':'Int64', 'activity_type':'object', 'time_spent':'Float64'})

data = [[123, '31-35'], [789, '21-25'], [456, '26-30']]
age = pd.DataFrame(data, columns=['user_id', 'age_bucket']).astype({'user_id':'Int64', 'age_bucket':'object'})


query = """ 

WITH cte AS (SELECT 
a.activity_id, a.user_id, a.time_spent,a.activity_type, ag.age_bucket,
SUM(a.time_spent) OVER(PARTITION BY ag.age_bucket) AS total_spent
FROM activities a INNER JOIN age ag ON a.user_id = ag.user_id)

SELECT 
age_bucket,
ROUND(SUM(CASE WHEN activity_type = 'send' THEN time_spent END) / total_spent * 100, 2) AS send_perc,
ROUND(SUM(CASE WHEN activity_type = 'open' THEN time_spent END) / total_spent * 100, 2) AS open_perc 
FROM cte 
GROUP BY age_bucket
ORDER BY age_bucket

"""

make_select(query)

,age_bucket,send_perc,open_perc
0,21-25,54.31,45.69
1,26-30,82.26,17.74
2,31-35,37.84,62.16


In [177]:
users_data = {
    "seller_id": [1, 2, 3, 4],
    "join_date": ["2019-01-01", "2019-02-09", "2019-01-19", "2019-05-21"],
    "favorite_brand": ["Lenovo", "Samsung", "LG", "HP"],
}
users = pd.DataFrame(users_data)

# Orders table
orders_data = {
    "order_id": [1, 2, 3, 4, 5, 6, 7, 8],
    "order_date": [
        "2019-08-01", "2019-08-02", "2019-08-03", 
        "2019-08-04", "2019-08-04", "2019-08-05", 
        "2019-08-05", "2019-08-06"
    ],
    "item_id": [4, 2, 3, 1, 1, 2, 3, 2],
    "seller_id": [2, 3, 3, 2, 4, 4, 1, 1],
}
orders = pd.DataFrame(orders_data)

# Items table
items_data = {
    "item_id": [1, 2, 3, 4],
    "item_brand": ["Samsung", "Lenovo", "LG", "HP"],
}
items = pd.DataFrame(items_data)

# Ensure date columns are properly parsed
users["join_date"] = pd.to_datetime(users["join_date"])
orders["order_date"] = pd.to_datetime(orders["order_date"])




query = """ 

WITH cte  AS (SELECT 
o.order_id, o.item_id, o.seller_id,
u.join_date, u.favorite_brand, i.item_brand
FROM orders o 
INNER JOIN users u ON o.seller_id = u.seller_id
INNER JOIN items i ON o.item_id = i.item_id),

cte2 AS (SELECT 
seller_id, 
COUNT(DISTINCT CASE WHEN favorite_brand != item_brand THEN item_id END) AS total
FROM cte
GROUP BY seller_id),

cte3 AS (SELECT 
*,
DENSE_RANK() OVER(ORDER BY total DESC) AS ranking
FROM cte2)

SELECT 
seller_id, total AS num_items
FROM cte3 WHERE ranking = 1

"""

make_select(query)

,seller_id,num_items
0,4,2


In [257]:
transactions = pd.DataFrame({
    "transaction_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43],
    "customer_id": [1, 2, 2, 2, 2, 2, 2, 2, 3, 3, 3, 3, 3, 4, 4, 4, 5, 5, 5, 5, 6, 6, 6, 6, 6, 6, 6, 7, 8, 8, 8, 8, 8, 8, 8, 8, 8, 8, 9, 9, 9, 9, 9],
    "product_id": [11, 7, 5, 7, 7, 4, 5, 11, 13, 5, 1, 1, 4, 13, 1, 6, 8, 11, 9, 8, 7, 5, 8, 11, 10, 3, 2, 1, 10, 1, 13, 8, 9, 7, 6, 13, 6, 8, 1, 8, 7, 7, 10],
    "transaction_date": ["2023-05-31", "2022-05-18", "2021-11-26", "2022-10-05", "2021-01-09", "2021-06-05", "2022-12-06", "2022-01-04", "2020-10-08", "2021-03-08", "2023-02-11", "2020-12-16", "2021-08-30", "2021-09-30", "2023-05-27", "2023-11-03", "2020-07-19", "2020-09-16", "2022-09-23", "2021-12-29", "2022-11-24", "2020-02-12", "2022-01-20", "2022-05-24", "2022-02-06", "2021-12-14", "2022-05-16", "2021-05-26", "2023-08-24", "2023-04-29", "2022-08-31", "2021-11-20", "2021-10-09", "2020-06-27", "2020-04-30", "2023-11-05", "2020-10-18", "2023-05-14", "2021-02-08", "2022-06-11", "2021-10-12", "2022-09-20", "2023-07-25"],
    "amount": [806.28, 636.91, 436.56, 415.42, 211.34, 154.17, 893.28, 981.28, 866.01, 761.55, 342.99, 235.64, 930.63, 568.86, 915.26, 523.33, 551.19, 810.67, 653.77, 652.36, 76.76, 928.62, 705.88, 571.33, 766.18, 109.27, 130.03, 79.44, 759.39, 266.54, 647.76, 452.37, 810.1, 673.22, 987.39, 936.74, 333.27, 179.86, 267.11, 744.45, 372.71, 375.41, 559.13]
})

# Product DataFrame
products = pd.DataFrame({
    "product_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13],
    "category": ["D", "A", "C", "A", "D", "C", "D", "E", "D", "C", "E", "E", "B"],
    "price": [335, 281.4, 181.61, 167.91, 448.89, 93.47, 120.58, 75.9, 32.5, 149.96, 148.57, 167.27, 137.06]
})


transactions["transaction_date"] = pd.to_datetime(transactions["transaction_date"])



query = """  

WITH cte AS (SELECT 
t.transaction_id, t.customer_id, t.product_id, t.transaction_date, t.amount, p.category, p.price
FROM transactions t INNER JOIN products p ON t.product_id = p.product_id),

cte2 AS (SELECT 
customer_id,
ROUND(SUM(amount), 2) AS total_amount, 
COUNT(transaction_id) AS transaction_count,
COUNT(DISTINCT category) AS unique_categories,
ROUND(AVG(amount),2) AS avg_transaction_amount,
ROUND(COUNT(transaction_id) * 10 + SUM(amount) / 100, 2) AS loyalty_score
FROM cte
GROUP BY customer_id),

cte3 AS (SELECT 
customer_id, 
category,
COUNT(*) AS freq, 
MAX(transaction_date) AS latest_date,
RANK() OVER(PARTITION BY customer_id ORDER BY COUNT(*) DESC, MAX(transaction_date) DESC) AS ranking
FROM cte
GROUP BY customer_id,category),


cte4 AS (SELECT customer_id, category FROM cte3 WHERE ranking = 1)


SELECT 
a.customer_id, a.total_amount, a.transaction_count, a.unique_categories, a.avg_transaction_amount, b.category AS top_category , a.loyalty_score
FROM cte2 a INNER JOIN cte4 b ON a.customer_id = b.customer_id


"""

make_select(query)

,customer_id,total_amount,transaction_count,unique_categories,avg_transaction_amount,top_category,loyalty_score
0,1,806.28,1,1,806.28,E,18.06
1,2,3728.96,7,3,532.71,D,107.29
2,3,3136.82,5,3,627.36,D,81.37
3,4,2007.45,3,3,669.15,C,50.07
4,5,2667.99,4,2,667.00,E,66.68
5,6,3288.07,7,4,469.72,D,102.88
6,7,79.44,1,1,79.44,D,10.79
7,8,6046.64,10,4,604.66,C,160.47
8,9,2318.81,5,3,463.76,D,73.19


In [270]:
survey_log = pd.DataFrame({
    "id": [5, 5, 6, 6, 5, 5, 5],
    "action": ["show", "answer", "show", "answer", "show", "answer", "skip"],
    "question_id": [285, 285, 285, 285, 369, 369, 369],
    "answer_id": [None, 123, None, 123, None, 123, None],
    "q_num": [1, 1, 1, 1, 2, 2, 2],
    "timestamp": [1, 2, 3, 4, 5, 6, 7]
})

query = """  

WITH cte AS (SELECT 
question_id,
COUNT(CASE WHEN action = 'answer' THEN id END) * 1.0 / COUNT(CASE WHEN action = 'show' THEN id END) AS rate
FROM survey_log
GROUP BY question_id
)

SELECT question_id AS survey_log FROM (SELECT 
*,
RANK() OVER(ORDER BY rate DESC, question_id ASC) AS ranking
FROM cte) WHERE ranking = 1


"""

make_select(query)

,survey_log
0,285


In [ ]:
data = [[4, 'IOS', 'Programming'], [13, 'IOS', 'Sports'], [14, 'Android', 'Reading'], [8, 'Web', 'Reading'], [12, 'Web', 'Reading'], [18, 'Web', 'Programming']]
experiments = pd.DataFrame(data, columns=['experiment_id', 'platform', 'experiment_name']).astype({'experiment_id':'Int64', 'platform':'object', 'experiment_name':'object'})


query = """  

WITH cte AS (SELECT 
platform, experiment_name,
COUNT(experiment_id) AS total
FROM experiments
GROUP BY platform, experiment_name),


all_com AS (SELECT platform, experiment_name
              FROM (SELECT 'IOS' AS platform UNION SELECT 'Android' UNION SELECT 'Web') E1 
                   CROSS JOIN
                   (SELECT 'Programming' AS experiment_name UNION SELECT 'Sports' UNION SELECT 'Reading') E2

)

SELECT 
a.platform, a.experiment_name, COALESCE(c.total,0) AS num_experiments
FROM all_com a
LEFT JOIN cte c ON a.platform = c.platform AND a.experiment_name = c.experiment_name

"""

make_select(query)

,platform,experiment_name,num_experiments
0,Android,Programming,0
1,Android,Reading,1
2,Android,Sports,0
3,IOS,Programming,1
4,IOS,Reading,0
5,IOS,Sports,1
6,Web,Programming,1
7,Web,Reading,2
8,Web,Sports,0


In [ ]:
data = [[4, 2, '2022-03-13'], [1, 5, '2022-02-11'], [3, 7, '2022-06-19'], [6, 2, '2022-03-20'], [5, 7, '2022-06-19'], [2, 2, '2022-06-08']]
purchases = pd.DataFrame(data, columns=['purchase_id', 'user_id', 'purchase_date']).astype({'purchase_id':'Int64', 'user_id':'Int64', 'purchase_date':'datetime64[ns]'})


query = """  

WITH cte AS (SELECT 
    user_id,
    purchase_date,
   
    ) AS purchase_count COUNT(purchase_id) OVER (PARTITION BY user_id  ORDER BY purchase_date RANGE BETWEEN INTERVAL '7 days' PRECEDING AND CURRENT ROW
FROM purchases)

SELECT DISTINCT user_id FROM cte WHERE purchase_count >= 2

"""

make_select(query)

In [11]:
data = [[1, 3, 5, '2019-08-01'], [3, 4, 5, '2019-08-01'], [1, 3, 6, '2019-08-02'], [2, 7, 7, '2019-08-01'], [2, 7, 6, '2019-08-02'], [4, 7, 1, '2019-07-22'], [3, 4, 4, '2019-07-21'], [3, 4, 4, '2019-07-21']]
views = pd.DataFrame(data, columns=['article_id', 'author_id', 'viewer_id', 'view_date']).astype({'article_id':'Int64', 'author_id':'Int64', 'viewer_id':'Int64', 'view_date':'datetime64[ns]'})


query = """   

WITH cte  AS (SELECT 
viewer_id, view_date, COUNT(DISTINCT article_id) AS views
FROM views
GROUP BY viewer_id, view_date)


SELECT DISTINCT viewer_id AS id
FROM cte 
WHERE views >= 2
ORDER BY viewer_id ASC

"""

make_select(query)

,id
0,5
1,6


In [ ]:
data = [[1, 20], [2, 12], [3, 2]]
employees = pd.DataFrame(data, columns=['employee_id', 'needed_hours']).astype({'employee_id':'Int64', 'needed_hours':'Int64'})
data = [[1, '2022-10-01 09:00:00', '2022-10-01 17:00:00'], [1, '2022-10-06 09:05:04', '2022-10-06 17:09:03'], [1, '2022-10-12 23:00:00', '2022-10-13 03:00:01'], [2, '2022-10-29 12:00:00', '2022-10-29 23:58:58']]
logs = pd.DataFrame(data, columns=['employee_id', 'in_time', 'out_time']).astype({'employee_id':'Int64', 'in_time':'datetime64[ns]', 'out_time':'datetime64[ns]'})


query = """  

WITH cte AS (SELECT employee_id, SUM((julianday(out_time) - julianday(in_time)) * 24) AS worked
FROM logs 
GROUP BY employee_id
),

cte2 AS (SELECT 
e.employee_id, e.needed_hours, COALESCE(c.worked,0) AS worked
FROM employees e LEFT JOIN cte c ON e.employee_id = c.employee_id)


SELECT employee_id FROM cte2 WHERE needed_hours > worked

"""

make_select(query)

,employee_id
0,2
1,3


In [54]:
data = [['2020-05-01', 'apples', 10], ['2020-05-01', 'oranges', 8], ['2020-05-02', 'apples', 15], ['2020-05-02', 'oranges', 15], ['2020-05-03', 'apples', 20], ['2020-05-03', 'oranges', 0], ['2020-05-04', 'apples', 15], ['2020-05-04', 'oranges', 16]]
sales = pd.DataFrame(data, columns=['sale_date', 'fruit', 'sold_num']).astype({'sale_date':'datetime64[ns]', 'fruit':'object', 'sold_num':'Int64'})


query = """ 

SELECT 
sale_date,
SUM(CASE WHEN fruit = 'apples' THEN sold_num END)-
SUM(CASE WHEN fruit = 'oranges' THEN sold_num END) AS diff 
FROM sales
GROUP BY sale_date
ORDER BY sale_date
"""

make_select(query)

,sale_date,diff
0,2020-05-01 00:00:00.000000,2
1,2020-05-02 00:00:00.000000,0
2,2020-05-03 00:00:00.000000,20
3,2020-05-04 00:00:00.000000,-1


In [82]:
data = [[4, 2], [2, 3], [3, 1], [1, 4]]
data = pd.DataFrame(data, columns=['first_col', 'second_col']).astype({'first_col':'Int64', 'second_col':'Int64'})

query = """  

WITH right_col AS (SELECT first_col, ROW_NUMBER() OVER(ORDER BY first_col) AS ranking FROM data ORDER BY first_col),

left_col AS (SELECT second_col, ROW_NUMBER() OVER(ORDER BY second_col DESC) AS ranking FROM data ORDER BY second_col DESC)

SELECT r.first_col, l.second_col FROM right_col r INNER JOIN left_col l ON r.ranking = l.ranking

"""

make_select(query)

,first_col,second_col
0,1,4
1,2,3
2,3,2
3,4,1


In [106]:
data = [[1, 1], [2, 0], [3, 1], [4, 1], [5, 1]]
cinema = pd.DataFrame(data, columns=['seat_id', 'free']).astype({'seat_id':'Int64', 'free':'int'})

query = """   

SELECT * FROM (SELECT 
*,
LAG(seat_id,1) OVER(ORDER BY seat_id) AS next,
LEAD(seat_id,1) OVER(ORDER BY seat_id) AS prev
FROM cinema)

"""


make_select(query)

,seat_id,free,next,prev
0,1,1,NaN,2.0
1,2,0,1.0,3.0
2,3,1,2.0,4.0
3,4,1,3.0,5.0
4,5,1,4.0,NaN


In [ ]:
data = {
    "user_id": [1, 6, 1, 6, 3, 5, 5, 4, 2, 4, 5, 4],
    "time_stamp": [
        "2022-07-27 08:41:59", "2022-04-18 12:03:09", "2022-11-24 02:31:07", 
        "2022-07-29 18:25:31", "2022-01-10 15:45:25", "2022-04-20 09:03:00", 
        "2022-01-27 12:54:43", "2022-07-11 21:07:11", "2022-03-19 19:24:02", 
        "2022-02-11 18:28:11", "2022-02-01 05:05:34", "2022-11-24 21:50:35"
    ],
    "amount": [1059, 4523, 2632, 7636, 3149, 4416, 512, 6443, 4678, 7134, 6756, 1260],
}

purchases = pd.DataFrame(data)

# Ensure time_stamp column is in datetime format
purchases["time_stamp"] = pd.to_datetime(purchases["time_stamp"])


query = """  


WITH cte AS (SELECT * FROM 
purchases
WHERE time_stamp BETWEEN '2022-03-08' AND '2022-03-20')

SELECT COUNT(DISTINCT user_id) AS user_cnt FROM cte WHERE amount >= 1000

"""


make_select(query)

,user_cnt
0,1


In [31]:
data = [[1, 2, '2016-03-01', 5], [1, 2, '2016-05-02', 6], [2, 3, '2017-06-25', 1], [3, 1, '2016-03-02', 0], [3, 4, '2018-07-03', 5]]
activity = pd.DataFrame(data, columns=['player_id', 'device_id', 'event_date', 'games_played']).astype({'player_id':'Int64', 'device_id':'Int64', 'event_date':'datetime64[ns]', 'games_played':'Int64'})


query = """  

WITH cte AS (SELECT 
player_id, device_id, MIN(event_date) AS first_date,
RANK() OVER(PARTITION BY player_id ORDER BY  MIN(event_date)) AS ranking
FROM activity
GROUP BY player_id, device_id)


SELECT player_id, device_id FROM cte WHERE ranking = 1
"""

make_select(query)

,player_id,device_id
0,1,2
1,2,3
2,3,1


In [91]:
customers_data = {
    "customer_id": [1, 2, 3, 4],
    "customer_name": ["Daniel", "Diana", "Elizabeth", "Jhon"]
}
customers = pd.DataFrame(customers_data)

orders_data = {
    "order_id": [10, 20, 30, 40, 50, 60, 70, 80, 90],
    "customer_id": [1, 1, 1, 1, 2, 3, 3, 3, 4],
    "product_name": ["A", "B", "D", "C", "A", "A", "B", "A", "C"]
}
orders= pd.DataFrame(orders_data)





query = """  

WITH cte AS (SELECT 
c.customer_id, c.customer_name,o.product_name
FROM orders o INNER JOIN customers c ON o.customer_id = c.customer_id),


buy_ab AS (SELECT 
customer_id, customer_name
FROM cte
WHERE product_name IN ('A', 'B')
GROUP BY customer_id, customer_name
HAVING COUNT(DISTINCT product_name) = 2
),

only_c AS (SELECT customer_id, customer_name, product_name
FROM cte
WHERE product_name = 'C')


SELECT 
a.customer_id, a.customer_name
FROM buy_ab a LEFT JOIN only_c b ON a.customer_id = b.customer_id
WHERE b.customer_id IS NULL



"""

make_select(query)

,customer_id,customer_name
0,3,Elizabeth


In [ ]:
data = [[3, '2020-03-21 10:16:13'], [7, '2020-01-04 13:57:59'], [2, '2020-07-29 23:09:44'], [6, '2020-12-09 10:39:37']]
signups = pd.DataFrame(data, columns=['user_id', 'time_stamp']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]'})
data = [[3, '2021-01-06 03:30:46', 'timeout'], [3, '2021-01-06 03:37:45', 'timeout'], [7, '2021-06-12 11:57:29', 'confirmed'], [7, '2021-06-13 11:57:30', 'confirmed'], [2, '2021-01-22 00:00:00', 'confirmed'], [2, '2021-01-23 00:00:00', 'timeout'], [6, '2021-10-23 14:14:14', 'confirmed'], [6, '2021-10-24 14:14:13', 'timeout']]
confirmations = pd.DataFrame(data, columns=['user_id', 'time_stamp', 'action']).astype({'user_id':'Int64', 'time_stamp':'datetime64[ns]', 'action':'object'})


query = """ 

SELECT
*,
COUNT(action) 
OVER(PARTITION BY user_id ORDER BY time_stamp RANGE BETWEEN INTERVAL '24 hours' PRECEDING AND CURRENT ROW)
FROM confirmations

"""

make_select(query)

In [111]:
data = [[1, None], [2, None], [1, None], [12, None], [3, 1], [5, 2], [3, 1], [4, 1], [9, 1], [10, 2], [6, 7]]
submissions = pd.DataFrame(data, columns=['sub_id', 'parent_id']).astype({'sub_id':'Int64', 'parent_id':'Int64'})


query = """  

WITH cte AS (SELECT 
parent_id, 
COUNT(DISTINCT sub_id) AS comments_count
FROM submissions
WHERE parent_id IS NOT NULL
GROUP BY parent_id),

cte2 AS (SELECT 
DISTINCT sub_id,
COUNT(DISTINCT parent_id) AS total_requests
FROM submissions
WHERE parent_id IS NULL
GROUP BY sub_id)


SELECT 
a.sub_id AS post_id, 
COALESCE(b.comments_count, 0) AS number_of_comments
FROM cte2 a LEFT JOIN cte b ON a.sub_id = b.parent_id
ORDER BY post_id ASC
"""

make_select(query)

,post_id,number_of_comments
0,1,3
1,2,2
2,12,0


In [143]:
sessions_data = {
    "session_id": [10, 14, 17, 12, 9, 13, 18, 2, 8],
    "customer_id": [3, 14, 2, 16, 2, 16, 3, 3, 16],
    "start_time": [31, 151, 120, 658, 376, 511, 680, 282, 178],
    "end_time": [210, 178, 252, 676, 482, 534, 777, 506, 278]
}

playback = pd.DataFrame(sessions_data)


ads_data = {
    "ad_id": [18, 5, 8, 4, 6, 16, 17, 10, 3],
    "customer_id": [3, 3, 2, 3, 16, 3, 2, 3, 16],
    "timestamp": [156, 308, 183, 119, 526, 711, 416, 417, 516]
}



ads = pd.DataFrame(ads_data)

query = """  

WITH cte AS (SELECT 
p.session_id, p.customer_id, 
p.start_time,
p.end_time ,
COALESCE(a.ad_id,0) AS ad_id, COALESCE(a.timestamp,0) AS timestamp
FROM playback p FULL OUTER JOIN ads a ON p.customer_id = a.customer_id),


cte2 AS (SELECT DISTINCT session_id FROM cte WHERE timestamp  BETWEEN start_time AND end_time),

cte3 AS (SELECT DISTINCT session_id FROM cte WHERE timestamp NOT BETWEEN start_time AND end_time)


SELECT DISTINCT b.session_id FROM cte2 a FULL OUTER JOIN cte3 b ON a.session_id = b.session_id WHERE a.session_id IS NULL

"""


make_select(query)

,session_id
0,14
1,12
2,8


In [410]:
data = [[1, 7, 1], [2, 7, 2], [3, 11, 1], [4, 11, 7], [5, 11, 7], [6, 11, 3]]
rides = pd.DataFrame(data, columns=['ride_id', 'driver_id', 'passenger_id']).astype({'ride_id':'Int64', 'driver_id':'Int64', 'passenger_id':'Int64'})

query = """  

WITH cte AS (SELECT 
passenger_id,
COUNT(passenger_id) AS been
FROM rides
GROUP BY passenger_id)


SELECT 
DISTINCT r.driver_id, COALESCE(c.been,0) AS cnt
FROM rides r LEFT JOIN cte c ON r.driver_id = c.passenger_id

"""

make_select(query)

,driver_id,cnt
0,7,2
1,11,0


In [ ]:
data = [[9, 'Rum and Coke'], [6, None], [7, None], [3, 'St Germain Spritz'], [1, 'Orange Margarita'], [2, None]]
coffee_shop = pd.DataFrame(data, columns=['id', 'drink']).astype({'id':'Int64', 'drink':'object'})

def change_null_values(coffee_shop: pd.DataFrame) -> pd.DataFrame:
    coffee_shop["drink"] = coffee_shop["drink"].fillna(method="ffill") 
    return coffee_shop

change_null_values(coffee_shop)

In [ ]:
data = {
    'student_id': [8, 13, 3, 9, 2, 12, 4, 6, 11, 7, 14, 10, 1, 5],
    'department_id': [4, 3, 1, 3, 3, 3, 2, 4, 2, 2, 3, 1, 4, 1],
    'mark': [860, 40, 130, 40, 20, 390, 270, 670, 150, 130, 790, 900, 910, 610]
}

students= pd.DataFrame(data)

query = """  

WITH cte AS (SELECT 
*,
RANK() OVER(PARTITION BY department_id ORDER BY mark DESC) AS ranking,
COUNT(student_id) OVER(PARTITION BY department_id ) AS num_students
FROM students)


SELECT 
*,
COALESCE(ROUND((ranking - 1) * 1.0 * 100 / NULLIF(num_students - 1, 0), 0), 0) AS percentage
FROM cte 

"""

make_select(query)

,student_id,department_id,mark,ranking,num_students,percentage
0,2,3,20,5,5,100.0


In [255]:
drivers_data = {
    'driver_id': [1, 2, 3, 4],
    'name': ['Charlie Brown', 'David Brown', 'Charlie Davis', 'Eve Johnson'],
    'age': [28, 46, 38, 42],
    'experience': [27, 3, 13, 27],
    'accidents': [3, 5, 2, 1]
}
drivers = pd.DataFrame(drivers_data)

vehicles_data = {
    'vehicle_id': [100, 101, 102, 103],
    'driver_id': [4, 4, 1, 4],
    'model': ['Convertible', 'Sedan', 'Sedan', 'SUV'],
    'fuel_type': ['Diesel', 'Diesel', 'Diesel', 'Electric'],
    'mileage': [30272, 34099, 45905, 16756]
}
vehicles = pd.DataFrame(vehicles_data)

trips_data = {
    'trip_id': [201, 202, 203, 204, 205, 206],
    'vehicle_id': [103, 101, 101, 101, 103, 100],
    'distance': [157, 139, 187, 36, 36, 172],
    'duration': [61, 22, 85, 103, 49, 118],
    'rating': [3, 2, 4, 5, 3, 5]
}
trips = pd.DataFrame(trips_data)



query = """   

WITH cte AS (SELECT 
d.driver_id, d.name, d.age, d.experience, d.accidents, 
v.vehicle_id,v.model, v.fuel_type, v.mileage, t.trip_id, t.distance,t.rating
FROM drivers d 
INNER JOIN vehicles v ON v.driver_id = d.driver_id
INNER JOIN trips t ON t.vehicle_id = v.vehicle_id),

report AS (SELECT 
driver_id, fuel_type,
SUM(rating) * 1.0 /
COUNT(driver_id) AS avg_rating,
SUM(distance) AS total_distance,
accidents
FROM cte
GROUP BY driver_id, fuel_type)


SELECT fuel_type, driver_id,avg_rating AS rating , total_distance AS distance 
FROM (SELECT 
*,
DENSE_RANK() OVER(PARTITION BY fuel_type ORDER BY avg_rating DESC, total_distance DESC, accidents) AS rnk
FROM report) WHERE rnk = 1

"""

make_select(query)

,fuel_type,driver_id,rating,distance
0,Diesel,4,4.0,534
1,Electric,4,3.0,193


In [268]:
data = [[1, 2, 4], [2, 1, 5], [2, 4, 5]]
flights = pd.DataFrame(data, columns=['departure_airport', 'arrival_airport', 'flights_count']).astype({'departure_airport':'Int64', 'arrival_airport':'Int64', 'flights_count':'Int64'})



query = """  

WITH dep AS (SELECT 
departure_airport,
SUM(flights_count) AS total
FROM flights
GROUP BY departure_airport),

arr AS (SELECT 
arrival_airport,
SUM(flights_count) AS total
FROM flights
GROUP BY arrival_airport),


cte AS (SELECT 
departure_airport AS airport_id,
SUM(total) AS total
FROM (SELECT * FROM dep
UNION ALL
SELECT * FROM arr)
GROUP BY departure_airport)


SELECT airport_id
FROM cte 
WHERE total = (SELECT MAX(total) FROM cte)

"""

make_select(query)

,airport_id
0,2


In [294]:
data = [[1, 'Moustafa', 100], [2, 'Jonathan', 200], [3, 'Winston', 10000], [4, 'Luis', 800]]
users = pd.DataFrame(data, columns=['user_id', 'user_name', 'credit']).astype({'user_id':'Int64', 'user_name':'object', 'credit':'Int64'})
data = [[1, 1, 3, 400, '2020-08-01'], [2, 3, 2, 500, '2020-08-02'], [3, 2, 1, 200, '2020-08-03']]
transactions = pd.DataFrame(data, columns=['trans_id', 'paid_by', 'paid_to', 'amount', 'transacted_on']).astype({'trans_id':'Int64', 'paid_by':'Int64', 'paid_to':'Int64', 'amount':'Int64', 'transacted_on':'datetime64[ns]'})


query = """  

WITH cte AS (SELECT 
paid_by,
SUM(amount) AS total_gets
FROM transactions
GROUP BY paid_by),

cte2 AS (SELECT 
paid_to,
SUM(amount) AS total_spent
FROM transactions
GROUP BY paid_to),


cte3 AS (SELECT 
u.user_id, u.user_name, u.credit - COALESCE(c.total_gets,0) AS total
FROM users u
LEFT JOIN cte c ON u.user_id = c.paid_by)

SELECT 
c.user_id, c.user_name, c.total + COALESCE(b.total_spent, 0) AS credit,
CASE WHEN c.total + COALESCE(b.total_spent, 0) < 0 THEN 'Yes' ELSE 'No' END AS credit_limit_breached
FROM cte3 c 
LEFT JOIN cte2 b ON c.user_id = b.paid_to

"""

make_select(query)

,user_id,user_name,credit,credit_limit_breached
0,1,Moustafa,-100,Yes
1,2,Jonathan,500,No
2,3,Winston,9900,No
3,4,Luis,800,No


In [352]:

data = {
    "account_id": [1, 1, 1, 2, 3, 3, 4, 4],
    "ip_address": [1, 1, 6, 7, 9, 13, 10, 11],
    "login": [
        "2021-02-01 09:00:00", "2021-02-01 10:00:00", "2021-02-01 12:00:00",
        "2021-02-02 20:30:00", "2021-02-01 16:00:00", "2021-02-01 17:00:00",
        "2021-02-01 16:00:00", "2021-02-01 17:00:00"
    ],
    "logout": [
        "2021-02-01 15:00:00", "2021-02-01 11:00:00", "2021-02-01 13:00:00",
        "2021-02-02 22:00:00", "2021-02-01 16:59:59", "2021-02-01 17:59:59",
        "2021-02-01 17:00:00", "2021-02-01 17:59:59"
    ]
}

log_info = pd.DataFrame(data)
log_info["login"] = pd.to_datetime(log_info["login"])
log_info["logout"] = pd.to_datetime(log_info["logout"])

# data = [[1, 1, '2021-02-01 09:00:00', '2021-02-01 09:30:00'], [1, 2, '2021-02-01 08:00:00', '2021-02-01 11:30:00'], [2, 6, '2021-02-01 20:30:00', '2021-02-01 22:00:00'], [2, 7, '2021-02-02 20:30:00', '2021-02-02 22:00:00'], [3, 9, '2021-02-01 16:00:00', '2021-02-01 16:59:59'], [3, 13, '2021-02-01 17:00:00', '2021-02-01 17:59:59'], [4, 10, '2021-02-01 16:00:00', '2021-02-01 17:00:00'], [4, 11, '2021-02-01 17:00:00', '2021-02-01 17:59:59']]
# log_info = pd.DataFrame(data, columns=['account_id', 'ip_address', 'login', 'logout']).astype({'account_id':'Int64', 'ip_address':'Int64', 'login':'datetime64[ns]', 'logout':'datetime64[ns]'})



# SELECT DISTINCT account_id FROM cte2 WHERE df BETWEEN login AND logout AND ip_address != df_ip

query = """  

WITH cte AS (SELECT 
*
FROM log_info
ORDER BY account_id, login
),

cte2 AS (SELECT 
*,
LEAD(login) OVER(PARTITION BY account_id ORDER BY login) AS df,
LEAD(ip_address) OVER(PARTITION BY account_id ORDER BY login) AS df_ip
FROM cte )

SELECT DISTINCT a.account_id FROM log_info a JOIN log_info b ON a.account_id = b.account_id
WHERE a.ip_address != b.ip_address AND a.login BETWEEN b.login AND b.logout

"""

make_select(query)

,account_id
0,1
1,4


In [395]:
data = [[1, 65.56, '2023-11-18 13:49:42'], [1, 96.0, '2023-11-30 02:47:26'], [1, 7.44, '2023-11-02 12:15:23'], [1, 49.78, '2023-11-12 00:13:46'], [2, 40.89, '2023-11-21 04:39:15'], [2, 100.44, '2023-11-20 07:39:34'], [3, 37.33, '2023-11-03 06:22:02'], [3, 13.89, '2023-11-11 16:00:14'], [3, 7.0, '2023-11-29 22:32:36']]
transactions = pd.DataFrame(data, columns=['user_id', 'spend', 'transaction_date']).astype({
    'user_id': 'Int64', 
    'spend': 'float',  
    'transaction_date': 'datetime64[ns]'  
})


# SELECT 
#     user_id, total3, transaction_date
# FROM cte3
# WHERE transaction_count IN (1, 2, 3) AND total < total3 
# GROUP BY user_id
# HAVING COUNT(transaction_count) = 3




query = """  

WITH cte AS (SELECT * 
FROM transactions 
ORDER BY user_id,transaction_date
),

cte2 AS (SELECT 
*,
ROW_NUMBER() OVER(PARTITION BY user_id) AS transaction_count
FROM cte),

cte3 AS (
SELECT 
*,
SUM(CASE WHEN transaction_count = 1 THEN spend END) OVER(PARTITION BY user_id) AS total1,
SUM(CASE WHEN transaction_count = 2 THEN spend END) OVER(PARTITION BY user_id) AS total2,
SUM(CASE WHEN transaction_count = 3 THEN spend END) OVER(PARTITION BY user_id) AS total3
FROM cte2 
WHERE transaction_count <= 3
),

final AS (SELECT * FROM cte3 WHERE total3 IS NOT NULL AND total1 < total2  AND total2 < total3)


SELECT user_id, total3 AS third_transaction_spend , MAX(transaction_date) AS third_transaction_date
FROM final 
GROUP BY user_id, total3

"""


make_select(query)

,user_id,third_transaction_spend,third_transaction_date
0,1,65.56,2023-11-18 13:49:42.000000


In [ ]:
orders = pd.DataFrame({
    "minute": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    "order_count": [0, 2, 4, 6, 1, 4, 1, 2, 4, 1, 4, 6]
})


query = """  

SELECT 
*,
ROW_NUMBER(CASE WHEN TH)
FROM orders

"""

make_select(query)

,minute,order_count
0,1,0
1,2,2
2,3,4
3,4,6
4,5,1
5,6,4
6,7,1
7,8,2
8,9,4
9,10,1


In [422]:
data = [['Alice', 'Bob'], ['Bob', 'Cena'], ['Bob', 'Donald'], ['Donald', 'Edward']]
follow = pd.DataFrame(data, columns=['followee', 'follower']).astype({'followee':'object', 'follower':'object'})


query = """   

WITH cte AS (SELECT  
followee,
COUNT(follower) AS total_followers
FROM follow
GROUP BY followee
ORDER BY total_followers DESC),


cte2 AS (SELECT  
follower,
COUNT(followee) AS total_followings
FROM follow
GROUP BY follower
ORDER BY total_followings DESC)


SELECT
DISTINCT b.follower,
c.total_followers AS num
FROM cte c
FULL OUTER JOIN cte2 b ON c.followee = b.follower 
WHERE b.total_followings >= 1 AND c.total_followers >= 1
"""

make_select(query)

,follower,num
0,Bob,2
1,Donald,1


In [430]:
data = [[1, 'LCPHONE', '2000-01-16'], [2, 'LCPhone', '2000-01-17'], [3, 'LcPhOnE', '2000-02-18'], [4, 'LCKeyCHAiN', '2000-02-19'], [5, 'LCKeyChain', '2000-02-28'], [6, 'Matryoshka', '2000-03-31']]
sales = pd.DataFrame(data, columns=['sale_id', 'product_name', 'sale_date']).astype({'sale_id':'Int64', 'product_name':'object', 'sale_date':'datetime64[ns]'})


query = """   

SELECT 
TRIM(LOWER(product_name)) AS product_name,
strftime('%Y-%m', sale_date) AS sale_date,
COUNT(sale_id) AS total
FROM sales
GROUP BY TRIM(LOWER(product_name)),strftime('%Y-%m', sale_date)
ORDER BY product_name,sale_date

"""

make_select(query)

,product_name,sale_date,total
0,lckeychain,2000-02,2
1,lcphone,2000-01,2
2,lcphone,2000-02,1
3,matryoshka,2000-03,1


In [21]:
data = [[1, 'Manchester City', 10, 6, 2, 2], [2, 'Liverpool', 10, 6, 2, 2], [3, 'Chelsea', 10, 5, 3, 2], [4, 'Arsenal', 10, 4, 4, 2], [5, 'Tottenham', 10, 3, 5, 2]]
team_stats = pd.DataFrame(data, columns=["team_id", "team_name", "matches_played", "wins", "draws", "losses"]).astype({"team_id": "int", "team_name": "string", "matches_played": "int", "wins": "int", "draws": "int", "losses": "int"})



query = """   




SELECT 
team_id ,team_name,  (wins * 3) + (draws * 1 ) + (losses * 0) AS points,    
RANK() OVER(ORDER BY (wins * 3) + (draws * 1 ) + (losses * 0) DESC) AS ranking 
FROM team_stats

"""

make_select(query)

,team_id,team_name,points,ranking
0,1,Manchester City,20,1
1,2,Liverpool,20,1
2,3,Chelsea,18,3
3,4,Arsenal,16,4
4,5,Tottenham,14,5


In [32]:
data = [[1, 2018, 100], [7, 2020, 30], [13, 2019, 40], [1, 2019, 113], [2, 2008, 121], [3, 2009, 21], [11, 2020, 99], [7, 2019, 0]]
npv = pd.DataFrame(data, columns=['id', 'year', 'npv']).astype({'id':'Int64', 'year':'Int64', 'npv':'Int64'})
data = [[1, 2019], [2, 2008], [3, 2009], [7, 2018], [7, 2019], [7, 2020], [13, 2019]]
queries = pd.DataFrame(data, columns=['id', 'year']).astype({'id':'Int64', 'year':'Int64'})



query = """   

SELECT 
q.id, q.year, COALESCE(n.npv, 0) AS  npv 
FROM queries q LEFT JOIN npv n ON q.id = n.id AND q.year = n.year


"""


make_select(query)

,id,year,npv
0,1,2019,113
1,2,2008,121
2,3,2009,21
3,7,2018,0
4,7,2019,0
5,7,2020,30
6,13,2019,40


In [36]:
data = [['Leetcode FC'], ['Ahly SC'], ['Real Madrid']]
teams = pd.DataFrame(data, columns=['team_name']).astype({'team_name':'object'})


query = """  

SELECT 
a.team_name AS home_team,
b.team_name AS away_team
FROM teams a JOIN teams b ON a.team_name <> b.team_name

"""

make_select(query)

,home_team,away_team
0,Leetcode FC,Ahly SC
1,Leetcode FC,Real Madrid
2,Ahly SC,Leetcode FC
3,Ahly SC,Real Madrid
4,Real Madrid,Leetcode FC
5,Real Madrid,Ahly SC


In [48]:
data = [[1, 'S8', 1000], [2, 'G4', 800], [3, 'iPhone', 1400]]
product = pd.DataFrame(data, columns=['product_id', 'product_name', 'unit_price']).astype({'product_id':'Int64', 'product_name':'object', 'unit_price':'Int64'})
data = [[1, 1, 1, '2019-01-21', 2, 2000], [1, 2, 2, '2019-02-17', 1, 800], [2, 1, 3, '2019-06-02', 1, 800], [3, 3, 3, '2019-05-13', 2, 2800]]
sales = pd.DataFrame(data, columns=['seller_id', 'product_id', 'buyer_id', 'sale_date', 'quantity', 'price']).astype({'seller_id':'Int64', 'product_id':'Int64', 'buyer_id':'Int64', 'sale_date':'datetime64[ns]', 'quantity':'Int64', 'price':'Int64'})



query = """   

WITH cte AS (SELECT 
s.buyer_id, s.product_id, s.sale_date, p.product_name
FROM sales s FULL OUTER JOIN product p ON s.product_id = p.product_id),


iphone AS (SELECT * FROM cte WHERE product_name = 'iPhone'),

s_phone AS (SELECT * FROM cte WHERE product_name = 'S8')

SELECT
DISTINCT s.buyer_id
FROM s_phone s 
LEFT JOIN iphone i ON i.buyer_id = s.buyer_id
WHERE i.product_name IS NULL

"""

make_select(query)

,buyer_id
0,1


In [57]:
data = [[101, 'Alice'], [102, 'Bob'], [103, 'Charlie']]
customer = pd.DataFrame(data, columns=['customer_id', 'customer_name']).astype({'customer_id':'Int64', 'customer_name':'object'})
data = [[1, '2020-03-01', 1500, 101, 1], [2, '2020-05-25', 2400, 102, 2], [3, '2019-05-25', 800, 101, 3], [4, '2020-09-13', 1000, 103, 2], [5, '2019-02-11', 700, 101, 2]]
orders = pd.DataFrame(data, columns=['order_id', 'sale_date', 'order_cost', 'customer_id', 'seller_id']).astype({'order_id':'Int64', 'sale_date':'datetime64[ns]', 'order_cost':'Int64', 'customer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Daniel'], [2, 'Elizabeth'], [3, 'Frank']]
seller = pd.DataFrame(data, columns=['seller_id', 'seller_name']).astype({'seller_id':'Int64', 'seller_name':'object'})


query = """  

SELECT 
s.seller_name
FROM seller s LEFT JOIN orders o  ON o.seller_id = s.seller_id
WHERE s.seller_id NOT IN (SELECT seller_id FROM orders WHERE strftime('%Y', sale_date) = '2020')


 

"""

make_select(query)

,seller_name
0,Frank


In [82]:
data = [[11, '2023-11-07', 1126], [15, '2023-11-30', 7473], [17, '2023-11-14', 2414], [12, '2023-11-24', 9692], [8, '2023-11-03', 5117], [1, '2023-11-16', 5241], [10, '2023-11-12', 8266], [13, '2023-11-24', 12000]]
purchases = pd.DataFrame(data, columns=['user_id', 'purchase_date', 'amount_spend']).astype({'user_id':'Int64', 'purchase_date':'datetime64[ns]', 'amount_spend':'Int64'})



query = """  

WITH cte AS (SELECT 
*,
strftime('%w', purchase_date) AS week_days,
(CAST(strftime('%d', purchase_date) AS INTEGER) - 1) / 7 + 1 AS week_of_month
FROM purchases
WHERE strftime('%Y-%m', purchase_date) = '2023-11'),


cte2 AS (SELECT 
week_of_month, 
purchase_date,
SUM(amount_spend) AS total_amount
FROM cte
WHERE week_days = '5' 
GROUP BY week_of_month, purchase_date),


report AS (SELECT 1 AS week_of_month, '2023-11-03' AS purchase_date
UNION ALL
SELECT 2, '2023-11-10'
UNION ALL
SELECT 3, '2023-11-17'
UNION ALL
SELECT 4, '2023-11-24')


SELECT 
r.week_of_month, 
r.purchase_date,
COALESCE(c.total_amount, 0) AS total_amount
FROM cte2 c 
RIGHT JOIN report r ON c.week_of_month = r.week_of_month 
ORDER BY r.week_of_month ASC

"""

make_select(query)

,week_of_month,purchase_date,total_amount
0,1,2023-11-03,5117
1,2,2023-11-10,0
2,3,2023-11-17,0
3,4,2023-11-24,21692


In [175]:
data = [[3, 'Jonathan', '051-1234567'], [12, 'Elvis', '051-7654321'], [1, 'Moncef', '212-1234567'], [2, 'Maroua', '212-6523651'], [7, 'Meir', '972-1234567'], [9, 'Rachel', '972-0011100']]
person = pd.DataFrame(data, columns=['id', 'name', 'phone_number']).astype({'id':'Int64', 'name':'object', 'phone_number':'object'})
data = [['Peru', '051'], ['Israel', '972'], ['Morocco', '212'], ['Germany', '049'], ['Ethiopia', '251']]
country = pd.DataFrame(data, columns=['name', 'country_code']).astype({'name':'object', 'country_code':'object'})
data = [[1, 9, 33], [2, 9, 4], [1, 2, 59], [3, 12, 102], [3, 12, 330], [12, 3, 5], [7, 9, 13], [7, 1, 3], [9, 7, 1], [1, 7, 7]]
calls = pd.DataFrame(data, columns=['caller_id', 'callee_id', 'duration']).astype({'caller_id':'Int64', 'callee_id':'Int64', 'duration':'Int64'})


# SELECT 
# b.country_name, 
# AVG(a.total) + AVG(b.total) AS avg
# FROM callers a
# FULL OUTER JOIN callees b ON a.caller_id = b.callee_id
# GROUP BY b.country_name

# SELECT AVG(duration) FROM calls

query = """   

WITH cte AS (SELECT 
p.id, p.name, p.phone_number, c.name AS country_name,c.country_code
FROM person p INNER JOIN country c ON SUBSTR(p.phone_number, 1, 3) = c.country_code),

final AS (SELECT caller_id, duration FROM calls
UNION ALL 
SELECT callee_id, duration FROM calls
)

SELECT 
c.country_name
FROM final f FULL OUTER JOIN cte c ON f.caller_id = c.id
GROUP BY c.country_name
HAVING AVG(f.duration) > (SELECT AVG(duration) FROM calls)

"""


make_select(query)

,country_name
0,Peru


In [209]:
data = [[1, '2020-01-01'], [2, '2020-01-02'], [12, '2020-01-01'], [19, '2020-01-03'], [1, '2020-01-02'], [2, '2020-01-03'], [1, '2020-01-04'], [7, '2020-01-11'], [9, '2020-01-25'], [8, '2020-01-28']]
visits = pd.DataFrame(data, columns=['user_id', 'visit_date']).astype({'user_id':'Int64', 'visit_date':'datetime64[ns]'})
data = [[1, '2020-01-02', 120], [2, '2020-01-03', 22], [7, '2020-01-11', 232], [1, '2020-01-04', 7], [9, '2020-01-25', 33], [9, '2020-01-25', 66], [8, '2020-01-28', 1], [9, '2020-01-25', 99]]
transactions = pd.DataFrame(data, columns=['user_id', 'transaction_date', 'amount']).astype({'user_id':'Int64', 'transaction_date':'datetime64[ns]', 'amount':'Int64'})


query = """

WITH ctr_made AS (SELECT 
t.user_id, 
COUNT(visit_date) AS transaction_made
FROM visits v 
FULL OUTER JOIN transactions t ON t.user_id = v.user_id
WHERE v.visit_date = t.transaction_date
GROUP BY t.user_id
ORDER BY transaction_made
),


not_made AS (SELECT * FROM (
SELECT 
t.user_id
FROM visits v 
FULL OUTER JOIN transactions t ON t.user_id != v.user_id
WHERE v.visit_date = t.transaction_date
GROUP BY t.user_id)
UNION ALL
SELECT * FROM (SELECT v.user_id 
FROM visits v LEFT JOIN transactions t ON v.user_id = t.user_id
WHERE t.user_id IS NULL))


SELECT 
    transaction_made AS transactions_count, 
    COUNT(user_id) AS visits_count
FROM (
    SELECT user_id, transaction_made FROM ctr_made
    UNION ALL
    SELECT user_id, 0 AS transaction_made FROM not_made
) AS all_visits
GROUP BY transaction_made
ORDER BY transactions_count

"""

make_select(query)

,transactions_count,visits_count
0,0,4
1,1,3
2,2,1
3,3,1


In [231]:
data = [[1, 'Alice', 'alice@leetcode.com'], [2, 'Bob', 'bob@leetcode.com'], [13, 'John', 'john@leetcode.com'], [6, 'Alex', 'alex@leetcode.com']]
customers = pd.DataFrame(data, columns=['customer_id', 'customer_name', 'email']).astype({'customer_id':'Int64', 'customer_name':'object', 'email':'object'})
data = [[1, 'Bob', 'bob@leetcode.com'], [1, 'John', 'john@leetcode.com'], [1, 'Jal', 'jal@leetcode.com'], [2, 'Omar', 'omar@leetcode.com'], [2, 'Meir', 'meir@leetcode.com'], [6, 'Alice', 'alice@leetcode.com']]
contacts = pd.DataFrame(data, columns=['user_id', 'contact_name', 'contact_email']).astype({'user_id':'Int64', 'contact_name':'object', 'contact_email':'object'})
data = [[77, 100, 1], [88, 200, 1], [99, 300, 2], [66, 400, 2], [55, 500, 13], [44, 60, 6]]
invoices = pd.DataFrame(data, columns=['invoice_id', 'price', 'user_id']).astype({'invoice_id':'Int64', 'price':'Int64', 'user_id':'Int64'})


query = """   

WITH cte AS (SELECT 
i.invoice_id, c.customer_name, i.price, c.customer_id,c.email
FROM invoices i INNER JOIN customers c ON i.user_id = c.customer_id
ORDER BY i.invoice_id),

cte2 AS (SELECT 
user_id, 
COUNT(contact_name) as num
FROM contacts 
GROUP BY user_id),

cte3 AS (SELECT c.user_id, COUNT(*) AS trusted_contacts_count
FROM contacts c
INNER JOIN customers cu 
    ON c.contact_email = cu.email
GROUP BY c.user_id)

SELECT 
a.invoice_id, a.customer_name, a.price, COALESCE(b.num, 0) AS contacts_cnt,
COALESCE(c.trusted_contacts_count,0) AS trusted_contacts_count
FROM cte a 
FULL OUTER JOIN cte2 b ON a.customer_id = b.user_id
FULL OUTER JOIN cte3 c ON a.customer_id = c.user_id
ORDER BY a.invoice_id



"""

make_select(query)

,invoice_id,customer_name,price,contacts_cnt,trusted_contacts_count
0,44,Alex,60,1,1
1,55,John,500,0,0
2,66,Bob,400,2,0
3,77,Alice,100,3,2
4,88,Alice,200,3,2
5,99,Bob,300,2,0


In [252]:
data = [[11, '2023-11-03', 1126], [15, '2023-11-10', 7473], [17, '2023-11-17', 2414], [12, '2023-11-24', 9692], [8, '2023-11-24', 5117], [1, '2023-11-24', 5241], [10, '2023-11-22', 8266], [13, '2023-11-21', 12000]]
purchases = pd.DataFrame(data, columns=['user_id', 'purchase_date', 'amount_spend']).astype({'user_id':'Int64', 'purchase_date':'datetime64[ns]', 'amount_spend':'Int64'})

data = [[11, 'Premium'], [15, 'VIP'], [17, 'Standard'], [12, 'VIP'], [8, 'Premium'], [1, 'VIP'], [10, 'Standard'], [13, 'Premium']]
users = pd.DataFrame(data, columns=['user_id', 'membership']).astype({
    'user_id': 'Int64',
    'membership': pd.CategoricalDtype(categories=['Standard', 'Premium', 'VIP'])
})


query = """  

WITH cte AS (SELECT 
u.user_id, u.membership, p.purchase_date,p.amount_spend,
strftime('%w', p.purchase_date) AS week_days,
(CAST(strftime('%d', p.purchase_date) AS INTEGER) - 1) / 7 + 1 AS week_of_month
FROM users u INNER JOIN purchases p ON u.user_id = p.user_id
WHERE strftime('%Y-%m', p.purchase_date) = '2023-11'),

cte2 AS (SELECT 
membership,
week_of_month, 
SUM(amount_spend) AS total_amount
FROM cte
WHERE membership IN ('Premium','VIP') AND week_days = '5'
GROUP BY membership,week_of_month ),

cte3 AS (SELECT 1 AS week_of_month, 'Premium' AS membership
UNION ALL
SELECT 1, 'VIP'
UNION ALL
SELECT 2, 'Premium'
UNION ALL
SELECT 2, 'VIP'
UNION ALL
SELECT 3, 'Premium'
UNION ALL
SELECT 3, 'VIP'
UNION ALL
SELECT 4, 'Premium'
UNION ALL
SELECT 4, 'VIP')


SELECT
a.week_of_month, 
a.membership, 
COALESCE(b.total_amount, 0) AS total_amount 
FROM cte3 a LEFT JOIN cte2 b ON a.week_of_month = b.week_of_month AND a.membership = b.membership
ORDER BY a.week_of_month,a.membership
"""

make_select(query)

,week_of_month,membership,total_amount
0,1,Premium,1126
1,1,VIP,0
2,2,Premium,0
3,2,VIP,7473
4,3,Premium,0
5,3,VIP,0
6,4,Premium,5117
7,4,VIP,14933


In [291]:
data = {
    "season_id": [2021, 2021, 2021, 2021, 2021, 2022, 2022, 2022, 2022, 2022],
    "team_id": [1, 2, 3, 4, 5, 1, 2, 3, 4, 5],
    "team_name": [
        "Manchester City", "Liverpool", "Chelsea", "Tottenham", "Arsenal",
        "Manchester City", "Arsenal", "Manchester United", "Newcastle", "Liverpool"
    ],
    "matches_played": [38] * 10,
    "wins": [29, 28, 21, 22, 22, 28, 26, 23, 19, 19],
    "draws": [6, 8, 11, 5, 3, 5, 6, 6, 14, 10],
    "losses": [3, 2, 6, 11, 13, 5, 6, 9, 5, 9],
    "goals_for": [99, 94, 76, 69, 61, 94, 88, 58, 68, 75],
    "goals_against": [26, 26, 33, 40, 48, 33, 43, 43, 33, 47]
}

season_stats = pd.DataFrame(data)

query = """  

WITH cte AS (SELECT 
season_id, team_id, team_name,
wins * 3 + draws * 1 + losses*0 AS points,
goals_for - goals_against AS goal_diff
FROM season_stats
)

SELECT
*,
RANK() OVER(PARTITION BY season_id ORDER BY points DESC, goal_diff DESC, team_name ASC) AS ranking
FROM cte
ORDER BY season_id ASC, ranking ASC, team_name ASC
"""

make_select(query)

,season_id,team_id,team_name,points,goal_diff,ranking
0,2021,1,Manchester City,93,73,1
1,2021,2,Liverpool,92,68,2
2,2021,3,Chelsea,74,43,3
3,2021,4,Tottenham,71,29,4
4,2021,5,Arsenal,69,13,5
5,2022,1,Manchester City,89,61,1
6,2022,2,Arsenal,84,45,2
7,2022,3,Manchester United,75,15,3
8,2022,4,Newcastle,71,35,4
9,2022,5,Liverpool,67,28,5


In [347]:
students = pd.DataFrame({
    "student_id": [1, 2, 3, 4, 5],
    "name": ["Grace", "Alice", "David", "Bob", "Henry"],
    "major": ["Economics", "Economics", "Economics", "Economics", "Economics"]
})

# Courses DataFrame
courses = pd.DataFrame({
    "course_id": [1, 2, 3, 4, 5, 6, 7, 8, 9, 10],
    "name": ["Database Systems", "Calculus", "Operating Systems", "Algorithms", "Linear Algebra",
             "Computer Networks", "Data Structures", "Calculus", "Operating Systems", "Database Systems"],
    "credits": [4, 3, 2, 3, 1, 2, 3, 1, 1, 1],
    "major": ["Computer Science", "Computer Science", "Computer Science", "Computer Science", "Computer Science",
              "Economics", "Economics", "Economics", "Economics", "Economics"]
})

# Enrollments DataFrame
enrollments = pd.DataFrame({
    "student_id": [
        1, 1, 1, 1, 1, 1, 1, 1,
        2, 2, 2, 2, 2, 2, 2,
        3, 3, 3, 3, 3, 3, 3,
        4, 4, 4, 4, 4, 4, 4, 4,
        5, 5, 5, 5, 5, 5, 5, 5
    ],
    "course_id": [
        1, 3, 4, 5, 6, 7, 8, 9,
        2, 3, 4, 5, 7, 9, 10,
        1, 3, 4, 5, 7, 8, 9,
        1, 4, 5, 6, 7, 8, 9, 10,
        1, 2, 3, 4, 6, 7, 8, 9
    ],
    "semester": [
        "Spring", "Spring", "Spring", "Fall", "Spring", "Spring", "Fall", "Spring",
        "Spring", "Spring", "Fall", "Spring", "Spring", "Fall", "Fall",
        "Fall", "Spring", "Spring", "Fall", "Fall", "Fall", "Spring",
        "Spring", "Spring", "Fall", "Spring", "Spring", "Spring", "Fall", "Fall",
        "Spring", "Spring", "Spring", "Fall", "Spring", "Fall", "Fall", "Fall"
    ],
    "grade": [
        "A", "A", "B", "B", "A", "B", "B", "B",
        "B", "B", "B", "A", "B", "B", "B",
        "B", "B", "B", "A", "A", "A", "A",
        "B", "A", "B", "A", "A", "A", "A", "A",
        "B", "A", "A", "A", "B", "B", "B", "B"
    ]
})



query = """   


WITH cte AS (SELECT s.student_id, s.name, s.major, c.course_id, c.name AS course_name
FROM students s INNER JOIN courses c ON s.major = c.major),

cte2 AS (SELECT 
c.student_id, c.major, c.course_id, c.course_name, e.grade,
COUNT(c.course_id) OVER(PARTITION BY e.student_id) AS num
FROM cte c 
INNER JOIN enrollments e 
ON c.student_id = e.student_id AND c.course_id = e.course_id),



num_coures AS (SELECT 
major, 
COUNT(DISTINCT name) AS total_coures
FROM courses
GROUP BY major),

final AS (SELECT 
a.student_id, a.course_id, a.grade, a.course_name, a.major, a.num, b.total_coures
FROM cte2 a INNER JOIN num_coures b ON a.major = b.major)

SELECT 
student_id
FROM final
WHERE grade = 'A' 
GROUP BY student_id,total_coures
HAVING COUNT(grade) = total_coures

"""


make_select(query)

,student_id
0,4


In [397]:
friendship = pd.DataFrame({
    "user1_id": [1, 1, 1, 2, 3, 3, 4, 5, 8],
    "user2_id": [3, 5, 6, 3, 5, 9, 6, 9, 9]
})

# User-Page DataFrame
likes= pd.DataFrame({
    "user_id": [6, 8, 9],
    "page_id": [13, 10, 14]
})


# data = [[1, 2], [1, 3], [1, 4], [2, 3], [2, 4], [2, 5], [6, 1]]
# friendship = pd.DataFrame(data, columns=['user1_id', 'user2_id']).astype({'user1_id':'Int64', 'user2_id':'Int64'})
# data = [[1, 88], [2, 23], [3, 24], [4, 56], [5, 11], [6, 33], [2, 77], [3, 77], [6, 88]]
# likes = pd.DataFrame(data, columns=['user_id', 'page_id']).astype({'user_id':'Int64', 'page_id':'Int64'})


# SELECT 
# DISTINCT page_id
# FROM likes l
# FULL OUTER JOIN cte c ON l.user_id = c.friend_id
# WHERE c.friend_id IS NOT NULL AND page_id != (SELECT 
# page_id
# FROM likes 
# WHERE user_id = 1)

query = """

WITH cte AS (SELECT user2_id AS friend_id
FROM friendship
WHERE user1_id = 1
UNION
SELECT user1_id AS friend_id
FROM friendship
WHERE user2_id = 1)



SELECT 
*
FROM likes l
LEFT JOIN cte c ON l.user_id = c.friend_id
WHERE page_id NOT IN (SELECT page_id FROM likes WHERE user_id = 1)


"""

make_select(query)



,user_id,page_id,friend_id
0,6,13,6.0
1,8,10,NaN
2,9,14,NaN


In [ ]:
data = [[2, 1], [1, 3], [4, 1], [1, 5], [1, 6], [2, 6], [7, 2], [8, 3], [3, 9]]
friends = pd.DataFrame(data, columns=['user1', 'user2']).astype({'user1':'Int64', 'user2':'Int64'})



query = """   

WITH cte AS (SELECT * FROM friends
UNION ALL 
SELECT user2, user1 FROM friends),


cte2 AS (SELECT
user1,
COUNT(DISTINCT user2) AS total_friends
FROM cte
GROUP BY user1)

SELECT
user1,
ROUND(total_friends * 1.0 / (SELECT COUNT(DISTINCT user1) AS total FROM cte) * 100,2) AS percentage_popularity
FROM cte2

"""

make_select(query)

,user1,percentage_popularity
0,1,55.56
1,2,33.33
2,3,33.33
3,4,11.11
4,5,11.11
5,6,22.22
6,7,11.11
7,8,11.11
8,9,11.11


In [ ]:
data = [[336, 'hwkiy@test.edu'], [489, 'adcmaf@outlook.com'], [449, 'vrzmwyum@yahoo.com'], [95, 'tof@test.edu'], [320, 'jxhbagkpm@example.org'], [411, 'zxcf@outlook.com']]
emails = pd.DataFrame(data, columns=['id', 'email']).astype({'id':'Int64', 'email':'object'})


def find_unique_email_domains(emails: pd.DataFrame) -> pd.DataFrame:
    valid  = emails["email"].str.endswith(".com")

    emails = emails[valid]

    emails["valid"] = emails["email"].str.split("@")

    emails["second"] = emails["valid"].apply(lambda x: x[1] if len(x) > 1 else None)

    emails = emails.drop(["email", "valid"], axis=1)

    emails = emails.rename(columns={"second":"email_domain"})


    return emails.groupby("email_domain")["email_domain"].count().reset_index(name="count").sort_values(by="email_domain",ascending=True)

find_unique_email_domains(emails)

In [ ]:
import re

data = [[1, 'Winston', 'winston@leetcode.com'], [2, 'Jonathan', 'jonathanisgreat'], [3, 'Annabelle', 'bella-@leetcode.com'], [4, 'Sally', 'sally.come@leetcode.com'], [5, 'Marwan', 'quarz#2020@leetcode.com'], [6, 'David', 'david69@gmail.com'], [7, 'Shapiro', '.shapo@leetcode.com']]
users = pd.DataFrame(data, columns=['user_id', 'name', 'mail']).astype({'user_id':'int64', 'name':'object', 'mail':'object'})


def is_valid_prefix(prefix: str):
    
    if not prefix[0].isalpha():
        return False
    
    for c in prefix:
        if not (c.isalnum() or c  in '_.-'):
            return False
    
    return True

def valid_emails(users: pd.DataFrame) -> pd.DataFrame:
    valid_domains = users["mail"].str.endswith("leetcode.com")

    users = users[valid_domains]

    users["splt"] = users['mail'].str.split("@")

    users["prefix"] = users['splt'].apply(lambda x: x[0] if len(x[0]) > 0 else None)

    valid_prefix = users["prefix"].apply(is_valid_prefix)

    return users[valid_prefix][["user_id", "name", "mail"]]

valid_emails(users)


In [177]:
# data = {
#     'order_id': [5, 7],
#     'product_id': [2, 2],
#     'quantity': [262, 128]
# }

# # Create DataFrame
# orders_details = pd.DataFrame(data)


data = [[1, 1, 12], [1, 2, 10], [1, 3, 15], [2, 1, 8], [2, 4, 4], [2, 5, 6], [3, 3, 5], [3, 4, 18], [4, 5, 2], [4, 6, 8], [5, 7, 9], [5, 8, 9], [3, 9, 20], [2, 9, 4]]
orders_details = pd.DataFrame(data, columns=['order_id', 'product_id', 'quantity']).astype({'order_id':'Int64', 'product_id':'Int64', 'quantity':'Int64'})



query = """   


WITH cte AS (SELECT 
order_id, 
ROUND(SUM(quantity) * 1.0 /COUNT(product_id), 2) AS avg_quantity,
MAX(quantity) AS max_quantity
FROM orders_details
GROUP BY order_id)

SELECT 
order_id
FROM cte
WHERE max_quantity > (SELECT MAX(avg_quantity) FROM cte)


"""

make_select(query)

,order_id
0,1
1,3


In [190]:
data = {
    'minute': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12],
    'order_count': [0, 2, 4, 6, 1, 4, 1, 2, 4, 1, 4, 6]
}

orders = pd.DataFrame(data)


query = """   

WITH cte AS (SELECT
*,
CEIL(minute/6) AS interval_no
FROM orders)

SELECT 
interval_no,
SUM(order_count)
FROM cte
GROUP BY interval_no
ORDER BY interval_no

"""

make_select(query)

,interval_no,SUM(order_count)
0,0,13
1,1,16
2,2,6


In [ ]:
data = [[1, 'Winston'], [7, 'Jonathan']]
accounts = pd.DataFrame(data, columns=['id', 'name']).astype({'id':'Int64', 'name':'object'})
data = [[7, '2020-05-30'], [1, '2020-05-30'], [7, '2020-05-31'], [7, '2020-06-01'], [7, '2020-06-02'], [7, '2020-06-02'], [7, '2020-06-03'], [1, '2020-06-07'], [7, '2020-06-10']]
logins = pd.DataFrame(data, columns=['id', 'login_date']).astype({'id':'Int64', 'login_date':'datetime64[ns]'})


query = """   

WITH cte AS (SELECT DISTINCT l.id, a.name, l.login_date 
FROM logins l INNER JOIN accounts a ON l.id = a.id ORDER BY l.id, l.login_date),


cte2 AS (SELECT 
id, name,login_date,
LAG(login_date) OVER(PARTITION BY id ORDER BY login_date) AS prev_day
FROM cte),

cte3 AS (SELECT 
    id, 
    name,
    login_date,
    CASE 
      WHEN julianday(login_date) - julianday(prev_day) = 1 THEN 0 
      ELSE 1 
    END AS gap
  FROM cte2),


cte4 AS (SELECT 
*,
SUM(gap) OVER(PARTITION BY id ORDER BY login_date) AS stk
FROM cte3)


SELECT
DISTINCT id, name
FROM cte4
GROUP BY id,name
HAVING COUNT(stk) >= 5

"""

make_select(query)

,id,name,stk
0,7,Jonathan,1


In [339]:
data = [[3, 21000], [4, 10400]]
accounts = pd.DataFrame(data, columns=['account_id', 'max_income']).astype({'account_id':'Int64', 'max_income':'Int64'})
data = [[2, 3, 'Creditor', 107100, '2021-06-02 11:38:14'], [4, 4, 'Creditor', 10400, '2021-06-20 12:39:18'], [11, 4, 'Debtor', 58800, '2021-07-23 12:41:55'], [1, 4, 'Creditor', 49300, '2021-05-03 16:11:04'], [15, 3, 'Debtor', 75500, '2021-05-23 14:40:20'], [10, 3, 'Creditor', 102100, '2021-06-15 10:37:16'], [14, 4, 'Creditor', 56300, '2021-07-21 12:12:25'], [19, 4, 'Debtor', 101100, '2021-05-09 15:21:49'], [8, 3, 'Creditor', 64900, '2021-07-26 15:09:56'], [7, 3, 'Creditor', 90900, '2021-06-14 11:23:07']]
transactions = pd.DataFrame(data, columns=['transaction_id', 'account_id', 'type', 'amount', 'day']).astype({'transaction_id':'Int64', 'account_id':'Int64', 'type':'object', 'amount':'Int64', 'day':'datetime64[ns]'})



query = """  

WITH cte AS (SELECT 
account_id,
strftime('%Y-%m', day) AS year_month,
SUM(CASE WHEN type = 'Creditor' THEN amount END) AS total_amount
FROM transactions
GROUP BY strftime('%Y-%m', day), account_id
HAVING SUM(CASE WHEN type = 'Creditor' THEN amount END) > 0
ORDER BY account_id, year_month),


cte2 AS (SELECT c.account_id, c.year_month, 
LAG(c.year_month) OVER(PARTITION BY c.account_id ORDER BY c.year_month) AS prev_month,
c.total_amount, a.max_income 
FROM cte c INNER JOIN accounts a ON c.account_id = a.account_id
WHERE c.total_amount > a.max_income),


cte3 AS (SELECT 
*,
LAG(year_month) OVER(PARTITION BY account_id ORDER BY year_month) AS prev_month
FROM cte2
)

SELECT 
DISTINCT account_id
FROM cte3
WHERE substr(year_month, 6, 2)  - substr(prev_month, 6, 2) = 1

"""

make_select(query)

,account_id
0,3


In [350]:
data = [[1, 1, 7, '2020-03-16'], [2, 1, 4, '2020-12-02'], [3, 1, 7, '2020-05-10'], [4, 1, 6, '2021-12-23'], [5, 1, 5, '2021-05-21'], [6, 1, 6, '2021-10-11'], [7, 2, 6, '2022-10-11']]
orders = pd.DataFrame(data, columns=['order_id', 'product_id', 'quantity', 'purchase_date']).astype({'order_id':'Int64', 'product_id':'Int64', 'quantity':'Int64', 'purchase_date':'datetime64[ns]'})


query = """  

WITH cte AS (SELECT 
product_id, 
strftime('%Y', purchase_date) AS year,
COUNT(order_id) AS total_orders
FROM orders 
GROUP BY product_id,strftime('%Y', purchase_date)
HAVING COUNT(order_id) >= 3),


cte2 AS (SELECT
*,
LAG(year) OVER(PARTITION BY product_id ORDER BY year) AS prev
FROM cte)

SELECT DISTINCT product_id FROM cte2 WHERE year - prev =1


"""


make_select(query)

,product_id
0,1


In [374]:
data = [[3, 'Algeria', 1431], [1, 'Senegal', 2132], [2, 'New Zealand', 1402], [4, 'Croatia', 1817]]
team_points = pd.DataFrame(data, columns=['team_id', 'name', 'points']).astype({'team_id':'Int64', 'name':'object', 'points':'Int64'})
data = [[3, 399], [2, 0], [4, 13], [1, -22]]
points_change = pd.DataFrame(data, columns=['team_id', 'points_change']).astype({'team_id':'Int64', 'points_change':'Int64'})

query = """   

WITH cte AS (SELECT t.team_id, t.name, t.points, t.points + p.points_change AS points_after
FROM team_points t INNER JOIN points_change p ON t.team_id = p.team_id ORDER BY t.team_id)


SELECT 
team_id, name,
DENSE_RANK() OVER(ORDER BY points DESC, name ASC) -
DENSE_RANK() OVER(ORDER BY points_after DESC, name ASC) AS rank_diff 
FROM cte

"""

make_select(query)

,team_id,name,rank_diff
0,1,Senegal,0
1,4,Croatia,-1
2,3,Algeria,1
3,2,New Zealand,0


In [398]:
data = [['x', 66], ['y', 77]]
variables = pd.DataFrame(data, columns=['name', 'value']).astype({'name':'object', 'value':'Int64'})
data = [['x', '>', 'y'], ['x', '<', 'y'], ['x', '=', 'y'], ['y', '>', 'x'], ['y', '<', 'x'], ['x', '=', 'x']]
expressions = pd.DataFrame(data, columns=['left_operand', 'operator', 'right_operand']).astype({'left_operand':'object', 'operator':'object', 'right_operand':'object'})


query = """  

WITH cte AS (SELECT v.value, e.left_operand, e.operator, e.right_operand FROM expressions e INNER JOIN variables v ON e.left_operand = v.name),

cte2 AS (SELECT c.value AS first,c.left_operand, c.operator, v.value AS second, c.right_operand FROM cte c INNER JOIN variables v ON c.right_operand = v.name)


SELECT 
left_operand, operator, right_operand,
CASE 
    WHEN operator = '>' THEN CASE WHEN first > second THEN 'true' ELSE 'false' END
    WHEN operator = '<' THEN CASE WHEN first < second THEN  'true' ELSE  'false' END
    WHEN operator = '=' THEN CASE WHEN first = second THEN 'true' ELSE  'false' END
END AS value
FROM cte2

"""

make_select(query)

,left_operand,operator,right_operand,value
0,x,>,y,false
1,x,<,y,true
2,x,=,y,false
3,y,>,x,true
4,y,<,x,false
5,x,=,x,true


In [ ]:
data = [[1, 'Ajax'], [4, 'Dortmund'], [6, 'Arsenal']]
teams = pd.DataFrame(data, columns=['team_id', 'team_name']).astype({'team_id':'Int64', 'team_name':'object'})
data = {
    "home_team_id": [4, 7, 7, 7, 3, 3, 3, 2],
    "away_team_id": [2, 4, 3, 2, 4, 7, 2, 7],
    "home_team_goals": [3, 3, 1, 0, 0, 4, 3, 5],
    "away_team_goals": [1, 4, 2, 1, 4, 3, 2, 3],
}

matches = pd.DataFrame(data)


query = """   


WITH cte AS (SELECT * FROM matches
UNION ALL
SELECT away_team_id,home_team_id,away_team_goals,home_team_goals FROM matches),

mathes AS (SELECT
home_team_id AS team_id,
COUNT(away_team_id) AS matches_played,
COUNT(CASE WHEN home_team_goals = away_team_goals THEN 1 END) * 1 +
COUNT(CASE WHEN home_team_goals > away_team_goals THEN 1 END) * 3 +
COUNT(CASE WHEN home_team_goals < away_team_goals THEN 1 END) * 0 AS points
FROM cte 
GROUP BY home_team_id),

goal_for AS (SELECT
home_team_id AS team_id,
SUM(home_team_goals) AS goal_for
FROM  cte
GROUP BY home_team_id),


goal_against AS (SELECT
home_team_id AS team_id,
SUM(away_team_goals) AS goal_against
FROM  cte
GROUP BY home_team_id)


SELECT 
t.team_name, m.matches_played, m.points, g.goal_for, a.goal_against, g.goal_for - a.goal_against AS goal_diff
FROM mathes m
INNER JOIN goal_for g ON m.team_id = g.team_id
INNER JOIN goal_against a ON m.team_id = a.team_id
INNER JOIN teams t ON t.team_id = m.team_id
ORDER BY m.points DESC, goal_diff DESC, t.team_name ASC




"""

make_select(query)

,team_id,matches_played,points,goal_for,goal_against,goal_diff
0,2,4,6,9,9,0
1,3,4,9,9,10,-1
2,4,3,9,11,4,7
3,7,5,0,10,16,-6


In [492]:
data = [[1, 'reviews', 7], [3, 'reviews', 3], [1, 'ads', 11], [2, 'ads', 7], [3, 'ads', 6], [1, 'page views', 3], [2, 'page views', 12]]
events = pd.DataFrame(data, columns=['business_id', 'event_type', 'occurrences']).astype({'business_id': 'Int64', 'event_type': 'object', 'occurrences': 'Int64'})


query = """  


WITH cte AS (SELECT 
event_type, 
SUM(occurrences) * 1.0/ COUNT(*) AS average_activity
FROM events
GROUP BY event_type),


cte2 AS (SELECT 
business_id, 
event_type,
MAX(occurrences) AS max_occ
FROM events
GROUP BY business_id, event_type)


SELECT 
a.business_id
FROM cte2 a
INNER JOIN cte b ON a.event_type = b.event_type
WHERE a.max_occ > b.average_activity
GROUP BY a.business_id
HAVING COUNT(a.max_occ) >= 2

"""


make_select(query)

,business_id
0,1


In [ ]:
data = [[1, 'Chelsea', 22, 13, 2, 7], [2, 'Nottingham Forest', 27, 6, 6, 15], [3, 'Liverpool', 17, 1, 8, 8], [4, 'Aston Villa', 20, 1, 6, 13], [5, 'Fulham', 31, 18, 1, 12], [6, 'Burnley', 26, 6, 9, 11], [7, 'Newcastle United', 33, 11, 10, 12], [8, 'Sheffield United', 20, 18, 2, 0], [9, 'Luton Town', 5, 4, 0, 1], [10, 'Everton', 14, 2, 6, 6]]
team_stats = pd.DataFrame(data, columns=["team_id", "team_name", "matches_played", "wins", "draws", "losses"]).astype({"team_id": "int", "team_name": "string", "matches_played": "int", "wins": "int", "draws": "int", "losses": "int"})



query = """  

WITH cte AS (SELECT team_name,wins*3+draws * 1+losses *0 AS points,
RANK() OVER(ORDER BY wins*3+draws * 1+losses *0 DESC) AS position
FROM team_stats)


SELECT
*,
CASE 
    WHEN position <= ROUND((SELECT COUNT(*) FROM cte) * 0.35,0) THEN 'Tier 1'
    WHEN position <= ROUND((SELECT COUNT(*) FROM cte) * 0.7, 0) THEN 'Tier 2'
    ELSE 'Tier 3'
END AS tier
FROM cte


"""

make_select(query)

,team_name,points,position,tier
0,Sheffield United,56,1,Tier 1
1,Fulham,55,2,Tier 1
2,Newcastle United,43,3,Tier 1
3,Chelsea,41,4,Tier 1
4,Burnley,27,5,Tier 2
5,Nottingham Forest,24,6,Tier 2
6,Luton Town,12,7,Tier 2
7,Everton,12,7,Tier 2
8,Liverpool,11,9,Tier 3
9,Aston Villa,9,10,Tier 3


In [538]:
data = [[1, 'Nadal'], [2, 'Federer'], [3, 'Novak']]
players = pd.DataFrame(data, columns=['player_id', 'player_name']).astype({'player_id':'Int64', 'player_name':'object'})
data = [[2018, 1, 1, 1, 1], [2019, 1, 1, 2, 2], [2020, 2, 1, 2, 2]]
championships = pd.DataFrame(data, columns=['year', 'Wimbledon', 'Fr_open', 'US_open', 'Au_open']).astype({'year':'Int64', 'Wimbledon':'Int64', 'Fr_open':'Int64', 'US_open':'Int64', 'Au_open':'Int64'})


query = """   

WITH cte  AS (SELECT Wimbledon FROM championships
UNION ALL
SELECT Fr_open FROM championships
UNION ALL
SELECT US_open FROM championships
UNION ALL 
SELECT 	Au_open FROM championships)


SELECT 
Wimbledon AS player_id,p.player_name ,
COUNT(Wimbledon) AS grand_slams_count
FROM cte c
INNER JOIN players p ON p.player_id = c.Wimbledon
GROUP BY Wimbledon

"""

make_select(query)

,player_id,player_name,grand_slams_count
0,1,Nadal,7
1,2,Federer,5


In [554]:
data = [[10, 'Leetcode FC'], [20, 'NewYork FC'], [30, 'Atlanta FC'], [40, 'Chicago FC'], [50, 'Toronto FC']]
teams = pd.DataFrame(data, columns=['team_id', 'team_name']).astype({'team_id':'Int64', 'team_name':'object'})
data = [[1, 10, 20, 3, 0], [2, 30, 10, 2, 2], [3, 10, 50, 5, 1], [4, 20, 30, 1, 0], [5, 50, 30, 1, 0]]
matches = pd.DataFrame(data, columns=['match_id', 'host_team', 'guest_team', 'host_goals', 'guest_goals']).astype({'match_id':'Int64', 'host_team':'Int64', 'guest_team':'Int64', 'host_goals':'Int64', 'guest_goals':'Int64'})


query = """   

WITH cte AS (SELECT * FROM matches
UNION ALL
SELECT match_id, guest_team,host_team,guest_goals,host_goals FROM matches),

cte2 AS (SELECT
host_team,
COUNT(CASE WHEN host_goals = guest_goals THEN host_goals END) * 1 +
COUNT(CASE WHEN host_goals > guest_goals THEN host_goals END) * 3 +
COUNT(CASE WHEN host_goals < guest_goals THEN host_goals END) * 0 AS num_points
FROM cte
GROUP BY host_team)


SELECT 
t.team_id, t.team_name, COALESCE(num_points, 0) AS num_points 
FROM cte2 c 
RIGHT JOIN teams t ON t.team_id = c.host_team
ORDER BY num_points DESC, t.team_id ASC

"""

make_select(query)

,team_id,team_name,num_points
0,10,Leetcode FC,7
1,20,NewYork FC,3
2,50,Toronto FC,3
3,30,Atlanta FC,1
4,40,Chicago FC,0


In [569]:
data = [[2, None, 6, 15], [18, 14, 4, 15], [19, 3, 8, 4], [12, 2, 19, 20], [20, 6, 12, 9], [8, 6, 9, 9], [3, 14, 16, 7]]
boxes = pd.DataFrame(data, columns=['box_id', 'chest_id', 'apple_count', 'orange_count']).astype({'box_id':'Int64', 'chest_id':'Int64', 'apple_count':'Int64', 'orange_count':'Int64'})
data = [[6, 5, 6], [14, 20, 10], [2, 8, 8], [3, 19, 4], [16, 19, 19]]
chests = pd.DataFrame(data, columns=['chest_id', 'apple_count', 'orange_count']).astype({'chest_id':'Int64', 'apple_count':'Int64', 'orange_count':'Int64'})


query = """  

SELECT 
SUM(b.apple_count + COALESCE(ch.apple_count,0)) AS apple_count,
SUM(b.orange_count + COALESCE(ch.orange_count,0)) AS orange_count
FROM boxes b LEFT JOIN chests ch ON b.chest_id = ch.chest_id

"""

make_select(query)

,apple_count,orange_count
0,151,123


In [104]:
books = pd.DataFrame({
    "book_id": [1, 2, 3, 4, 5],
    "name": ["Kalila And Demna", "28 Letters", "The Hobbit", "13 Reasons Why", "The Hunger Games"],
    "available_from": pd.to_datetime(["2010-01-01", "2010-01-01", "2010-01-01", "2010-01-01", "2010-01-01"])
})



orders = pd.DataFrame({
    "order_id": [1, 2, 3, 4, 5, 6, 7],
    "book_id": [1, 1, 3, 4, 4, 5, 5],
    "quantity": [2, 1, 8, 6, 5, 9, 8],
    "dispatch_date": pd.to_datetime(["2018-07-26", "2018-11-05", "2019-06-11", "2019-06-05", "2019-06-20", "2009-02-02", "2010-04-13"])
})



query = """   

WITH cte AS (SELECT 
book_id, name
FROM books
WHERE available_from < '2019-05-23'
GROUP BY book_id, name)


SELECT 
b.book_id,b.name
FROM cte b LEFT JOIN orders o ON b.book_id = o.book_id
GROUP BY b.book_id, b.name
HAVING SUM(CASE WHEN o.dispatch_date BETWEEN '2018-06-23' AND '2019-06-23' THEN quantity ELSE 0 END) < 10

"""

make_select(query)


,book_id,name
0,1,Kalila And Demna
1,2,28 Letters
2,3,The Hobbit
3,5,The Hunger Games


In [126]:
data = [[9, 'Alice'], [11, 'Bob'], [3, 'Winston'], [8, 'Hercy'], [1, 'Narihan']]
members = pd.DataFrame(data, columns=['member_id', 'name']).astype({'member_id':'Int64', 'name':'object'})
data = [[22, 11, '2021-10-28'], [16, 11, '2021-01-12'], [18, 9, '2021-12-10'], [19, 3, '2021-10-19'], [12, 11, '2021-03-01'], [17, 8, '2021-05-07'], [21, 9, '2021-05-12']]
visits = pd.DataFrame(data, columns=['visit_id', 'member_id', 'visit_date']).astype({'visit_id':'Int64', 'member_id':'Int64', 'visit_date':'datetime64[ns]'})
data = [[12, 2000], [18, 9000], [17, 7000]]
purchases = pd.DataFrame(data, columns=['visit_id', 'charged_amount']).astype({'visit_id':'Int64', 'charged_amount':'Int64'})



query = """  

WITH cte AS (SELECT 
m.member_id, m.name, COALESCE(v.visit_id, 0) AS visit_id,
COALESCE(p.charged_amount, 0) AS charged_amount
FROM members m
FULL OUTER JOIN visits v ON m.member_id = v.member_id
FULL OUTER JOIN purchases p ON v.visit_id = p.visit_id),


cte2 AS (SELECT 
member_id, name,
(COUNT(CASE WHEN charged_amount > 0 THEN 1 END) * 100) * 1.0 / NULLIF(COUNT(CASE WHEN visit_id > 0  THEN 1 END),0) AS con_rate
FROM cte
GROUP BY member_id, name)


SELECT 
member_id, name,
CASE 
    WHEN con_rate >= 80 THEN  'Diamond'
    WHEN con_rate >= 50 AND con_rate < 80 THEN  'Gold'
    WHEN con_rate < 50 THEN  'Silver'
    WHEN con_rate IS NULL THEN  'Bronze'
END AS category
FROM cte2


"""

make_select(query)

,member_id,name,category
0,1,Narihan,Bronze
1,3,Winston,Silver
2,8,Hercy,Diamond
3,9,Alice,Gold
4,11,Bob,Silver


In [169]:
data = [[190, 1, 5, 2], [191, 2, 3, 5], [192, 5, 2, 3], [193, 1, 3, 5], [194, 4, 5, 2], [195, 4, 2, 1], [196, 1, 5, 2]]
contests = pd.DataFrame(data, columns=['contest_id', 'gold_medal', 'silver_medal', 'bronze_medal']).astype({'contest_id':'Int64', 'gold_medal':'Int64', 'silver_medal':'Int64', 'bronze_medal':'Int64'})
data = [[1, 'sarah@leetcode.com', 'Sarah'], [2, 'bob@leetcode.com', 'Bob'], [3, 'alice@leetcode.com', 'Alice'], [4, 'hercy@leetcode.com', 'Hercy'], [5, 'quarz@leetcode.com', 'Quarz']]
users = pd.DataFrame(data, columns=['user_id', 'mail', 'name']).astype({'user_id':'Int64', 'mail':'object', 'name':'object'})



query = """   

WITH gold AS (SELECT 
gold_medal AS user_id
FROM contests
GROUP BY gold_medal
HAVING COUNT(gold_medal) >= 3),


cte AS (SELECT contest_id, gold_medal AS user_id FROM contests
UNION ALL 
SELECT contest_id , silver_medal FROM contests
UNION ALL 
SELECT contest_id,bronze_medal FROM contests),


cte2 AS (SELECT 
contest_id, 
user_id,
COUNT(*) AS medals
FROM cte
GROUP BY contest_id, user_id
ORDER BY user_id, contest_id),


cte3 AS (SELECT 
*,
SUM(medals) OVER(PARTITION BY user_id) AS sum
FROM cte2),

cte4 AS (SELECT 
user_id,
LAG(contest_id) OVER(PARTITION BY user_id) AS prev,
contest_id,
LEAD(contest_id) OVER(PARTITION BY user_id) AS next
FROM cte3 WHERE sum >=3) ,

final AS (SELECT DISTINCT user_id 
FROM cte4 
WHERE next - contest_id = 1 AND contest_id - prev = 1
UNION ALL 
SELECT * FROM gold)


SELECT DISTINCT u.name, u.mail FROM users u INNER JOIN final f ON u.user_id = f.user_id

"""

make_select(query)

,name,mail
0,Sarah,sarah@leetcode.com
1,Bob,bob@leetcode.com
2,Alice,alice@leetcode.com
3,Quarz,quarz@leetcode.com


In [192]:
data = [
    [1, 'hello world of SQL'], 
    [2, 'the QUICK-brown fox'], 
    [3, 'modern-day DATA science'], 
    [4, 'web-based FRONT-end development']
]

# Create DataFrame without specifying the dtype for columns
user_content = pd.DataFrame(data, columns=['content_id', 'content_text'])


def correct_text(s:str):

    words = s.split()

    sp = ["SQL"]

    words = [word if word in sp else word.capitalize() for word in words]
  
    return " ".join(words)


def process_text(user_content: pd.DataFrame) -> pd.DataFrame:
    
    user_content["converted_text"] = user_content["content_text"].apply(correct_text)
    return user_content


process_text(user_content)

,content_id,content_text,converted_text
0,1,hello world of SQL,Hello World Of SQL
1,2,the QUICK-brown fox,The Quick-brown Fox
2,3,modern-day DATA science,Modern-day Data Science
3,4,web-based FRONT-end development,Web-based Front-end Development


In [215]:
data = [[1, 2], [2, 2], [3, 1]]
flights = pd.DataFrame(data, columns=['flight_id', 'capacity']).astype({'flight_id':'Int64', 'capacity':'Int64'})
data = [[101, 1, '2023-07-10 16:30:00'], [102, 1, '2023-07-10 17:45:00'], [103, 1, '2023-07-10 12:00:00'], [104, 2, '2023-07-05 13:23:00'], [105, 2, '2023-07-05 09:00:00'], [106, 3, '2023-07-08 11:10:00'], [107, 3, '2023-07-08 09:10:00']]
passengers = pd.DataFrame(data, columns=['passenger_id', 'flight_id', 'booking_time']).astype({'passenger_id':'Int64', 'flight_id':'Int64', 'booking_time':'datetime64[ns]'})


query = """   

WITH cte AS (SELECT 
p.passenger_id,p.flight_id,p.booking_time,f.capacity,
COUNT(p.flight_id) OVER(PARTITION BY p.flight_id ORDER BY p.booking_time) AS booking
FROM passengers p 
INNER JOIN flights f ON p.flight_id = f.flight_id
ORDER BY p.flight_id, p.booking_time)


SELECT DISTINCT passenger_id, status
FROM (SELECT 
*,
CASE WHEN booking <= capacity THEN 'Confirmed' ELSE  'Waitlist' END AS  status
FROM cte) AS sub
ORDER BY passenger_id

"""


make_select(query)

,passenger_id,status
0,101,Confirmed
1,102,Waitlist
2,103,Confirmed
3,104,Confirmed
4,105,Confirmed
5,106,Waitlist
6,107,Confirmed


In [239]:
data = [[1, 100], [2, 200]]
products = pd.DataFrame(data, columns=['product_id', 'price']).astype({'product_id':'Int64', 'price':'Int64'})
data = [[1, 1, 2], [3, 2, 1], [2, 2, 3], [2, 1, 4], [4, 1, 10]]
purchases = pd.DataFrame(data, columns=['invoice_id', 'product_id', 'quantity']).astype({'invoice_id':'Int64', 'product_id':'Int64', 'quantity':'Int64'})


query = """  

WITH cte AS (SELECT 
pr.invoice_id,p.product_id,  p.price , pr.quantity,p.price * pr.quantity AS amount
FROM products p 
FULL OUTER JOIN purchases pr ON p.product_id = pr.product_id),


cte2 AS (SELECT DISTINCT invoice_id FROM (SELECT 
invoice_id,
SUM(amount) AS total,
DENSE_RANK() OVER(ORDER BY SUM(amount) DESC, invoice_id ASC) AS ranking
FROM cte
GROUP BY invoice_id) WHERE ranking =1)


SELECT product_id, quantity, amount FROM cte WHERE invoice_id = (SELECT invoice_id FROM cte2)



"""


make_select(query)

,product_id,quantity,amount
0,1,4,400
1,2,3,600


In [ ]:
data = [[1, 1, 45], [1, 2, 90], [2, 3, 12], [2, 4, 68]]
project = pd.DataFrame(data, columns=['project_id', 'employee_id', 'workload']).astype({'project_id':'Int64', 'employee_id':'Int64', 'workload':'Int64'})

data = [[1, 'Khaled', 'A'], [2, 'Ali', 'B'], [3, 'John', 'B'], [4, 'Doe', 'A']]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'team']).astype({'employee_id':'Int64', 'name':'object', 'team':'object'})


query = """  

WITH cte AS (SELECT 
p.project_id, p.employee_id, e.name, p.workload, e.team,
AVG(p.workload) OVER(PARTITION BY e.team) AS avg_work
FROM project p 
INNER JOIN employees e ON p.employee_id = e.employee_id)


SELECT 
employee_id, project_id, name AS employee_name, workload AS project_workload
FROM cte
WHERE workload > avg_work
ORDER BY employee_id, project_id

"""

make_select(query)

,employee_id,project_id,employee_name,project_workload
0,2,1,Ali,90
1,4,2,Doe,68


In [276]:
data = [['Alice', 'Travel', '2020-02-12', '2020-02-20'], ['Alice', 'Dancing', '2020-02-21', '2020-02-23'], ['Alice', 'Travel', '2020-02-24', '2020-02-28'], ['Bob', 'Travel', '2020-02-11', '2020-02-18']]
user_activity = pd.DataFrame(data, columns=['username', 'activity', 'startDate', 'endDate']).astype({'username':'object', 'activity':'object', 'startDate':'datetime64[ns]', 'endDate':'datetime64[ns]'})


query = """   

WITH cte AS (SELECT 
*,
RANK() OVER(PARTITION BY username ORDER BY endDate DESC) AS ranking
FROM user_activity),

cte2 AS (SELECT
*,
COUNT(ranking) OVER(PARTITION BY username) AS count
FROM cte)


SELECT username,activity,startDate,endDate FROM cte WHERE ranking =2
UNION ALL
SELECT 
username,activity,startDate,endDate
FROM cte2
WHERE count =1

"""

make_select(query)

,username,activity,startDate,endDate
0,Alice,Dancing,2020-02-21 00:00:00.000000,2020-02-23 00:00:00.000000
1,Bob,Travel,2020-02-11 00:00:00.000000,2020-02-18 00:00:00.000000


In [333]:
# user_data = {
#     'user_id': [1, 2, 3, 4],
#     'join_date': ['2019-01-01', '2019-02-09', '2019-01-19', '2019-05-21'],
#     'favorite_brand': ['Lenovo', 'Samsung', 'LG', 'HP']
# }
# users = pd.DataFrame(user_data)

# # Convert join_date to datetime if needed
# users['join_date'] = pd.to_datetime(users['join_date'])



# order_data = {
#     'order_id': [1, 2, 3, 4, 5, 6, 7, 8],
#     'order_date': ['2019-08-01', '2019-08-02', '2019-08-03', '2019-08-04', '2019-08-04', '2019-08-05', '2019-08-06', '2019-08-04'],
#     'item_id': [4, 2, 3, 1, 1, 4, 2, 2],
#     'buyer_id': [1, 1, 2, 4, 3, 3, 2, 3],
#     'seller_id': [2, 3, 3, 2, 4, 4, 4, 1]
# }
# orders = pd.DataFrame(order_data)

# # Convert order_date to datetime if needed
# orders['order_date'] = pd.to_datetime(orders['order_date'])


# item_data = {
#     'item_id': [1, 2, 3, 4],
#     'item_brand': ['Samsung', 'Lenovo', 'LG', 'HP']
# }
# items = pd.DataFrame(item_data)

data = [[1, '2019-01-01', 'Lenovo'], [2, '2019-02-09', 'Samsung'], [3, '2019-01-19', 'LG'], [4, '2019-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})

data = [[1, '2019-08-01', 4, 1, 2], [2, '2019-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2019-08-04', 1, 4, 2], [5, '2019-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})


# SELECT 
# seller_id,
# CASE WHEN favorite_brand=item_brand THEN 'yes' ELSE  'no' END AS '2nd_item_fav_brand'
# FROM cte2 WHERE rn = 2


query = """   

WITH cte AS (SELECT 
o.item_id, 
u.user_id AS seller_id, u.favorite_brand, i.item_brand,o.order_id, o.order_date,
RANK() OVER(PARTITION BY u.user_id ORDER BY o.order_date) AS rn
FROM orders o 
FULL OUTER JOIN users u ON o.seller_id = u.user_id
FULL OUTER JOIN items i ON o.item_id = i.item_id),


cte2 AS (SELECT 
*,
COUNT(rn) OVER(PARTITION BY seller_id) AS count
FROM cte)


SELECT 
seller_id,
CASE WHEN favorite_brand=item_brand THEN 'yes' ELSE  'no' END AS "2nd_item_fav_brand"
FROM cte2
WHERE rn = 2
UNION ALL 
SELECT 
seller_id,
CASE WHEN favorite_brand=item_brand THEN 'no' ELSE  'no' END AS "2nd_item_fav_brand"
FROM cte2
WHERE count < 2



"""

make_select(query)

,seller_id,2nd_item_fav_brand
0,2,yes
1,3,yes
2,4,no
3,1,no


In [423]:
data = [[1, 65.56, '2023-11-18 13:49:42'], [1, 96.0, '2023-11-30 02:47:26'], [1, 7.44, '2023-11-02 12:15:23'], [1, 49.78, '2023-11-12 00:13:46'], [2, 40.89, '2023-11-21 04:39:15'], [2, 100.44, '2023-11-20 07:39:34'], [3, 37.33, '2023-11-03 06:22:02'], [3, 13.89, '2023-11-11 16:00:14'], [3, 7.0, '2023-11-29 22:32:36']]
transactions = pd.DataFrame(data, columns=['user_id', 'spend', 'transaction_date']).astype({
    'user_id': 'Int64', 
    'spend': 'float',  # pandas does not have a fixed decimal; float is used for decimal numbers
    'transaction_date': 'datetime64[ns]'  # specifying datetime type
})




query = """  

WITH cte AS (SELECT 
*,
LAG(spend) OVER(PARTITION BY user_id ORDER BY transaction_date) AS fr,
LAG(spend,2) OVER(PARTITION BY user_id ORDER BY transaction_date) AS sc,
ROW_NUMBER() OVER(PARTITION BY user_id ORDER BY transaction_date) AS rn
FROM transactions 
ORDER BY user_id)


SELECT 
user_id, spend AS third_transaction_spend, transaction_date AS third_transaction_date
FROM cte
WHERE spend > fr AND spend > sc AND rn = 3



"""

make_select(query)

,user_id,third_transaction_spend,third_transaction_date
0,1,65.56,2023-11-18 13:49:42.000000


In [449]:
data = [['Kathy', None], ['Charles', 'Ryan'], ['Charles', 'Christine'], ['Charles', 'Kathy'], ['Benjamin', 'Christine'], ['Anthony', 'Ryan'], ['Edward', 'Ryan'], ['Terry', None], ['Evelyn', 'Kathy'], ['Arthur', 'Christine']]
votes = pd.DataFrame(data, columns=['voter', 'candidate']).astype({'voter':'object', 'candidate':'object'})



query = """  


WITH cte AS (SELECT 
*,
COUNT(candidate) OVER(PARTITION BY voter) AS votes
FROM votes),

cte2 AS (SELECT 
*,
CASE WHEN votes >1 THEN 1.0 / votes ELSE votes END AS total
FROM cte)

SELECT candidate
FROM (SELECT
candidate,
DENSE_RANK() OVER(ORDER BY SUM(total) DESC) AS ranking
FROM cte2
WHERE candidate IS NOT NULL
GROUP BY candidate)
WHERE ranking = 1
ORDER BY candidate

"""

make_select(query)

,candidate
0,Christine
1,Ryan


In [16]:
data = [['1', '1', 'N'], ['2', '1', 'Y'], ['2', '2', 'N'], ['3', '3', 'N'], ['4', '2', 'N'], ['4', '3', 'Y'], ['4', '4', 'N']]
employee = pd.DataFrame(data, columns=['employee_id', 'department_id', 'primary_flag']).astype({'employee_id':'Int64', 'department_id':'Int64', 'primary_flag':'object'})


query = """   

WITH cte AS (SELECT 
*,
ROW_NUMBER()  OVER(PARTITION BY employee_id ORDER BY department_id) AS rn
FROM employee),


cte2 AS (SELECT 
*,
COUNT(rn) OVER(PARTITION BY employee_id) AS count
FROM cte),


cte3 AS (SELECT 
*,
CASE WHEN count >= 2 THEN 'Y' ELSE 'N' END AS nn
FROM cte2)

SELECT employee_id, department_id FROM cte3 WHERE primary_flag = nn

"""


make_select(query)

,employee_id,department_id
0,1,1
1,2,1
2,3,3
3,4,3


In [64]:
data = [[1, 3], [2, 3], [7, 3], [1, 4], [2, 4], [7, 4], [1, 5], [2, 6], [7, 5]]
relations = pd.DataFrame(data, columns=['user_id', 'follower_id']).astype({'user_id':'Int64', 'follower_id':'Int64'})

query = """   


WITH cte AS (SELECT
a.user_id,
b.user_id AS follower_id,
COUNT(*) AS cc
FROM relations a 
JOIN relations b 
ON a.follower_id = b.follower_id
AND a.user_id < b.user_id
GROUP BY a.user_id,b.user_id 
)


SELECT user_id AS user1_id, follower_id AS user2_id
FROM cte WHERE cc = (SELECT MAX(cc) FROM cte)

"""

make_select(query)

,user1_id,user2_id
0,1,7


In [137]:
data = [[10, '2019-12-10'], [8, '2020-1-13'], [5, '2020-2-16'], [7, '2020-3-8'], [4, '2020-5-17'], [1, '2020-10-24'], [6, '2021-1-5']]
drivers = pd.DataFrame(data, columns=['driver_id', 'join_date']).astype({'driver_id':'Int64', 'join_date':'datetime64[ns]'})
data = [[6, 75, '2019-12-9'], [1, 54, '2020-2-9'], [10, 63, '2020-3-4'], [19, 39, '2020-4-6'], [3, 41, '2020-6-3'], [13, 52, '2020-6-22'], [7, 69, '2020-7-16'], [17, 70, '2020-8-25'], [20, 81, '2020-11-2'], [5, 57, '2020-11-9'], [2, 42, '2020-12-9'], [11, 68, '2021-1-11'], [15, 32, '2021-1-17'], [12, 11, '2021-1-19'], [14, 18, '2021-1-27']]
rides = pd.DataFrame(data, columns=['ride_id', 'user_id', 'requested_at']).astype({'ride_id':'Int64', 'user_id':'Int64', 'requested_at':'datetime64[ns]'})
data = [[10, 10, 63, 38], [13, 10, 73, 96], [7, 8, 100, 28], [17, 7, 119, 68], [20, 1, 121, 92], [5, 7, 42, 101], [2, 4, 6, 38], [11, 8, 37, 43], [15, 8, 108, 82], [12, 8, 38, 34], [14, 1, 90, 74]]
accepted_rides = pd.DataFrame(data, columns=['ride_id', 'driver_id', 'ride_distance', 'ride_duration']).astype({'ride_id':'Int64', 'driver_id':'Int64', 'ride_distance':'Int64', 'ride_duration':'Int64'})




query = """  

WITH cte AS (SELECT 
a.ride_distance, a.ride_duration,CAST(strftime('%m', r.requested_at) AS INTEGER) AS month
FROM accepted_rides a 
INNER JOIN rides r ON a.ride_id = r.ride_id
INNER JOIN drivers d ON a.driver_id = d.driver_id
WHERE strftime('%Y', r.requested_at) = '2020'
ORDER BY r.requested_at),

cte2 AS (SELECT 1 AS month
UNION ALL
SELECT 2
UNION ALL
SELECT 3
UNION ALL
SELECT 4
UNION ALL
SELECT 5
UNION ALL
SELECT 6
UNION ALL
SELECT 7
UNION ALL
SELECT 8
UNION ALL
SELECT 9
UNION ALL
SELECT 10
UNION ALL
SELECT 11
UNION ALL
SELECT 12
),



final AS (SELECT 
b.month, SUM(COALESCE(a.ride_distance, 0)) AS ride_distance, SUM(COALESCE(a.ride_duration,0)) AS ride_duration
FROM cte a
FULL OUTER JOIN cte2 b ON a.month = b.month
GROUP BY b.month
ORDER BY b.month),


rr AS (SELECT 
    month,
    ROUND(AVG(ride_distance) OVER(ORDER BY month ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING),2) AS average_ride_distance,
    ROUND(AVG(ride_duration) OVER(ORDER BY month ROWS BETWEEN CURRENT ROW AND 2 FOLLOWING),2) AS average_ride_duration 
FROM final
ORDER BY month)

SELECT * FROM rr WHERE month <= 10




"""


make_select(query)

,month,average_ride_distance,average_ride_duration
0,1,21.00,12.67
1,2,21.00,12.67
2,3,21.00,12.67
3,4,24.33,32.00
4,5,57.67,41.33
5,6,97.33,64.00
6,7,73.00,32.00
7,8,39.67,22.67
8,9,54.33,64.33
9,10,56.33,77.00


In [185]:
data = [[2, 'Meir', 3000], [3, 'Michael', 3000], [7, 'Addilyn', 7400], [8, 'Juan', 6100], [9, 'Kannon', 7400]]
employees = pd.DataFrame(data, columns=['employee_id', 'name', 'salary']).astype({'employee_id':'Int64', 'name':'object', 'salary':'Int64'})


query = """  

WITH cte AS (SELECT 
*,
COUNT(*) OVER(PARTITION BY salary) AS same
FROM employees),

cte2 AS (SELECT 
*
FROM 
cte
WHERE same >= 2)


SELECT 
employee_id, name, salary,
DENSE_RANK() OVER(ORDER BY salary) AS team_id
FROM cte2
ORDER BY employee_id, team_id

"""

make_select(query)

,employee_id,name,salary,team_id
0,2,Meir,3000,1
1,3,Michael,3000,1
2,7,Addilyn,7400,2
3,9,Kannon,7400,2


In [204]:
data = [[101, 'US', 'approved', 1000, '2019-05-18'], [102, 'US', 'declined', 2000, '2019-05-19'], [103, 'US', 'approved', 3000, '2019-06-10'], [104, 'US', 'declined', 4000, '2019-06-13'], [105, 'US', 'approved', 5000, '2019-06-15']]
transactions = pd.DataFrame(data, columns=['id', 'country', 'state', 'amount', 'trans_date']).astype({'id':'Int64', 'country':'object', 'state':'object', 'amount':'Int64', 'trans_date':'datetime64[ns]'})
data = [[102, '2019-05-29'], [101, '2019-06-30'], [105, '2019-09-18']]
chargebacks = pd.DataFrame(data, columns=['trans_id', 'trans_date']).astype({'trans_id':'Int64', 'trans_date':'datetime64[ns]'})


query = """  

WITH charge AS (SELECT 
strftime('%Y-%m', ch.trans_date) AS year_month,
t.country,
COUNT(*) AS chargeback_count,
SUM(t.amount) AS chargeback_amount
FROM transactions t RIGHT JOIN chargebacks ch ON t.id = ch.trans_id
GROUP BY strftime('%Y-%m', ch.trans_date)),


cte AS (SELECT 
strftime('%Y-%m', ch.trans_date) AS year_month, 
t.country,
COALESCE(SUM(CASE WHEN t.state = 'approved' THEN t.amount END),0) AS approved_amount,
COALESCE(COUNT(CASE WHEN t.state = 'approved' THEN t.amount END),0) AS approved_count
FROM transactions t 
RIGHT JOIN chargebacks ch ON strftime('%Y-%m', t.trans_date) = strftime('%Y-%m', ch.trans_date)
GROUP BY strftime('%Y-%m', ch.trans_date), t.country)


SELECT 
a.year_month AS month, a.country, b.approved_count,b.approved_amount,
a.chargeback_count,a.chargeback_amount
FROM charge a INNER JOIN cte b ON a.year_month = b.year_month


"""

make_select(query)

,month,country,approved_count,approved_amount,chargeback_count,chargeback_amount
0,2019-05,US,1,1000,1,2000
1,2019-06,US,2,8000,1,1000
2,2019-09,US,0,0,1,5000


In [240]:
data = [
    [1, '08:00:00', '12:00:00'],
    [1, '11:00:00', '15:00:00'],
    [1, '14:00:00', '18:00:00'],
    [2, '09:00:00', '17:00:00'],
    [2, '16:00:00', '20:00:00'],
    [3, '10:00:00', '12:00:00'],
    [3, '13:00:00', '15:00:00'],
    [3, '16:00:00', '18:00:00'],
    [4, '08:00:00', '10:00:00'],
    [4, '09:00:00', '11:00:00']
]

# Create DataFrame with column names
employee_shifts = pd.DataFrame(data, columns=['employee_id', 'start_time', 'end_time'])

# Convert start_time and end_time to datetime format
employee_shifts['start_time'] = pd.to_datetime(employee_shifts['start_time'], format='%H:%M:%S')
employee_shifts['end_time'] = pd.to_datetime(employee_shifts['end_time'], format='%H:%M:%S')


query = """  

SELECT 
a.employee_id,
COUNT(*) AS overlap
FROM employee_shifts a 
JOIN employee_shifts b ON a.employee_id = b.employee_id
AND a.start_time > b.start_time AND a.start_time < b.end_time
GROUP BY a.employee_id
ORDER BY a.employee_id

"""

make_select(query)

,employee_id,overlap
0,1,2
1,2,1
2,4,1


In [278]:
data = [[1, 2, 59], [2, 1, 11], [1, 3, 20], [3, 4, 100], [3, 4, 200], [3, 4, 200], [4, 3, 499]]
calls = pd.DataFrame(data, columns=['from_id', 'to_id', 'duration']).astype({'from_id':'Int64', 'to_id':'Int64', 'duration':'Int64'})



query = """   

WITH cte AS (SELECT
*
FROM calls
UNION ALL
SELECT to_id, from_id, duration FROM calls)

SELECT
DISTINCT from_id AS person1, 
to_id AS person2, 
COUNT(*) AS call_count,
SUM(duration) AS total_duration
FROM cte
WHERE from_id < to_id
GROUP BY from_id, to_id


"""

make_select(query)

,person1,person2,call_count,total_duration
0,1,2,2,70
1,1,3,1,20
2,3,4,4,999


In [290]:
data = [[1, 1, 0], [2, 1, 0], [11, 2, 0], [12, 2, 1], [21, 3, 1], [22, 3, 0], [31, 4, 1], [32, 4, 1]]
orders = pd.DataFrame(data, columns=['order_id', 'customer_id', 'order_type']).astype({'order_id':'Int64', 'customer_id':'Int64', 'order_type':'Int64'})


query = """  

WITH cte AS (SELECT 
*,
COUNT(CASE WHEN order_type = 0 THEN 1 END) OVER(PARTITION BY customer_id) AS type_count
FROM orders),


cte2 AS (SELECT
*,
CASE WHEN type_count >= 1 THEN 0 ELSE order_type END AS type
FROM cte)

SELECT
order_id, customer_id, order_type
FROM cte2 
WHERE order_type = type

"""

make_select(query)

,order_id,customer_id,order_type
0,1,1,0
1,2,1,0
2,11,2,0
3,22,3,0
4,31,4,1
5,32,4,1


In [301]:
data = [[1, 5], [2, 12], [3, 7], [4, 3]]

# Create DataFrame with proper column names
user_permissions = pd.DataFrame(data, columns=['user_id', 'permissions'])



def analyze_permissions(user_permissions: pd.DataFrame) -> pd.DataFrame:
     
    rows = user_permissions["permissions"].tolist()

    bitwise_or, bitwise_and = 0, 0

    for n in rows:
        bitwise_or|=n
        bitwise_and&=n
    
    return pd.DataFrame({
        "common_perms":[bitwise_and],
        "any_perms":[bitwise_or]
        
    })

analyze_permissions(user_permissions)

,common_perms,any_perms
0,0,15


In [ ]:
data = [[1, 2], [2, 2], [3, 1]]
flights = pd.DataFrame(data, columns=['flight_id', 'capacity']).astype({'flight_id':'Int64', 'capacity':'Int64'})
data = [[101, 1], [102, 1], [103, 1], [104, 2], [105, 2], [106, 3], [107, 3]]
passengers = pd.DataFrame(data, columns=['passenger_id', 'flight_id']).astype({'passenger_id':'Int64', 'flight_id':'Int64'})


query = """  

WITH cte AS (SELECT 
p.passenger_id,p.flight_id, f.capacity,
COUNT(p.flight_id) OVER(PARTITION BY p.flight_id ORDER BY p.passenger_id) AS count
FROM passengers p
INNER JOIN flights f ON p.flight_id = f.flight_id),


cte2 AS (SELECT
*,
CASE WHEN count <= capacity THEN 'con' ELSE 'wait' END AS c
FROM cte),


cte3 AS (SELECT
flight_id,
COUNT(CASE WHEN c='con' THEN 1 END) AS booked_cnt,
COUNT(CASE WHEN c='wait' THEN 1 END) AS waitlist_cnt
FROM cte2
GROUP BY flight_id
ORDER BY flight_id),


cte4 AS (SELECT DISTINCT flight_id FROM flights)



SELECT
*
FROM cte3 a RIGHT JOIN cte4 b ON a.flight_id = b.flight_id

"""

make_select(query)

,flight_id,booked_cnt,waitlist_cnt,flight_id
0,1,2,1,1
1,2,2,0,2
2,3,1,1,3


In [90]:
actions = pd.DataFrame({
    "user_id": [1, 1, 1, 2, 2, 3, 3, 4, 4, 5, 5, 5, 5],
    "post_id": [1, 1, 1, 2, 2, 4, 4, 3, 3, 2, 2, 5, 5],
    "action_date": [
        "2019-07-01", "2019-07-01", "2019-07-01", 
        "2019-07-04", "2019-07-04", "2019-07-04", 
        "2019-07-04", "2019-07-02", "2019-07-02", 
        "2019-07-07", "2019-07-07", "2019-07-07", "2019-07-07"
    ],
    "action": [
        "view", "reaction", "share", "view", "report", 
        "view", "report", "view", "report", 
        "view", "report", "view", "report"
    ],
    "extra": [None, "love", None, None, "spam", None, "spam", None, "spam", None, "spam", None, "spam"]
})

# Define the second table
removals = pd.DataFrame({
    "post_id": [2, 3],
    "remove_date": ["2019-07-20", "2019-07-18"]
})

query = """  

WITH cte AS (SELECT DISTINCT  action_date, post_id,
COUNT(post_id) OVER(PARTITION BY action_date) AS count
FROM actions 
WHERE action = 'report' AND extra  = 'spam'
ORDER BY action_date),


cte2 AS (SELECT 
a.action_date, 
COUNT(a.action_date) AS ac_count,
COUNT(r.remove_date) AS re_count
FROM cte a 
LEFT JOIN removals r ON a.post_id = r.post_id
GROUP BY a.action_date)


SELECT ROUND(AVG(re_count * 1.0/ac_count* 100), 2) AS average_daily_percent
FROM cte2

"""


make_select(query)

,average_daily_percent
0,66.67


In [116]:
data = [[1, 'Joe', 70000, 3], [2, 'Henry', 80000, 4], [3, 'Sam', 60000, None], [4, 'Max', 90000, None]]
employee = pd.DataFrame(data, columns=['id', 'name', 'salary', 'managerId']).astype({'id':'Int64', 'name':'object', 'salary':'Int64', 'managerId':'Int64'})



query = """  

WITH cte AS (SELECT a.id, a.name, a.salary, a.managerId, b.salary AS manager_salary FROM employee a 
INNER JOIN employee b ON a.managerId = b.id)


SELECT name AS Employee FROM cte WHERE salary > manager_salary

"""

make_select(query)

,Employee
0,Joe


In [ ]:
data = [['California', 'Los Angeles'], ['California', 'San Francisco'], ['California', 'San Diego'], ['Texas', 'Houston'], ['Texas', 'Austin'], ['Texas', 'Dallas'], ['New York', 'New York City'], ['New York', 'Buffalo'], ['New York', 'Rochester']]
cities = pd.DataFrame(data,columns=['state', 'city'])


query = """  

SELECT 
state, 
GROUP_CONCAT(city ORDER BY city SEPARATOR ', ') AS cities
FROM cities
GROUP BY state

"""

make_select(query)

In [177]:
data = [[1, 30], [2, 199], [3, 299], [4, 580], [5, 1000]]
sessions = pd.DataFrame(data, columns=['session_id', 'duration']).astype({'session_id':'Int64', 'duration':'Int64'})

query = """   

WITH cte AS (SELECT '[0-5>' AS bins
UNION ALL
SELECT '[5-10>'
UNION ALL
SELECT '[10-15>'
UNION ALL
SELECT '15 or more'
),


cte2 AS (SELECT 
session_id, 
SUM(duration) * 1.0 / 60 AS duration
FROM sessions
GROUP BY session_id),


cte3 AS (SELECT 
*,
    CASE 
        WHEN duration >= 0 AND duration < 5 THEN '[0-5>'
        WHEN duration >= 5 AND duration < 10 THEN '[5-10>'
        WHEN duration >= 10 AND duration < 15 THEN '[10-15>'
        WHEN duration >= 15 THEN '15 or more'
    END AS bins
FROM cte2),

cte4 AS (SELECT
bins,
COUNT(session_id) AS total
FROM cte3
GROUP BY bins)

SELECT 
b.bins, COALESCE(a.total, 0) AS total
FROM cte4 a 
RIGHT JOIN cte b ON a.bins = b.bins
ORDER BY total 



"""

make_select(query)

,bins,total
0,[10-15>,0
1,15 or more,1
2,[5-10>,1
3,[0-5>,3


In [185]:
data = [[0, 95, 100, 105], [1, 70, None, 80]]
products = pd.DataFrame(data, columns=['product_id', 'store1', 'store2', 'store3']).astype({'product_id':'Int64', 'store1':'Int64', 'store2':'Int64', 'store3':'Int64'})


query = """  

SELECT product_id, store1 AS store, store1 AS price FROM products WHERE store1 IS NOT NULL
UNION ALL
SELECT product_id, store2, store2 FROM products WHERE store2 IS NOT NULL
UNION ALL
SELECT product_id, store3, store3 FROM products WHERE store3 IS NOT NULL

"""

make_select(query)

,product_id,store,price
0,0,95,95
1,1,70,70
2,0,100,100
3,0,105,105
4,1,80,80


In [ ]:
data = [[1, 'Alice'], [2, 'Bob']]
school_a = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [[3, 'Tom']]
school_b = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [[3, 'Tom'], [2, 'Jerry'], [10, 'Alice']]
school_c = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})



query = """   


SELECT 
a.student_name AS member_A, b.student_name AS member_B, c.student_name AS member_
FROM school_a a
JOIN school_b b
JOIN  school_c c
WHERE a.student_name != b.student_name AND b.student_name != c.student_name AND a.student_name != c.student_name
AND a.student_id != b.student_id AND b.student_id != c.student_id AND a.student_id != c.student_id
"""

make_select(query)

In [227]:
data = [[1, 'What an amazing meal @MaxPower @AlexJones @JohnDoe #Learning #Fitness #Love'], [2, 'Learning something new every day @AnnaWilson #Learning #Foodie'], [3, "Never been happier about today's achievements @SaraJohnson @JohnDoe @AnnaWilson #Fashion"], [4, 'Traveling, exploring, and living my best life @JaneSmith @JohnDoe @ChrisAnderson @AlexJones #WorkLife #Travel'], [5, 'Work hard, play hard, and cherish every moment @AlexJones #Fashion #Foodie'], [6, "Never been happier about today's achievements @ChrisAnderson #Fashion #WorkLife"], [7, "So grateful for today's experiences @AnnaWilson @LisaTaylor @ChrisAnderson @MikeBrown #Fashion #HappyDay #WorkLife #Nature"], [8, 'What an amazing meal @EmilyClark @AlexJones @MikeBrown #Fitness'], [9, 'Learning something new every day @EmilyClark @AnnaWilson @MaxPower #Travel'], [10, "So grateful for today's experiences @ChrisAnderson #Nature"], [11, "So grateful for today's experiences @AlexJones #Art #WorkLife"], [12, 'Learning something new every day @JaneSmith @MikeBrown #Travel'], [13, 'What an amazing meal @EmilyClark @JohnDoe @LisaTaylor @MaxPower #Foodie #Fitness'], [14, 'Work hard, play hard, and cherish every moment @LisaTaylor @SaraJohnson @MaxPower @ChrisAnderson #TechLife #Nature #Music'], [15, 'What a beautiful day it is @EmilyClark @MaxPower @SaraJohnson #Fashion'], [16, 'What a beautiful day it is @AnnaWilson @JaneSmith #Fashion #Love #TechLife']]
tweets = pd.DataFrame(data, columns=['tweet_id', 'content']).astype({'tweet_id':'Int64', 'content':'object'})


def checking(content: str):
    
    if len(content) > 140:
        return True
    if content.count('@') > 3:
        return True
    if content.count('#') > 3:
        return True
    return False

def find_invalid_tweets(tweets: pd.DataFrame) -> pd.DataFrame:
    
    tweets["is_valid"] = tweets["content"].apply(checking)

    valid = tweets["is_valid"] == True

    result = tweets[valid].sort_values(by="tweet_id", ascending=True)

    return result[["tweet_id"]]


find_invalid_tweets(tweets)

,tweet_id
3,4
6,7
12,13
13,14


In [257]:
data = [
    ['New York', 'New York City'], 
    ['New York', 'Newark'], 
    ['New York', 'Buffalo'], 
    ['New York', 'Rochester'], 
    ['California', 'San Francisco'], 
    ['California', 'Sacramento'], 
    ['California', 'San Diego'], 
    ['California', 'Los Angeles'], 
    ['Texas', 'Tyler'], 
    ['Texas', 'Temple'], 
    ['Texas', 'Taylor'], 
    ['Texas', 'Dallas'], 
    ['Pennsylvania', 'Philadelphia'], 
    ['Pennsylvania', 'Pittsburgh'], 
    ['Pennsylvania', 'Pottstown']
]
columns = ['state', 'city']

cities = pd.DataFrame(data, columns=columns)


def checking_cities(citites: str):
    
    cities = citites.split()

    if len(cities) < 3:
        return False
    return True


    
def check_same_letter(row):
    state_first_letter = row['state'][0].lower()
    cities = row['cities'].split(', ')

    count = 0

    for city in cities:
        if city[0].lower() == state_first_letter:
            count+=1

    return  count

def state_city_analysis(cities: pd.DataFrame) -> pd.DataFrame:

    cities = cities.groupby("state")["city"].apply(lambda x: ", ".join(x)).reset_index(name="cities")
    cities["is_val"] = cities["cities"].apply(checking_cities)
    cities["matching_letter_count"] =  cities.apply(check_same_letter, axis=1)

    valid = cities["is_val"] == True

    valid_cities =  cities[valid]

    valid2 = cities["matching_letter_count"] > 0

    result = valid_cities[valid2]

    res = result.sort_values(by=["matching_letter_count", "state"], ascending=[False, True])

    return res[["state", "cities", "matching_letter_count"]]
    

state_city_analysis(cities)


,state,cities,matching_letter_count
2,Pennsylvania,"Philadelphia, Pittsburgh, Pottstown",3
3,Texas,"Tyler, Temple, Taylor, Dallas",3
1,New York,"New York City, Newark, Buffalo, Rochester",2


In [270]:
data = [['Pepperoni', 0.5], ['Sausage', 0.7], ['Chicken', 0.55], ['Extra Cheese', 0.4]]
toppings = pd.DataFrame(data, columns=['topping_name', 'cost']).astype({'topping_name':'object', 'cost':'Float64'})


query = """  

SELECT
CONCAT(a.topping_name, ', ', b.topping_name, ', ', c.topping_name) AS pizza,
ROUND( a.cost + b.cost + c.cost, 2) AS total_cost FROM toppings a
INNER JOIN toppings b  ON a.topping_name < b.topping_name
INNER JOIN toppings c ON b.topping_name < c.topping_name 
ORDER BY total_cost DESC, pizza ASC

"""

make_select(query)

,pizza,total_cost
0,"Chicken, Pepperoni, Sausage",1.75
1,"Chicken, Extra Cheese, Sausage",1.65
2,"Extra Cheese, Pepperoni, Sausage",1.60
3,"Chicken, Extra Cheese, Pepperoni",1.45


In [56]:
data = [[1, 'Daniel'], [2, 'Jade'], [3, 'Stella'], [4, 'Jonathan'], [5, 'Will']]
student = pd.DataFrame(data, columns=['student_id', 'student_name']).astype({'student_id':'Int64', 'student_name':'object'})
data = [[10, 1, 70], [10, 2, 80], [10, 3, 90], [20, 1, 80], [30, 1, 70], [30, 3, 80], [30, 4, 90], [40, 1, 60], [40, 2, 70], [40, 4, 80]]
exam = pd.DataFrame(data, columns=['exam_id', 'student_id', 'score']).astype({'exam_id':'Int64', 'student_id':'Int64', 'score':'Int64'})


query = """  


WITH cte AS (SELECT 
*,
MAX(score) OVER(PARTITION BY exam_id ) AS max_score,
MIN(score) OVER(PARTITION BY exam_id) AS min_score
FROM exam),


cte2 AS (SELECT 
DISTINCT student_id
FROM cte
WHERE score = max_score OR score = min_score),



final AS (SELECT
DISTINCT b.student_id
FROM cte2 a 
FULL OUTER JOIN cte b ON a.student_id = b.student_id
WHERE a.student_id IS NULL
)


SELECT
f.student_id, s.student_name
FROM final f
INNER JOIN student s  ON f.student_id = s.student_id


"""

make_select(query)

,student_id,student_name
0,2,Jade


In [ ]:
data = [[1, 1, 9000, '2017/03/31'], [2, 2, 6000, '2017/03/31'], [3, 3, 10000, '2017/03/31'], [4, 1, 7000, '2017/02/28'], [5, 2, 6000, '2017/02/28'], [6, 3, 8000, '2017/02/28']]
salary = pd.DataFrame(data, columns=['id', 'employee_id', 'amount', 'pay_date']).astype({'id':'Int64', 'employee_id':'Int64', 'amount':'Int64', 'pay_date':'datetime64[ns]'})
data = [[1, 1], [2, 2], [3, 2]]
employee = pd.DataFrame(data, columns=['employee_id', 'department_id']).astype({'employee_id':'Int64', 'department_id':'Int64'})


# SELECT 
# *,
# AVG(amount) OVER(PARTITION BY department_id ORDER BY pay_month) AS avg_dep,
# AVG(amount) OVER(PARTITION BY pay_month) AS avg_com
# FROM cte
# ORDER BY pay_month

query = """   


WITH cte AS (SELECT 
s.id, e.department_id, s.employee_id, s.amount, strftime('%Y-%m', s.pay_date) AS pay_month
FROM salary s INNER JOIN employee e ON s.employee_id = e.employee_id),


cte2 AS (SELECT
pay_month,
department_id,
AVG(amount) AS avg_dep
FROM cte
GROUP BY pay_month, department_id),


cte3 AS (SELECT
pay_month, 
AVG(amount) AS avg_sal
FROM cte
GROUP BY pay_month)

SELECT
a.pay_month, a.department_id,
CASE 
    WHEN avg_dep = avg_sal  THEN 'same' 
    WHEN avg_dep > avg_sal THEN 'higher' 
    WHEN avg_dep < avg_sal THEN 'lower' 
END AS comparison
FROM cte2 a 
FULL OUTER JOIN cte3 b ON a.pay_month = b.pay_month
ORDER BY a.department_id

"""


make_select(query)

,pay_month,department_id,comparison
0,2017-02,1,same
1,2017-03,1,higher
2,2017-02,2,same
3,2017-03,2,lower


In [150]:
data1 = {
    'student_id': [17, 1, 4, 14, 15, 9, 21, 20, 16, 7, 2, 6, 13, 3, 5],
    'score': [66, 59, 85, 92, 51, 38, 80, 91, 58, 45, 30, 73, 44, 97, 42]
}

# Data for the second table
data2 = {
    'student_id': [11, 16, 10, 14, 5, 21, 3, 13, 15, 1, 8, 20, 4, 2, 6],
    'score': [98, 54, 13, 57, 91, 70, 27, 13, 86, 97, 36, 5, 69, 98, 15]
}

new_york = pd.DataFrame(data1)
california = pd.DataFrame(data2)

query = """  

WITH cte AS (SELECT 
*,
CASE WHEN student_id THEN 'New York University' END AS winner
FROM new_york
WHERE score >= 90
UNION ALL 
SELECT
*,
CASE WHEN student_id THEN 'California University' END AS winner
FROM california
WHERE score >= 90
),


cte2 AS (SELECT
*,
COUNT(student_id) OVER(PARTITION BY winner) AS score_count
FROM cte),

cte3 AS (SELECT 
DISTINCT winner
FROM cte2
WHERE score_count = (SELECT MAX(score_count) FROM cte2))


SELECT
CASE WHEN COUNT(*) >=2 THEN 'No Winner' ELSE winner END AS winner
FROM cte3




"""

make_select(query)

,winner
0,California University


In [ ]:
data = [[103, 'Australia', 84, 'WhisperingPines'], [737, 'Australia', 85, 'GrapesGalore'], [848, 'Australia', 100, 'HarmonyHill'], [222, 'Hungary', 60, 'MoonlitCellars'], [116, 'USA', 47, 'RoyalVines'], [124, 'USA', 45, "Eagle'sNest"], [648, 'India', 69, 'SunsetVines'], [894, 'USA', 39, 'RoyalVines'], [677, 'USA', 9, 'PacificCrest']]
wineries = pd.DataFrame(data, columns=['id', 'country', 'points', 'winery']).astype({'id':'Int64', 'country':'object', 'points':'Int64', 'winery':'object'})


query = """   

WITH cte AS (SELECT
country, 
winery,
SUM(points) AS points
FROM wineries
GROUP BY country, winery),


cte2 AS (SELECT
*,
DENSE_RANK() OVER(PARTITION BY country ORDER BY points DESC, winery ASC) AS ranking
FROM cte)


SELECT 
    country,
    MAX(CASE WHEN ranking = 1 THEN CONCAT(winery, ' (', points, ')') END) AS top_winery,
     COALESCE(
        MAX(CASE WHEN ranking = 2 THEN CONCAT(winery, ' (', points, ')') END),
        'No second winery'
    ) AS second_winery,
    COALESCE(
        MAX(CASE WHEN ranking = 3 THEN CONCAT(winery, ' (', points, ')') END),
        'No third winery'
    ) AS third_winery 
FROM cte2
WHERE ranking <= 3
GROUP BY country

"""

make_select(query)

,country,top_winery,second_winery,third_winery
0,Australia,HarmonyHill (100),GrapesGalore (85),WhisperingPines (84)
1,Hungary,MoonlitCellars (60),No second winery,No third winery
2,India,SunsetVines (69),No second winery,No third winery
3,USA,RoyalVines (86),Eagle'sNest (45),PacificCrest (9)


In [ ]:
candidate_data = {
    'candidate_id': [1, 2, 2, 2, 3, 3, 3, 4, 4, 5, 5, 5, 5],
    'skill': ['Python', 'Python', 'PostgreSQL', 'TensorFlow', 'Spark', 'Tableau', 'Python', 'Spark', 'TensorFlow', 'Python', 'PostgreSQL', 'Tableau', 'TensorFlow'],
    'proficiency': [5, 2, 4, 2, 4, 5, 1, 2, 1, 5, 4, 1, 4]
}
candidates = pd.DataFrame(candidate_data)

# Project Skills DataFrame
project_data = {
    'project_id': [1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 3, 3, 4, 4, 4, 4, 5, 5, 5, 5, 5, 5],
    'skill': ['Spark', 'TensorFlow', 'Tableau', 'PostgreSQL', 'R', 'Python', 'PostgreSQL', 'R', 'Tableau', 'TensorFlow', 'Python', 'Spark', 'PostgreSQL', 'R', 'Spark', 'Tableau', 'PostgreSQL', 'R', 'TensorFlow', 'PostgreSQL', 'Spark', 'R', 'Tableau', 'Python'],
    'importance': [3, 2, 2, 1, 2, 2, 4, 4, 3, 1, 4, 4, 4, 5, 5, 3, 1, 3, 5, 2, 4, 1, 1, 1]
}
projects = pd.DataFrame(project_data)


query = """  

SELECT 
p.project_id, c.candidate_id,
    100 + SUM(
        CASE 
            WHEN c.proficiency > p.importance THEN 10
            WHEN c.proficiency < p.importance THEN -5
            ELSE 0
        END
    ) AS score
FROM projects p 
LEFT JOIN candidates c ON p.skill = c.skill
GROUP BY p.project_id, c.candidate_id
HAVING COUNT(DISTINCT c.skill) = (SELECT
COUNT(DISTINCT sub.skill)
FROM candidates sub
WHERE sub.candidate_id = c.candidate_id
GROUP BY sub.candidate_id)

AND COUNT(DISTINCT p.skill) = (

SELECT COUNT(DISTINCT sub_p.skill)
FROM projects sub_p
WHERE sub_p.project_id = p.project_id
GROUP BY sub_p.project_id
)




"""

make_select(query)

,project_id,candidate_id,score


In [290]:
data = [[10, 2], [15, 1], [20, 3], [25, 1], [30, 1], [35, 2], [40, 3], [45, 1], [50, 2]]
players = pd.DataFrame(data, columns=['player_id', 'group_id']).astype({'player_id':'Int64', 'group_id':'Int64'})
data = [[1, 15, 45, 3, 0], [2, 30, 25, 1, 2], [3, 30, 15, 2, 0], [4, 40, 20, 5, 2], [5, 35, 50, 1, 1]]
matches = pd.DataFrame(data, columns=['match_id', 'first_player', 'second_player', 'first_score', 'second_score']).astype({'match_id':'Int64', 'first_player':'Int64', 'second_player':'Int64', 'first_score':'Int64', 'second_score':'Int64'})


query = """  

WITH cte AS (SELECT * FROM matches
UNION ALL
SELECT match_id, second_player, first_player, second_score, first_score FROM matches),

cte2 AS (SELECT
first_player AS player_id, SUM(first_score) AS score
FROM cte
GROUP BY first_player),


cte3 AS (SELECT
p.player_id, p.group_id, c.score,
DENSE_RANK() OVER(PARTITION BY p.group_id ORDER BY c.score DESC,p.player_id) AS winner
FROM players p 
INNER JOIN cte2 c ON p.player_id = c.player_id
ORDER BY p.group_id)


SELECT
DISTINCT group_id, player_id 
FROM cte3
WHERE winner = 1

"""

make_select(query)

,group_id,player_id
0,1,15
1,2,35
2,3,40


In [300]:
data = [[0, 7], [1, 1], [2, 3], [3, 1]]
numbers = pd.DataFrame(data, columns=['num', 'frequency']).astype({'num':'Int64', 'frequency':'Int64'})


def median_frequency(numbers: pd.DataFrame) -> pd.DataFrame:
    nums = numbers["num"].tolist()
    freq = numbers["frequency"].tolist()
    
    res = []

    for n, f in zip(nums, freq):
        res.extend([n]*f)
    
    return pd.DataFrame({
        "median":[round(np.median(res),0)]
    })


median_frequency(numbers)

,median
0,0.0


In [39]:
data = [[1], [2], [3], [7], [8], [10]]
logs = pd.DataFrame(data, columns=['log_id']).astype({'log_id':'Int64'})


query = """  


WITH cte AS (SELECT
log_id,
ROW_NUMBER() OVER(ORDER BY log_id) AS flag,
log_id - ROW_NUMBER() OVER(ORDER BY log_id) AS rn
FROM logs)


SELECT
MIN(log_id) AS start_id, MAX(log_id) AS end_id
FROM cte
GROUP BY rn


"""

make_select(query)

,start_id,end_id
0,1,3
1,7,8
2,10,10


In [171]:
data = [[1, 1], [2, 0], [3, 1], [4, 1], [5, 1]]
cinema = pd.DataFrame(data, columns=['seat_id', 'free']).astype({'seat_id':'Int64', 'free':'int'})

query = """  


SELECT
seat_id 
FROM cinema 
WHERE free = 1 
AND (
seat_id + 1 IN (SELECT seat_id FROM cinema WHERE free = 1) 
OR  seat_id - 1  IN (SELECT seat_id FROM cinema WHERE free = 1)
)


"""


make_select(query)

,seat_id
0,3
1,4
2,5


In [ ]:
data = [[1, 687, '2021-09-02'], [1, 395, '2021-09-04'], [1, 499, '2021-09-05'], [1, 712, '2021-09-06'], [1, 576, '2021-09-07'], [2, 153, '2021-09-06'], [2, 171, '2021-09-07'], [2, 530, '2021-09-08'], [3, 945, '2021-09-04'], [3, 120, '2021-09-07'], [3, 557, '2021-09-08'], [3, 840, '2021-09-09'], [3, 627, '2021-09-10'], [5, 382, '2021-09-05'], [6, 480, '2021-09-01'], [6, 191, '2021-09-02'], [6, 303, '2021-09-05']]
steps = pd.DataFrame(data, columns=['user_id', 'steps_count', 'steps_date']).astype({'user_id':'Int64', 'steps_count':'Int64', 'steps_date':'datetime64[ns]'})


query = """  

WITH cte AS (
    SELECT 
        user_id,
        steps_date,
        ROUND(AVG(steps_count) 
        OVER (
            PARTITION BY user_id 
            ORDER BY steps_date
            ROWS BETWEEN 2 PRECEDING AND CURRENT ROW
        ), 2) AS rolling_average,
        LAG(steps_date, 2) OVER(PARTITION BY user_id ) AS two_days_ago
    FROM Steps)

SELECT
user_id, 
steps_date, rolling_average
FROM cte
WHERE steps_date - INTERVAL '2 days' = two_days_ago

"""

make_select(query)

In [107]:
data = [[1, '2023-10-01 09:00:00', '2023-10-01 17:00:00'], [1, '2023-10-01 15:00:00', '2023-10-01 23:00:00'], [1, '2023-10-01 16:00:00', '2023-10-02 00:00:00'], [2, '2023-10-01 09:00:00', '2023-10-01 17:00:00'], [2, '2023-10-01 11:00:00', '2023-10-01 19:00:00'], [3, '2023-10-01 09:00:00', '2023-10-01 17:00:00']]
employee_shifts = pd.DataFrame(data, columns=['employee_id', 'start_time', 'end_time']).astype({
    'employee_id': 'Int64',                 # Nullable integer type
    'start_time': 'datetime64[ns]',         # Datetime for start time
    'end_time': 'datetime64[ns]'            # Datetime for end time
})

query = """   


WITH cte AS (SELECT
a.employee_id,
CASE WHEN COUNT(*) =1 THEN COUNT(*) + 1 ELSE COUNT(*) END AS overlapping_shift,
SUM((julianday(b.end_time) - julianday(a.start_time)) * 24 * 60) AS total_overlap_duration
FROM employee_shifts a
JOIN employee_shifts b ON a.employee_id = b.employee_id
AND a.start_time > b.start_time AND a.start_time < b.end_time
GROUP BY 1)


SELECT
DISTINCT a.employee_id,
COALESCE(b.overlapping_shift,1) AS overlapping_shift,
COALESCE(b.total_overlap_duration, 0) AS total_overlap_duration
FROM employee_shifts a 
LEFT JOIN cte b ON a.employee_id = b.employee_id
ORDER BY a.employee_id



"""

make_select(query)

,employee_id,overlapping_shift,total_overlap_duration
0,1,3,600.000001
1,2,2,360.000000
2,3,1,0.000000


In [144]:
data = [[1, '2019-01-01', 'Lenovo'], [2, '2019-02-09', 'Samsung'], [3, '2019-01-19', 'LG'], [4, '2019-05-21', 'HP']]
users = pd.DataFrame(data, columns=['user_id', 'join_date', 'favorite_brand']).astype({'user_id':'Int64', 'join_date':'datetime64[ns]', 'favorite_brand':'object'})

data = [[1, '2019-08-01', 4, 1, 2], [2, '2019-08-02', 2, 1, 3], [3, '2019-08-03', 3, 2, 3], [4, '2019-08-04', 1, 4, 2], [5, '2019-08-04', 1, 3, 4], [6, '2019-08-05', 2, 2, 4]]
orders = pd.DataFrame(data, columns=['order_id', 'order_date', 'item_id', 'buyer_id', 'seller_id']).astype({'order_id':'Int64', 'order_date':'datetime64[ns]', 'item_id':'Int64', 'buyer_id':'Int64', 'seller_id':'Int64'})
data = [[1, 'Samsung'], [2, 'Lenovo'], [3, 'LG'], [4, 'HP']]
items = pd.DataFrame(data, columns=['item_id', 'item_brand']).astype({'item_id':'Int64', 'item_brand':'object'})



query = """  

WITH cte AS (SELECT
o.order_date, u.user_id, u.favorite_brand, i.item_brand,
ROW_NUMBER() OVER(PARTITION BY u.user_id ORDER BY o.order_date) AS order_num
FROM orders o
INNER JOIN users u ON o.seller_id = u.user_id
INNER JOIN items i ON o.item_id = i.item_id
ORDER BY u.user_id,o.order_date),

final AS (SELECT
DISTINCT user_id,
CASE WHEN favorite_brand = item_brand THEN 'Yes' ELSE 'No' END AS ff
FROM cte
WHERE order_num = 2
GROUP BY 1),


cte3 AS (SELECT
u.user_id,
COUNT(*) AS order_num
FROM users u 
LEFT JOIN orders o ON u.user_id = o.seller_id
GROUP BY u.user_id),


cte4 AS (SELECT
DISTINCT user_id ,
CASE WHEN order_num THEN 'No' END order_num
FROM cte3 WHERE order_num = 1)



SELECT user_id, order_num AS "2nd_item_fav_brand" FROM cte4
UNION ALL 
SELECT * FROM final

"""

make_select(query)

,user_id,2nd_item_fav_brand
0,1,No
1,2,Yes
2,3,Yes
3,4,No


In [186]:
# data = {
#     'employee_id': [19, 12, 18, 10, 2, 1, 6, 17, 7, 15, 5, 20, 3, 11],
#     'experience': ['Senior', 'Junior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Senior', 'Junior', 'Junior', 'Junior'],
#     'salary': [88627, 5728, 59519, 12023, 40597, 73928, 37917, 4215, 13567, 84221, 94573, 3581, 971, 62]
# }


data = {
    'employee_id': [1, 9, 2, 11, 13, 4],
    'experience': ['Junior', 'Junior', 'Senior', 'Senior', 'Senior', 'Junior'],
    'salary': [25000, 10000, 85000, 80000, 90000, 30000]
}

# Create DataFrame
candidates = pd.DataFrame(data)

# SELECT cum_sum FROM seniors ORDER BY cum_sum DESC LIMIT 1

query = """  

WITH cte AS (SELECT
*,
ROW_NUMBER() OVER(ORDER BY salary) AS rn
FROM candidates
WHERE experience = 'Senior'
ORDER BY salary),

cte2 AS (SELECT
*,
SUM(salary) OVER(ORDER BY rn) AS cum_sum
FROM cte),


seniors AS (SELECT
*
FROM cte2
WHERE cum_sum <= 70000),



cte3 AS (SELECT
*,
ROW_NUMBER() OVER(ORDER BY salary) AS rn
FROM candidates
WHERE experience = 'Junior'
ORDER BY salary),


cte4 AS (SELECT
*,
SUM(salary) OVER(ORDER BY rn) AS cum_sum
FROM cte3)



SELECT 
DISTINCT employee_id
FROM seniors
UNION ALL
SELECT
DISTINCT employee_id
FROM cte4
WHERE cum_sum <= 70000 - COALESCE((SELECT cum_sum FROM seniors ORDER BY cum_sum DESC LIMIT 1), 0)



"""

make_select(query)

,employee_id
0,9
1,1
2,4


In [249]:
data = [[1, 'Junior', 10000], [9, 'Junior', 10000], [2, 'Senior', 20000], [11, 'Senior', 20000], [13, 'Senior', 50000], [4, 'Junior', 40000]]
candidates = pd.DataFrame(data, columns=['employee_id', 'experience', 'salary']).astype({'employee_id':'Int64', 'experience':'object', 'salary':'Int64'})





query = """  

WITH cte AS (SELECT
*,
SUM(salary) OVER(ORDER BY rn) AS cum_sum
FROM (SELECT
*,
ROW_NUMBER() OVER(ORDER BY salary) AS rn
FROM candidates
WHERE experience = 'Senior' 
ORDER BY salary) AS sub),


cte2 AS (SELECT
*
FROM cte
WHERE cum_sum <= 70000),


cte3 AS (SELECT
*,
SUM(salary) OVER(ORDER BY rn) AS cum_sum
FROM (SELECT
*,
ROW_NUMBER() OVER(ORDER BY salary) AS rn
FROM candidates
WHERE experience = 'Junior' 
ORDER BY salary) AS sub),



final AS (SELECT
DISTINCT experience, COUNT(*) AS accepted_candidates
FROM cte3
WHERE cum_sum <= 70000 - COALESCE((SELECT cum_sum FROM cte2 ORDER BY cum_sum DESC LIMIT 1),0)
UNION ALL
SELECT DISTINCT experience, COUNT(*)
FROM cte2),



frame AS (SELECT 'Senior' AS experience
UNION ALL
SELECT 'Junior' AS experience)


SELECT
f.experience, 
COALESCE(l.accepted_candidates, 0) AS accepted_candidates
FROM frame f
FULL OUTER JOIN final l ON f.experience = l.experience

"""

make_select(query)

,experience,accepted_candidates
0,Senior,2
1,Junior,2


In [ ]:
data = {
    'task_id': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15],
    'employee_id': [7001, 7001, 7001, 7001, 7001, 7002, 7002, 7002, 7002, 7003, 7003, 7003, 7003, 7003, 7003],
    'start_time': [
        '2023-11-01 08:00:00', '2023-11-01 09:30:00', '2023-11-01 11:00:00', 
        '2023-11-01 12:30:00', '2023-11-01 14:00:00', '2023-11-01 08:00:00', 
        '2023-11-01 10:00:00', '2023-11-01 12:00:00', '2023-11-01 14:00:00', 
        '2023-11-01 09:00:00', '2023-11-01 11:00:00', '2023-11-01 13:00:00', 
        '2023-11-01 14:00:00', '2023-11-01 15:30:00', '2023-11-01 17:00:00'
    ],
    'end_time': [
        '2023-11-01 10:00:00', '2023-11-01 11:30:00', '2023-11-01 13:00:00', 
        '2023-11-01 14:30:00', '2023-11-01 16:00:00', '2023-11-01 10:00:00', 
        '2023-11-01 12:00:00', '2023-11-01 14:00:00', '2023-11-01 16:00:00', 
        '2023-11-01 12:00:00', '2023-11-01 13:00:00', '2023-11-01 15:00:00', 
        '2023-11-01 16:00:00', '2023-11-01 17:30:00', '2023-11-01 19:00:00'
    ]
}


tasks = pd.DataFrame(data)
tasks["start_time"] = pd.to_datetime(tasks["start_time"])
tasks["end_time"] = pd.to_datetime(tasks["end_time"])


query = """  

WITH cte AS (SELECT
a.employee_id, 
a.task_id,
CASE WHEN COUNT(*) = 1 THEN COUNT(*) +1 ELSE COUNT(*) END AS total,
SUM((julianday(b.end_time) - julianday(a.start_time)) * 24 * 60) AS duration
FROM tasks a
JOIN tasks b ON a.employee_id = b.employee_id
AND a.start_time > b.start_time AND a.start_time < b.end_time
GROUP BY 1, 2),

cte2 AS (SELECT
employee_id,
SUM((julianday(end_time) - julianday(start_time)) * 24 * 60) AS total_min
FROM tasks
GROUP BY 1),


cte3 AS (SELECT
employee_id, 
MAX(total) AS total , SUM(duration) AS duration
FROM cte
GROUP BY 1)


SELECT
    DISTINCT b.employee_id,
    (COALESCE(b.total_min, 0) - COALESCE(a.duration, 0))/ 60 AS duration,
    COALESCE(a.total, 1) AS max_concurrent_tasks
FROM cte3 a 
RIGHT JOIN cte2 b ON a.employee_id = b.employee_id
ORDER BY 1

"""


make_select(query)

,employee_id,duration,max_concurrent_tasks
0,7001,8.0,2
1,7002,8.0,1
2,7003,10.0,2


In [2]:
data = [[101, '2023-11-06 13:53:42', '2023-11-06 14:05:42', 375, 'Viewer'], [101, '2023-11-22 16:45:21', '2023-11-22 20:39:21', 594, 'Streamer'], [102, '2023-11-16 13:23:09', '2023-11-16 16:10:09', 777, 'Streamer'], [102, '2023-11-17 13:23:09', '2023-11-17 16:10:09', 778, 'Streamer'], [101, '2023-11-20 07:16:06', '2023-11-20 08:33:06', 315, 'Streamer'], [104, '2023-11-27 03:10:49', '2023-11-27 03:30:49', 797, 'Viewer'], [103, '2023-11-27 03:10:49', '2023-11-27 03:30:49', 798, 'Streamer']]
sessions = pd.DataFrame(data, columns=['user_id', 'session_start', 'session_end', 'session_id', 'session_type']).astype({'user_id':'Int64', 'session_start':'datetime64[ns]', 'session_end':'datetime64[ns]', 'session_id':'Int64', 'session_type':'object'})


query = """ 

WITH cte AS (SELECT
*,
MIN(session_start) OVER(PARTITION BY user_id) AS fr,
MIN(CASE WHEN session_type = 'Viewer' THEN session_start END) OVER(PARTITION BY user_id) AS mn
FROM sessions
ORDER BY user_id),

cte2 AS (
SELECT
*,
ROW_NUMBER() OVER(PARTITION BY user_id) AS rn
FROM cte
WHERE mn IS NOT NULL AND fr = mn)



SELECT
user_id, 
COUNT(CASE WHEN session_type = 'Streamer' THEN rn END) AS sessions_count
FROM cte2
WHERE rn > 1
GROUP BY user_id


"""

make_select(query)

,user_id,sessions_count
0,101,2


In [ ]:
data = [[101, '2023-11-01 08:00:00', '2023-11-01 09:00:00', 1, 'Viewer'], [101, '2023-11-01 10:00:00', '2023-11-01 11:00:00', 2, 'Streamer'], [102, '2023-11-01 13:00:00', '2023-11-01 14:00:00', 3, 'Viewer'], [102, '2023-11-01 15:00:00', '2023-11-01 16:00:00', 4, 'Viewer'], [101, '2023-11-02 09:00:00', '2023-11-02 10:00:00', 5, 'Viewer'], [102, '2023-11-02 12:00:00', '2023-11-02 13:00:00', 6, 'Streamer'], [101, '2023-11-02 13:00:00', '2023-11-02 14:00:00', 7, 'Streamer'], [102, '2023-11-02 16:00:00', '2023-11-02 17:00:00', 8, 'Viewer'], [103, '2023-11-01 08:00:00', '2023-11-01 09:00:00', 9, 'Viewer'], [103, '2023-11-02 20:00:00', '2023-11-02 23:00:00', 10, 'Viewer'], [103, '2023-11-03 09:00:00', '2023-11-03 10:00:00', 11, 'Viewer']]
sessions = pd.DataFrame(data, columns=['user_id', 'session_start', 'session_end', 'session_id', 'session_type']).astype({'user_id': 'Int64', 'session_start': 'datetime64[ns]', 'session_end': 'datetime64[ns]', 'session_id': 'Int64', 'session_type': 'object'})

# WHERE prev_type = session_type AND (julianday(prev_date) - julianday(session_start))* 24 < 12

query = """  

WITH cte AS (SELECT
*,
LAG(session_end) OVER(PARTITION BY user_id, session_type ORDER BY session_end) as prev_type
FROM sessions)

SELECT
*
FROM cte
WHERE session_start - prev_type <12

"""

make_select(query)

,user_id,session_start,session_end,session_id,session_type,prev_type
0,101,2023-11-01 10:00:00.000000,2023-11-01 11:00:00.000000,2,Streamer,None
1,101,2023-11-02 13:00:00.000000,2023-11-02 14:00:00.000000,7,Streamer,2023-11-01 11:00:00.000000
2,101,2023-11-01 08:00:00.000000,2023-11-01 09:00:00.000000,1,Viewer,None
3,101,2023-11-02 09:00:00.000000,2023-11-02 10:00:00.000000,5,Viewer,2023-11-01 09:00:00.000000
4,102,2023-11-02 12:00:00.000000,2023-11-02 13:00:00.000000,6,Streamer,None
5,102,2023-11-01 13:00:00.000000,2023-11-01 14:00:00.000000,3,Viewer,None
6,102,2023-11-01 15:00:00.000000,2023-11-01 16:00:00.000000,4,Viewer,2023-11-01 14:00:00.000000
7,102,2023-11-02 16:00:00.000000,2023-11-02 17:00:00.000000,8,Viewer,2023-11-01 16:00:00.000000
8,103,2023-11-01 08:00:00.000000,2023-11-01 09:00:00.000000,9,Viewer,None
9,103,2023-11-02 20:00:00.000000,2023-11-02 23:00:00.000000,10,Viewer,2023-11-01 09:00:00.000000


In [ ]:
data = [[1374, 'prime_eligible', 'Watches', 68.0], [4245, 'not_prime', 'Art', 26.4], [5743, 'prime_eligible', 'Software', 325.0], [8543, 'not_prime', 'Clothing', 64.5], [2556, 'not_prime', 'Shoes', 15.0], [2452, 'prime_eligible', 'Scientific', 85.0], [3255, 'not_prime', 'Furniture', 22.6], [1672, 'prime_eligible', 'Beauty', 8.5], [4256, 'prime_eligible', 'Furniture', 55.5], [6325, 'prime_eligible', 'Food', 13.2]]
inventory = pd.DataFrame(data, columns=['item_id', 'item_type', 'item_category', 'square_footage']).astype({'item_id':'Int64', 'item_type':'object', 'item_category':'object', 'square_footage':'Float64'})


query = """  


WITH cte AS (SELECT
item_type,
COUNT(*) *
FLOOR(500000 / SUM(square_footage)) AS item_count,
FLOOR(SUM(square_footage) * FLOOR(500000 / SUM(square_footage))) AS remaining
FROM inventory
WHERE item_type = 'prime_eligible'
GROUP BY 1),


cte2 AS (SELECT
item_type,
COUNT(*)*
FLOOR((500000 - (SELECT remaining FROM cte)) / 
SUM(square_footage)) AS item_count
FROM inventory
WHERE item_type = 'not_prime'
GROUP BY 1)


SELECT item_type, item_count FROM cte
UNION ALL
SELECT item_type, item_count FROM cte2
ORDER BY item_count DESC



"""

make_select(query)

,item_type,item_count
0,prime_eligible,5400
1,not_prime,8


In [ ]:
data = [[1, 10, '2021-03-15'], [1, 11, '2021-03-15'], [1, 12, '2021-03-15'], [2, 10, '2021-03-15'], [2, 11, '2021-03-15'], [2, 12, '2021-03-15'], [3, 10, '2021-03-15'], [3, 11, '2021-03-15'], [3, 12, '2021-03-15'], [4, 10, '2021-03-15'], [4, 11, '2021-03-15'], [4, 13, '2021-03-15'], [5, 10, '2021-03-16'], [5, 11, '2021-03-16'], [5, 12, '2021-03-16']]
listens = pd.DataFrame(data, columns=['user_id', 'song_id', 'day']).astype({'user_id':'Int64', 'song_id':'Int64', 'day':'datetime64[ns]'})
data = [[1, 2]]
friendship = pd.DataFrame(data, columns=['user1_id', 'user2_id']).astype({'user1_id':'Int64', 'user2_id':'Int64'})


query = """   

WITH cte AS (SELECT
*
FROM friendship
UNION ALL
SELECT user2_id, user1_id FROM friendship)



SELECT
DISTINCT a.user_id AS user_id, b.user_id AS recommended_id
FROM listens a, listens b
WHERE a.day = b.day AND a.song_id = b.song_id AND a.user_id != b.user_id
AND (a.user_id, b.user_id) NOT IN (SELECT DISTINCT user1_id, user2_id FROM cte)
GROUP BY a.user_id, b.user_id
HAVING COUNT(DISTINCT a.song_id) >= 3


"""

make_select(query)

,user_id,recommended_id
0,1,3
1,2,3
2,3,1
3,3,2


In [55]:
data = [[1, 10, '2021-03-15'], [1, 11, '2021-03-15'], [1, 12, '2021-03-15'], [2, 10, '2021-03-15'], [2, 11, '2021-03-15'], [2, 12, '2021-03-15'], [3, 10, '2021-03-15'], [3, 11, '2021-03-15'], [3, 12, '2021-03-15'], [4, 10, '2021-03-15'], [4, 11, '2021-03-15'], [4, 13, '2021-03-15'], [5, 10, '2021-03-16'], [5, 11, '2021-03-16'], [5, 12, '2021-03-16']]
listens = pd.DataFrame(data, columns=['user_id', 'song_id', 'day']).astype({'user_id':'Int64', 'song_id':'Int64', 'day':'datetime64[ns]'})
data = [[1, 2], [2, 4], [2, 5]]
friendship = pd.DataFrame(data, columns=['user1_id', 'user2_id']).astype({'user1_id':'Int64', 'user2_id':'Int64'})


query = """

WITH cte AS (SELECT user1_id, user2_id FROM friendship
UNION ALL
SELECT user2_id, user1_id FROM friendship),


cte2 AS (SELECT
DISTINCT a.user_id AS user1_id, b.user_id AS user2_id, a.day
FROM listens a
INNER JOIN listens b ON a.day = b.day AND a.song_id = b.song_id AND a.user_id != b.user_id
AND (a.user_id, b.user_id) IN (SELECT DISTINCT user1_id, user2_id FROM friendship)
GROUP BY 1, 2, 3
HAVING COUNT(DISTINCT a.song_id) >= 3)


SELECT DISTINCT user1_id, user2_id
FROM cte2

"""

make_select(query)

,user1_id,user2_id
0,1,2


In [108]:
data = {
    "order_id": [3, 11, 10, 9, 20, 8, 19, 21, 4, 12, 15, 14, 24, 23, 17, 1, 13, 5, 7, 2, 18, 22, 16, 6],
    "customer_id": [1, 6, 6, 5, 1, 1, 3, 3, 3, 3, 2, 1, 5, 6, 6, 6, 3, 6, 2, 1, 4, 5, 4, 1],
    "order_date": [
        "2020-07-01", "2021-04-16", "2019-10-11", "2022-02-11", "2022-02-14", "2021-09-20",
        "2020-11-17", "2019-02-06", "2019-12-16", "2020-01-30", "2020-09-26", "2020-12-13",
        "2022-03-01", "2020-09-21", "2022-05-13", "2019-04-21", "2022-03-16", "2020-12-27",
        "2021-12-15", "2021-04-10", "2021-02-06", "2019-09-23", "2020-07-12", "2022-03-23"
    ],
    "price": [
        1100, 6700, 5400, 7100, 3000, 500, 400, 5800, 4500, 9800, 3600, 500, 7000, 4700, 2100,
        9600, 900, 900, 4900, 3300, 7700, 3500, 6300, 7700
    ]
}

orders = pd.DataFrame(data)

query = """  

WITH cte AS (
SELECT
customer_id, strftime('%Y', order_date) AS year, 
SUM(price) AS total_amount
FROM orders
GROUP BY 1,2),


cte2 AS (SELECT
*,
LAG(year) OVER(PARTITION BY customer_id ORDER BY year) AS prev,
LAG(total_amount) OVER(PARTITION BY customer_id ORDER BY year) AS prev_amount
FROM cte)


SELECT
    customer_id
FROM cte2
GROUP BY customer_id
HAVING SUM(CASE WHEN year - prev = 1 AND total_amount > prev_amount THEN 1 ELSE 0 END) = COUNT(*) - 1


"""

make_select(query)

,customer_id
0,1
1,2
2,4


In [153]:
data = [[10, '2019-12-10'], [8, '2020-1-13'], [5, '2020-2-16'], [7, '2020-3-8'], [4, '2020-5-17'], [1, '2020-10-24'], [6, '2021-1-5']]
drivers = pd.DataFrame(data, columns=['driver_id', 'join_date']).astype({'driver_id':'Int64', 'join_date':'datetime64[ns]'})
data = [[6, 75, '2019-12-9'], [1, 54, '2020-2-9'], [10, 63, '2020-3-4'], [19, 39, '2020-4-6'], [3, 41, '2020-6-3'], [13, 52, '2020-6-22'], [7, 69, '2020-7-16'], [17, 70, '2020-8-25'], [20, 81, '2020-11-2'], [5, 57, '2020-11-9'], [2, 42, '2020-12-9'], [11, 68, '2021-1-11'], [15, 32, '2021-1-17'], [12, 11, '2021-1-19'], [14, 18, '2021-1-27']]
rides = pd.DataFrame(data, columns=['ride_id', 'user_id', 'requested_at']).astype({'ride_id':'Int64', 'user_id':'Int64', 'requested_at':'datetime64[ns]'})
data = [[10, 10, 63, 38], [13, 10, 73, 96], [7, 8, 100, 28], [17, 7, 119, 68], [20, 1, 121, 92], [5, 7, 42, 101], [2, 4, 6, 38], [11, 8, 37, 43], [15, 8, 108, 82], [12, 8, 38, 34], [14, 1, 90, 74]]
accepted_rides = pd.DataFrame(data, columns=['ride_id', 'driver_id', 'ride_distance', 'ride_duration']).astype({'ride_id':'Int64', 'driver_id':'Int64', 'ride_distance':'Int64', 'ride_duration':'Int64'})


query = """

WITH months AS (SELECT 1 AS month
UNION
SELECT 2
UNION
SELECT 3
UNION
SELECT 4
UNION
SELECT 5
UNION
SELECT 6
UNION
SELECT 7
UNION
SELECT 8
UNION
SELECT 9
UNION
SELECT 10
UNION
SELECT 11
UNION
SELECT 12),


active_users AS (SELECT 
*,
strftime('%m', join_date) AS month,
COUNT(driver_id) OVER(ORDER BY strftime('%Y-%m', join_date)) AS num
FROM drivers
WHERE strftime('%Y-%m' , join_date) BETWEEN '2019-12' AND '2020-12'),


cte3 AS (SELECT CAST(month AS INTERGER) AS month, num FROM active_users WHERE strftime('%Y-%m', join_date) >= '2020-01'),


final_drivers AS (SELECT
    m.month,
    COALESCE(
        c.num,
        MAX(c.num) OVER (
            PARTITION BY 1
            ORDER BY m.month
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )
    ) AS active_drivers
FROM months m
LEFT JOIN cte3 c ON m.month = c.month
ORDER BY m.month),


acc_rides AS (SELECT
CAST(strftime('%m', r.requested_at) AS INTEGER) AS month,
COUNT(*) AS num
FROM rides r
INNER JOIN accepted_rides a ON r.ride_id = a.ride_id
WHERE strftime('%Y', r.requested_at) = '2020'
GROUP BY 1)


SELECT
f.month, f.active_drivers, COALESCE(a.num, 0) AS accepted_rides
FROM final_drivers f 
LEFT JOIN acc_rides a ON f.month = a.month


"""

make_select(query)

,month,active_drivers,accepted_rides
0,1,2,0
1,2,3,0
2,3,4,1
3,4,4,0
4,5,5,0
5,6,5,1
6,7,5,1
7,8,5,1
8,9,5,0
9,10,6,0


In [157]:
data = [[10, '2019-12-10'], [8, '2020-1-13'], [5, '2020-2-16'], [7, '2020-3-8'], [4, '2020-5-17'], [1, '2020-10-24'], [6, '2021-1-5']]
drivers = pd.DataFrame(data, columns=['driver_id', 'join_date']).astype({'driver_id':'Int64', 'join_date':'datetime64[ns]'})
data = [[6, 75, '2019-12-9'], [1, 54, '2020-2-9'], [10, 63, '2020-3-4'], [19, 39, '2020-4-6'], [3, 41, '2020-6-3'], [13, 52, '2020-6-22'], [7, 69, '2020-7-16'], [17, 70, '2020-8-25'], [20, 81, '2020-11-2'], [5, 57, '2020-11-9'], [2, 42, '2020-12-9'], [11, 68, '2021-1-11'], [15, 32, '2021-1-17'], [12, 11, '2021-1-19'], [14, 18, '2021-1-27']]
rides = pd.DataFrame(data, columns=['ride_id', 'user_id', 'requested_at']).astype({'ride_id':'Int64', 'user_id':'Int64', 'requested_at':'datetime64[ns]'})
data = [[10, 10, 63, 38], [13, 10, 73, 96], [7, 8, 100, 28], [17, 7, 119, 68], [20, 1, 121, 92], [5, 7, 42, 101], [2, 4, 6, 38], [11, 8, 37, 43], [15, 8, 108, 82], [12, 8, 38, 34], [14, 1, 90, 74]]
accepted_rides = pd.DataFrame(data, columns=['ride_id', 'driver_id', 'ride_distance', 'ride_duration']).astype({'ride_id':'Int64', 'driver_id':'Int64', 'ride_distance':'Int64', 'ride_duration':'Int64'})



query = """

WITH months AS (SELECT 1 AS month
UNION
SELECT 2
UNION
SELECT 3
UNION
SELECT 4
UNION
SELECT 5
UNION
SELECT 6
UNION
SELECT 7
UNION
SELECT 8
UNION
SELECT 9
UNION
SELECT 10
UNION
SELECT 11
UNION
SELECT 12),


active_users AS (SELECT 
*,
strftime('%m', join_date) AS month,
COUNT(driver_id) OVER(ORDER BY strftime('%Y-%m', join_date)) AS num
FROM drivers
WHERE strftime('%Y-%m' , join_date) BETWEEN '2019-12' AND '2020-12'),


cte3 AS (SELECT CAST(month AS INTERGER) AS month, num FROM active_users WHERE strftime('%Y-%m', join_date) >= '2020-01'),


final_drivers AS (SELECT
    m.month,
    COALESCE(
        c.num,
        MAX(c.num) OVER (
            PARTITION BY 1
            ORDER BY m.month
            ROWS BETWEEN UNBOUNDED PRECEDING AND CURRENT ROW
        )
    ) AS active_drivers
FROM months m
LEFT JOIN cte3 c ON m.month = c.month
ORDER BY m.month),


acc_rides AS (SELECT
CAST(strftime('%m', r.requested_at) AS INTEGER) AS month,
COUNT(*) AS num
FROM rides r
INNER JOIN accepted_rides a ON r.ride_id = a.ride_id
WHERE strftime('%Y', r.requested_at) = '2020'
GROUP BY 1)


SELECT
f.month,  ROUND(COALESCE(a.num, 0) * 1.0 /f.active_drivers * 100, 2)
FROM final_drivers f 
LEFT JOIN acc_rides a ON f.month = a.month


"""

make_select(query)

,month,"ROUND(COALESCE(a.num, 0) * 1.0 /f.active_drivers * 100, 2)"
0,1,0.00
1,2,0.00
2,3,25.00
3,4,0.00
4,5,0.00
5,6,20.00
6,7,20.00
7,8,20.00
8,9,0.00
9,10,0.00


In [190]:
data = [[1, 20, '2019-08-14'], [2, 50, '2019-08-14'], [1, 30, '2019-08-15'], [1, 35, '2019-08-16'], [2, 65, '2019-08-17'], [3, 20, '2019-08-18']]
products = pd.DataFrame(data, columns=['product_id', 'new_price', 'change_date']).astype({'product_id':'Int64', 'new_price':'Int64', 'change_date':'datetime64[ns]'})


query = """  

SELECT *
FROM products
WHERE change_date <= '2019-08-16'

"""

make_select(query)

,product_id,new_price,change_date
0,1,20,2019-08-14 00:00:00.000000
1,2,50,2019-08-14 00:00:00.000000
2,1,30,2019-08-15 00:00:00.000000


In [263]:
data = [[1, 'A', 2341], [2, 'A', 341], [3, 'A', 15], [4, 'A', 15314], [5, 'A', 451], [6, 'A', 513], [7, 'B', 15], [8, 'B', 13], [9, 'B', 1154], [10, 'B', 1345], [11, 'B', 1221], [12, 'B', 234], [13, 'C', 2345], [14, 'C', 2645], [15, 'C', 2645], [16, 'C', 2652], [17, 'C', 65]]
employee = pd.DataFrame(data, columns=['id', 'company', 'salary']).astype({'id':'Int64', 'company':'object', 'salary':'Int64'})

# rn = total / 2.0 OR rn = total / 2.0 + 1
query = """  

WITH cte AS (SELECT
id,company,salary,
ROW_NUMBER() OVER(PARTITION BY company ORDER BY salary) AS rn
FROM employee
ORDER BY company, salary),


cte2 AS (SELECT id,
company, salary,rn, COUNT(rn) OVER(PARTITION BY company) AS total
FROM cte),


evens AS (SELECT
company
FROM employee
GROUP BY 1
HAVING COUNT(*) % 2 = 0),

odds AS (SELECT
company
FROM employee
GROUP BY 1
HAVING COUNT(*) % 2 != 0),


final AS (SELECT
c.id, c.company, c.salary
FROM cte2 c 
INNER JOIN evens e ON c.company = e.company
WHERE c.rn = c.total / 2.0 OR c.rn = c.total / 2.0 + 1)



SELECT * FROM final
UNION ALL
SELECT
c.id, c.company, c.salary
FROM cte2 c
INNER JOIN odds o ON o.company = c.company
WHERE c.rn = ROUND(c.total / 2.0, 0)

"""

make_select(query)

,id,company,salary
0,5,A,451
1,6,A,513
2,9,B,1154
3,12,B,234
4,14,C,2645


In [210]:
products = pd.DataFrame({
    "product_id": [1, 2, 3],
    "LC_Store": [100, None, None],
    "Nozama": [None, 200, None],
    "Shop": [110, None, 1000],
    "Souq": [None, 190, 1900]
})


# products = pd.melt(products, id_vars="product_id", value_vars=[col for col in products.columns.tolist()], value_name="value", var_name="platform")

# products = products.dropna()

def find_valid_users(products: pd.DataFrame) -> pd.DataFrame:
    
    df = pd.melt(
        products, 
        id_vars="product_id", 
        value_vars=[col for col in products.columns.tolist()] , 
        var_name="store", 
        value_name="price")
    
    df = df.dropna()

    return df

find_valid_users(products)

,product_id,store,price
0,1,LC_Store,100.0
4,2,Nozama,200.0
6,1,Shop,110.0
8,3,Shop,1000.0
10,2,Souq,190.0
11,3,Souq,1900.0


In [43]:
data = [['Jane', 'America'], ['Pascal', 'Europe'], ['Xi', 'Asia'], ['Jack', 'America']]
student = pd.DataFrame(data, columns=['name', 'continent']).astype({'name':'object', 'continent':'object'})



query = """  

WITH cte AS (SELECT
CASE WHEN continent = 'America' THEN name END AS America,
CASE WHEN continent = 'Asia' THEN name END AS Asia ,
CASE WHEN continent = 'Europe' THEN name END AS Europe ,
ROW_NUMBER() OVER(PARTITION BY continent ORDER BY name) AS rn
FROM student)


SELECT
MIN(America) AS America,
MIN(Asia) AS Asia, 
MIN(Europe) AS Europe
FROM cte
GROUP BY rn

"""

make_select(query)

,America,Asia,Europe
0,Jack,Xi,Pascal
1,Jane,None,None


In [81]:
data = [[1, '2020-01-01'], [2, '2020-01-02'], [12, '2020-01-01'], [19, '2020-01-03'], [1, '2020-01-02'], [2, '2020-01-03'], [1, '2020-01-04'], [7, '2020-01-11'], [9, '2020-01-25'], [8, '2020-01-28']]
visits = pd.DataFrame(data, columns=['user_id', 'visit_date']).astype({'user_id':'Int64', 'visit_date':'datetime64[ns]'})
data = [[1, '2020-01-02', 120], [2, '2020-01-03', 22], [7, '2020-01-11', 232], [1, '2020-01-04', 7], [9, '2020-01-25', 33], [9, '2020-01-25', 66], [8, '2020-01-28', 1], [9, '2020-01-25', 99]]
transactions = pd.DataFrame(data, columns=['user_id', 'transaction_date', 'amount']).astype({'user_id':'Int64', 'transaction_date':'datetime64[ns]', 'amount':'Int64'})



query = """

WITH cte AS (SELECT
v.user_id, v.visit_date, 
COUNT(t.transaction_date) AS count
FROM visits v
LEFT JOIN transactions t ON v.user_id = t.user_id AND v.visit_date = t.transaction_date
GROUP BY 1, 2),

cte3  AS (SELECT
count, 
COUNT(user_id) AS visit_count
FROM cte
GROUP BY count),


report AS (
    SELECT MIN(count) AS number  
    FROM cte3
    UNION ALL
    SELECT number + 1           
    FROM report
    WHERE number < (SELECT MAX(count) FROM cte3) 
)


SELECT 
r.number AS transactions_count,
COALESCE(c.visit_count, 0) AS visit_count
FROM report r
LEFT JOIN cte3 c ON c.count = r.number

"""

make_select(query)

,transactions_count,visit_count
0,0,4
1,1,5
2,2,0
3,3,1


In [113]:
data = [['2018-12-28'], ['2018-12-29'], ['2019-01-04'], ['2019-01-05']]
failed = pd.DataFrame(data, columns=['fail_date']).astype({'fail_date':'datetime64[ns]'})
data = [['2018-12-30'], ['2018-12-31'], ['2019-01-01'], ['2019-01-02'], ['2019-01-03'], ['2019-01-06']]
succeeded = pd.DataFrame(data, columns=['success_date']).astype({'success_date':'datetime64[ns]'})

query ="""  


WITH cte AS (SELECT
*,
CASE WHEN fail_date IS NOT NULL THEN 'failed' END AS status
FROM failed
WHERE fail_date >= '2019-01-01' AND fail_date <= '2019-12-31'),

cte2 AS (SELECT
*,
CASE WHEN success_date IS NOT NULL THEN 'success' END AS status
FROM succeeded
WHERE success_date >= '2019-01-01' AND success_date <= '2019-12-31'),


cte3 AS (SELECT fail_date AS date, status FROM cte
UNION ALL
SELECT * FROM cte2
ORDER BY date),


cte4 AS (SELECT 
*,
ROW_NUMBER() OVER(ORDER BY date) ,ROW_NUMBER() OVER(PARTITION BY status ORDER BY date) AS grp
FROM cte3),


final AS (SELECT
status,
grp,
MIN(date) AS start_date, 
MAX(date) AS end_date
FROM cte4
GROUP BY 1,2
ORDER BY start_date)


--SELECT status AS period_state, start_date,end_date
--FROM final

SELECT * FROM cte4

"""

make_select(query)

,date,status,ROW_NUMBER() OVER(ORDER BY date),grp
0,2019-01-01 00:00:00.000000,success,1,1
1,2019-01-02 00:00:00.000000,success,2,2
2,2019-01-03 00:00:00.000000,success,3,3
3,2019-01-04 00:00:00.000000,failed,4,1
4,2019-01-05 00:00:00.000000,failed,5,2
5,2019-01-06 00:00:00.000000,success,6,4


In [137]:
data = [[1], [2], [3], [7], [8], [10]]
logs = pd.DataFrame(data, columns=['log_id']).astype({'log_id':'Int64'})


query = """  

WITH cte AS (SELECT
*,
ROW_NUMBER() OVER(ORDER BY log_id) AS num,
log_id - ROW_NUMBER() OVER(ORDER BY log_id) AS cat
FROM logs
)

SELECT
MIN(log_id) AS start_id,
MAX(log_id) AS end_id
FROM cte
GROUP BY cat


"""

make_select(query)

,start_id,end_id
0,1,3
1,7,8
2,10,10


In [164]:
data = [[1, 'Abbot'], [2, 'Doris'], [3, 'Emerson'], [4, 'Green'], [5, 'Jeames']]
seat = pd.DataFrame(data, columns=['id', 'student']).astype({'id':'Int64', 'student':'object'})


query = """  

WITH cte AS (SELECT
*,
LAG(id, 1, 0) OVER(ORDER BY id) AS ids
FROM seat)

SELECT
id, 
CASE WHEN id % 2 = 0 THEN LAG(student) OVER(ORDER BY id) 
ELSE COALESCE(LEAD(student) OVER(ORDER BY id), student) END AS student
FROM cte


"""

make_select(query)


,id,student
0,1,Doris
1,2,Abbot
2,3,Green
3,4,Emerson
4,5,Jeames
